In [ ]:
source("diabetes_analysis_v07.R")

rank_score_func <- function(df){
df <- df %>% mutate(score = -1*log(p_val_adj+(10^-310))*avg_log2FC*(pct.1/(pct.2+10^-300)))
return(df)
}



In [ ]:
library(msigdbr)
library(clusterProfiler)
library(org.Hs.eg.db)
library(enrichplot)
library(EnsDb.Hsapiens.v86)
library(fgsea)

# Load markers, add EnsID

In [ ]:
markers_cd4  <- read_csv("../tables/de_genes/240319_cd4_all_markers_without_sex.csv")
markers_cd8  <- read_csv("../tables/de_genes/240319_cd8_all_markers_without_sex.csv")

In [ ]:
all_markers  <- rbind(markers_cd4, markers_cd8)

In [ ]:
all_markers$source  %>% table

In [ ]:
all_markers  <- all_markers  %>% mutate(
direction = case_when(test_type == "Dia T1 vs Ctrl T0" & cluster == "Ctrl T0" ~ "Down",
                      test_type == "Dia T1 vs Ctrl T0" & cluster == "Dia T1" ~ "Up",
                      
                      test_type == "Dia T0 vs Ctrl T0" & cluster == "Ctrl T0" ~ "Down",
                      test_type == "Dia T0 vs Ctrl T0" & cluster == "Dia T0" ~ "Up",
                      
                      test_type == "Dia T0 vs Dia T1" & cluster == "Dia T0" ~ "Up",
                      test_type == "Dia T0 vs Dia T1" & cluster == "Dia T1" ~ "Down",
                      
                      test_type == "PR_0 T0 vs PR_0 T1" & cluster == "PR_0 T0" ~ "Up",
                      test_type == "PR_0 T0 vs PR_0 T1" & cluster == "PR_0 T1" ~ "Down",
                      
                      test_type == "PR_0 T0 vs PR_1 T0" & cluster == "PR_0 T0" ~ "Up",
                      test_type == "PR_0 T0 vs PR_1 T0" & cluster == "PR_1 T0" ~ "Down",
                      
                      test_type == "PR_0 T1 vs PR_1 T1" & cluster == "PR_0 T1" ~ "Up",
                      test_type == "PR_0 T1 vs PR_1 T1" & cluster == "PR_1 T1" ~ "Down",
                      
                      test_type == "PR_1 T0 vs PR_1 T1" & cluster == "PR_1 T0" ~ "Up",
                      test_type == "PR_1 T0 vs PR_1 T1" & cluster == "PR_1 T1" ~ "Down",
                      
                      test_type == "Keto_1 T0 vs Keto_0 T0" & cluster == "Keto_0 T0" ~ "Down",
                      test_type == "Keto_1 T0 vs Keto_0 T0" & cluster == "Keto_1 T0" ~ "Up"
                      
                     ))

In [ ]:
all_markers  %>% group_by(cluster, test_type, direction)  %>% tally

In [ ]:
geneIDs1 <- ensembldb::select(EnsDb.Hsapiens.v86, keys= (all_markers$gene  %>% unique ), 
                              keytype = "SYMBOL", columns = c("SYMBOL","ENTREZID"))

In [ ]:
colnames(geneIDs1)  <- c("gene", "entrezid")

In [ ]:
all_markers2  <- all_markers  %>% 
                    left_join(geneIDs1) 

In [ ]:
all_markers2

In [ ]:
all_markers$gene_direction  <- paste(all_markers$gene, all_markers$direction)

In [ ]:
all_markers2$gene_direction  <- paste(all_markers2$gene, all_markers2$direction)

## Gene network

### RNA

In [ ]:
 all_markers2  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
group_by(gene_direction)  %>% tally()  %>% arrange(desc(n))  

In [ ]:
global_genes  <- all_markers2  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
group_by(gene)  %>% tally()  %>% dplyr::filter(n>1)  %>% arrange(desc(n))  %>% pull(gene)

In [ ]:
global_genes

In [ ]:
global_genes  %>% length

In [ ]:
lvl  <- all_markers  %>% 
group_by(gene, cluster)  %>% arrange(cluster)  %>% pull(gene)  %>% unique

In [ ]:
is.more.than.one.cluster  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
group_by(gene, direction)  %>% tally  %>% arrange(desc(n))  %>% dplyr::filter(n>1)  %>% pull(gene)

In [ ]:
is.more.than.one.cluster

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
all_markers  %>% 
dplyr::filter(gene %in% is.more.than.one.cluster & p_val_adj < 0.01 &
              source == "scRNAseq_RNA" & direction == "Up")  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene, levels = lvl), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = test_type)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 30)
all_markers  %>% 
dplyr::filter(gene %in% is.more.than.one.cluster & p_val_adj < 0.01 &
              source == "scRNAseq_RNA" & direction == "Down")  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene, levels = lvl), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = test_type)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

# Counts of DE genes

### RNA

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
all_markers  %>% 
group_by(cluster, source, test_type, dataset)  %>% 
dplyr::filter(source == "scRNAseq_RNA")  %>% 
ggplot(aes(x = dataset)) +
geom_bar(aes(fill = test_type), position = "stack") + coord_flip() 

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5)
all_markers  %>% 
group_by(cluster, source, test_type, dataset)  %>% 
dplyr::filter(source == "scRNAseq_RNA" & test_type != "Cpept_HI_T1 vs Cpept_LO_T1")  %>% 
ggplot(aes(x = dataset)) +
facet_grid(cols = vars(test_type)) +
geom_bar(aes(fill = cluster), position = "stack") + coord_flip() 

In [ ]:
all_markers  %>% 
group_by(cluster, source, test_type, dataset)  %>% 
mutate(dataset = stringr::str_to_upper(gsub(dataset, pattern = "_", replacement = " ")))  %>% 
dplyr::filter(source == "scRNAseq_RNA" & test_type != "Cpept_HI_T1 vs Cpept_LO_T1")  %>% 
ggplot(aes(x = dataset)) +
facet_grid(cols = vars(test_type)) +
geom_bar(aes(fill = direction), position = "stack") + coord_flip() +
ggtheme() + 
theme_bw() + 
scale_fill_manual(values = c("#90c2f1ff", "#d0625dff")) + theme(axis.ticks.y = element_blank())


In [ ]:
all_markers$test_type  %>% table

In [ ]:
all_markers  %>% 
group_by(cluster, source, test_type, dataset)  %>% 
mutate(dataset = stringr::str_to_upper(gsub(dataset, pattern = "_", replacement = " ")))  %>% 
dplyr::filter(source == "scRNAseq_RNA" & test_type != "Cpept_HI_T1 vs Cpept_LO_T1" &
             test_type != "PR_0 T0 vs PR_0 T1" & test_type != "PR_0 T1 vs PR_1 T1" &
             test_type != "PR_1 T0 vs PR_1 T1" )  %>% 
ggplot(aes(x = dataset)) +
facet_grid(cols = vars(test_type)) +
geom_bar(aes(fill = direction), position = "stack") + coord_flip() +
ggtheme() + 
theme_bw() + 
scale_fill_manual(values = c("#90c2f1ff", "#d0625dff")) + theme(axis.ticks.y = element_blank())


In [ ]:
library(svglite)
dir.create("../figures/DE_genes/")
ggsave(filename = "../figures/DE_genes/count_of_RNA_markers.svg", width = 11, height = 4)

### CollecTRI

In [ ]:
all_markers  %>% 
group_by(cluster, source, test_type, dataset)  %>% 
dplyr::filter(source == "scRNAseq_collecTRI")  %>% 
ggplot(aes(x = dataset)) +
geom_bar(aes(fill = test_type), position = "stack") + coord_flip() 

In [ ]:
all_markers  %>% 
group_by(cluster, source, test_type, dataset)  %>% 
dplyr::filter(source == "scRNAseq_collecTRI")  %>% 
ggplot(aes(x = dataset)) +
facet_grid(cols = vars(test_type)) +
geom_bar(aes(fill = cluster), position = "stack") + coord_flip() 

In [ ]:
all_markers  %>% dplyr::filter(source == "scRNAseq_collecTRI")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction)  %>% tally  %>% arrange(desc(n))

## Global enrichment

In [ ]:
up_dia_t0_vs_ctrl_t0  <- all_markers2  %>% 
                        dplyr::filter(test_type == "Dia T0 vs Ctrl T0" & cluster == "Dia T0" & source == "scRNAseq_RNA")  %>% 
                        group_by(gene)  %>% 
                        tally()  %>% 
                        arrange(desc(n))  %>% 
                        pull(gene)
down_dia_t0_vs_ctrl_t0  <- all_markers2  %>% 
                        dplyr::filter(test_type == "Dia T0 vs Ctrl T0" & cluster == "Ctrl T0" & source == "scRNAseq_RNA")  %>% 
                        group_by(gene)  %>% 
                        tally()  %>% 
                        arrange(desc(n))  %>% 
                        pull(gene)

In [ ]:
up_dia_t0_vs_ctrl_t0

In [ ]:
ego2 <- enrichGO(gene         = up_dia_t0_vs_ctrl_t0,
                OrgDb         = org.Hs.eg.db,
                keyType       = 'SYMBOL',
                ont           = "ALL",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.01,
                qvalueCutoff  = 0.05)
head(ego2, 3)         

In [ ]:
options(repr.plot.width = 23, repr.plot.height = 7)
edox2 <- pairwise_termsim(ego2)

In [ ]:
edox2

In [ ]:
dir.create("../data/gsea/")

In [ ]:
saveRDS(edox2, "../data/gsea/up_dia_t0_vs_ctrl_t0.rds")


In [ ]:
ego2 <- enrichGO(gene         = down_dia_t0_vs_ctrl_t0,
                OrgDb         = org.Hs.eg.db,
                keyType       = 'SYMBOL',
                ont           = "ALL",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.01,
                qvalueCutoff  = 0.05)
head(ego2, 3)   

In [ ]:
edox2 <- pairwise_termsim(ego2)
saveRDS(edox2, "../data/gsea/down_dia_t0_vs_ctrl_t0.rds")


# DE genes T0 and T1 in clusters

In [ ]:
all_markers$test_type  %>% table

In [ ]:
datasets  <- levels(factor(all_markers$dataset))

In [ ]:
datasets

In [ ]:
i = 1

In [ ]:
 mrk_clust  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA") %>% 
mutate(gene_direction = paste(gene, direction))   %>% 
dplyr::filter(test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0") &
             dataset == datasets[1])  %>% group_by(gene_direction)   %>% tally()  %>% arrange(desc(n))  %>% 
dplyr::filter(n>1)
mrk_clust$dataset  <- datasets[1]

In [ ]:
mrk_clust_all  <- mrk_clust

In [ ]:
for(i in 2:length(datasets)){
    mrk_clust  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA") %>% 
mutate(gene_direction = paste(gene, direction))   %>% 
dplyr::filter(test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0") &
             dataset == datasets[i])  %>% group_by(gene_direction)   %>% tally()  %>% arrange(desc(n))  %>% 
dplyr::filter(n>1)
    mrk_clust$dataset  <- datasets[i]
    mrk_clust_all  <- rbind(mrk_clust_all, mrk_clust)
    }


In [ ]:
mrk_clust_all  %>% nrow

In [ ]:
mrk_clust_all  %>% group_by(gene_direction)  %>% 
tally  %>% arrange(desc(n))  %>% separate(gene_direction, into = c("gene", "direction"),
                                          sep = " ", remove = F)  %>% 
dplyr::filter(direction == "Down")

In [ ]:
down_genes  <- mrk_clust_all  %>% group_by(gene_direction)  %>% 
tally  %>% arrange(desc(n))  %>% separate(gene_direction, into = c("gene", "direction"),
                                          sep = " ", remove = F)  %>% 
dplyr::filter(direction == "Down")  %>% pull(gene)

In [ ]:
down_genes

In [ ]:
up_genesd  <- mrk_clust_all  %>% group_by(gene_direction)  %>% 
tally  %>% arrange(desc(n))  %>% separate(gene_direction, into = c("gene", "direction"),
                                          sep = " ", remove = F)  %>% 
dplyr::filter(direction == "Up")  %>% pull(gene)

In [ ]:
up_genesd

In [ ]:
all_markers$dataset  %>% table

In [ ]:
all_markers

In [ ]:
all_markers  %>% dplyr::filter(test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0") & 
                               source == "scRNAseq_RNA")  %>% 
dplyr::select(p_val_adj, gene, dataset, test_type)  %>% 
pivot_wider(names_from = "test_type", values_from = p_val_adj)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 16)
all_markers  %>% dplyr::filter(test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0") & 
                               source == "scRNAseq_RNA" & !(grepl(gene, pattern = "MTRN")))  %>% 
dplyr::select(avg_log2FC, gene, dataset, test_type)  %>% 
pivot_wider(names_from = "test_type", values_from = avg_log2FC)  %>% 
ggplot(aes(x = `Dia T0 vs Ctrl T0`, y = `Dia T1 vs Ctrl T0`, label = gene, fill = dataset))  +
geom_point() + geom_label()

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 25)
all_markers  %>% 
mutate(is_shared  = gene %in% up_genesd | gene %in% down_genes,
       direction_t0_t1 = ifelse(gene %in% up_genesd, "UP", ifelse(gene %in% down_genes, "DOWN", NA_character_)))  %>% 
dplyr::filter(source == "scRNAseq_RNA" & is_shared == T, 
              test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0"))  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
facet_grid(cols = vars(direction_t0_t1), rows = vars(test_type), space = "free", scales = "free") +
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)


In [ ]:
paths_fc

In [ ]:
paths_fc  <- paths_fc[which(grepl(paths_fc, pattern = "Ctrl"))]

In [ ]:
dataset_names  <- dataset_names[which(grepl(dataset_names, pattern = "Ctrl"))]

In [ ]:
dataset_names

In [ ]:
paths_fc

In [ ]:
paths_fc  %>% length

In [ ]:
all_fcs1  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
  fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia")) 
 fc.df$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 fc.df$dataset  <-   gsub(fc.df$dataset, pattern = "_Ctrl_vs_DiaT0", replacement = "")
 fc.df$comaprison = "DiaT0_vs_Ctrl"   
    
 return(fc.df)
    }

In [ ]:
all_fcs1_result  <- map(seq(from = 1, to = 41, by = 2), all_fcs1)

In [ ]:
all_fcs_result  <- bind_rows(all_fcs1_result)

In [ ]:
all_fcs_result$dataset  %>% table

In [ ]:
all_fcs2  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
  fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia")) 
 fc.df$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 fc.df$dataset  <-   gsub(fc.df$dataset, pattern = "_Ctrl_vs_DiaT1", replacement = "")
 fc.df$comaprison = "DiaT1_vs_Ctrl"
    
 return(fc.df)
    }

In [ ]:
all_fcs2_result  <- map(seq(from = 2, to = 42, by = 2), all_fcs2)

In [ ]:
all_fcs2_result  <- bind_rows(all_fcs2_result)

In [ ]:
all_fcs_result  <- rbind(all_fcs_result, all_fcs2_result)

In [ ]:
all_fcs_result

In [ ]:
all_fcs_result$dataset  %>% table

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 25)
all_fcs_result  %>% 
mutate(is_shared  = gene %in% up_genesd | gene %in% down_genes,
       direction_t0_t1 = ifelse(gene %in% up_genesd, "UP", ifelse(gene %in% down_genes, "DOWN", NA_character_)))  %>% 
dplyr::filter(is_shared == TRUE)  %>% 
ggplot(aes(factor(gene), dataset)) + 
facet_grid(cols = vars(direction_t0_t1), rows = vars(comaprison), space = "free", scales = "free") +
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = avg_log2FC, alpha = p_val_adj < 0.05)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_alpha_manual(values = c(0.4,1)) +
 scale_colour_gradient2(low = "blue", mid = "white", high = "red") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 25)
all_markers  %>% 
mutate(is_shared  = gene %in% up_genesd | gene %in% down_genes,
       direction_t0_t1 = ifelse(gene %in% up_genesd, "UP", ifelse(gene %in% down_genes, "DOWN", NA_character_)))  %>% 
dplyr::filter(source == "scRNAseq_RNA" & is_shared == T, 
              test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0"))  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
facet_grid(cols = vars(direction_t0_t1), rows = vars(test_type), space = "free", scales = "free") +
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 16)
all_markers  %>% dplyr::filter(test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0") & 
                               source == "scRNAseq_RNA" & !(grepl(gene, pattern = "MTRN")))  %>% 
dplyr::select(avg_log2FC, gene, dataset, test_type)  %>% 
pivot_wider(names_from = "test_type", values_from = avg_log2FC)  %>% 
ggplot(aes(x = `Dia T0 vs Ctrl T0`, y = `Dia T1 vs Ctrl T0`, label = gene, fill = dataset))  +
geom_point() 

In [ ]:
all_fcs_result

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 16)
all_fcs_result  %>% 
dplyr::filter(p_val_adj < 0.05)  %>% 
dplyr::filter(!(grepl(gene, pattern = "MTRN")))  %>% 
mutate(gene_in_genes = ifelse(gene %in% c("GZMK","GZMA","TNF","CX3CR1","GZMB"), gene, NA_character_))  %>% 
dplyr::select(avg_log2FC, gene, gene_in_genes, dataset, comaprison)  %>% 
pivot_wider(names_from = "comaprison", values_from = avg_log2FC)  %>% 
ggplot(aes(x = DiaT0_vs_Ctrl, y = DiaT1_vs_Ctrl, label = gene_in_genes, color = dataset))  +
geom_point() + geom_label()

In [ ]:
colors_of_populations = c(
    'cd4_l1_full' = "#ffa66aff",
    'cd4_l2_subcluster'  = "#ffa66aff",
    'cd4_l2_unc'  = "#cd3333ff",
    'cd4_l3_isaghi'  = "#a46464ff",
    'cd4_l3_naive'  = "#a58413ff",
    'cd4_l3_nfkb'  = "#a46c43ff",
    'cd4_l3_proliferating'  = "#9b3097ff", 
    'cd4_l3_temra'  = "#7d252aff",
    'cd4_l3_tfh'  = "#8e6a1dff",
    'cd4_l3_th1th17'  = "#a94e4eff",
    'cd4_l3_th2'  = "#b36672ff",
    'cd4_l3_treg'  = "#ba6d8dff",
    'cd8_l1_full'  = "#67934aff",
    'cd8_l2_nk'  = "#105292ff",
    'cd8_l2_subcluster' = "#67934aff", 
    'cd8_l2_unc'  = "#6d00c0ff",
    'cd8_l3_naive'  = "#004455ff",
    'cd8_l3_prolif'  = "#5f711aff",
    'cd8_l3_tcm'  = "#427c39ff",
    'cd8_l3_tem'  = "#004455ff",
    'cd8_l3_temra'  = "#16481fff",
    "grey" = "grey88"
)

In [ ]:
options(repr.plot.height = 12, repr.plot.width = 14)

all_fcs_result  %>% 
dplyr::filter(gene %in% all_markers$gene)  %>% 
dplyr::filter(p_val_adj < 0.05)  %>% 
dplyr::filter(!(grepl(gene, pattern = "MTRN")) & 
              dataset %in% c("cd4_l1_full_filt", "cd8_l1_full_filt"))  %>% 
dplyr::select(avg_log2FC, gene, dataset, comaprison)  %>% 
pivot_wider(names_from = "comaprison", values_from = avg_log2FC)  %>% 
mutate(gene_in_genes = ifelse(DiaT0_vs_Ctrl > 0.15 & DiaT1_vs_Ctrl > 0.15 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl < -0.2, 
                              gene, NA_character_))  %>% 
ggplot(aes(x = DiaT0_vs_Ctrl, y = DiaT1_vs_Ctrl, label = gene_in_genes, color = dataset))  +
scale_color_manual(values = c("#ffa66aff","#67934aff"))  +
geom_point(aes(shape = dataset)) + 
scale_shape_manual(values = c(21,22)) +
ggrepel::geom_text_repel() +
theme_bw() +
ggtheme()
ggsave(filename = "../figures/DE_genes/corr_DE_t0_t1.svg", width = 14, height = 12)

In [ ]:
all_fcs_result  %>% 
dplyr::filter(gene %in% all_markers$gene)  %>% 
dplyr::filter(p_val_adj < 0.05)  %>% 
dplyr::filter(!(grepl(gene, pattern = "MTRN")) & 
              dataset %in% c("cd4_l1_full_filt"))  %>% 
dplyr::select(avg_log2FC, gene, dataset, comaprison)  %>% 
pivot_wider(names_from = "comaprison", values_from = avg_log2FC)  %>% 
mutate(gene_in_genes = ifelse(DiaT0_vs_Ctrl > 0.10 & DiaT1_vs_Ctrl > 0.10 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl < -0.2 |
                              DiaT0_vs_Ctrl < -0.1 & DiaT1_vs_Ctrl > 0.1 |
                              DiaT0_vs_Ctrl > 0.1 & DiaT1_vs_Ctrl < -0.1 |
                              DiaT1_vs_Ctrl < -0.5 |
                              DiaT1_vs_Ctrl > 0.10, 
                              gene, NA_character_))  %>% 
ggplot(aes(x = DiaT0_vs_Ctrl, y = DiaT1_vs_Ctrl, label = gene_in_genes, color = dataset))  +
scale_color_manual(values = c("#d45500ff"))  +
geom_point(aes(shape = dataset)) + 
scale_shape_manual(values = c(21,22)) +
ggrepel::geom_text_repel() +
theme_bw() +
ggtheme()
ggsave(filename = "../figures/DE_genes/corr_DE_t0_t1_cd4.svg", width = 14, height = 12)

In [ ]:
all_fcs_result  %>% 
dplyr::filter(gene %in% all_markers$gene)  %>% 
dplyr::filter(p_val_adj < 0.05)  %>% 
dplyr::filter(!(grepl(gene, pattern = "MTRN")) & 
              dataset %in% c("cd8_l1_full_filt"))  %>% 
dplyr::select(avg_log2FC, gene, dataset, comaprison)  %>% 
pivot_wider(names_from = "comaprison", values_from = avg_log2FC)  %>% 
mutate(gene_in_genes = ifelse(DiaT0_vs_Ctrl > 0.20 & DiaT1_vs_Ctrl > 0.20 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl < -0.2 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl > 0.2 |
                              DiaT0_vs_Ctrl > 0.1 & DiaT1_vs_Ctrl < -0.1 |
                              DiaT1_vs_Ctrl < -0.25 |
                             DiaT0_vs_Ctrl < -0.3 |
                              DiaT1_vs_Ctrl < -0.1, 
                              gene, NA_character_))  %>% 
ggplot(aes(x = DiaT0_vs_Ctrl, y = DiaT1_vs_Ctrl, label = gene_in_genes, color = dataset))  +
scale_color_manual(values = c("#67934aff"))  +
geom_point(aes(shape = dataset)) + 
scale_shape_manual(values = c(21,22)) +
ggrepel::geom_text_repel() +
theme_bw() +
ggtheme()
ggsave(filename = "../figures/DE_genes/corr_DE_t0_t1_cd8.svg", width = 14, height = 12)

In [ ]:
colors_of_populations = c(
    'cd4_l1_full' = "#ffa66aff",
    'cd4_l2_subcluster'  = "#ffa66aff",
    'cd4_l2_unc'  = "#cd3333ff",
    'cd4_l3_isaghi'  = "#a46464ff",
    'cd4_l3_naive'  = "#a58413ff",
    'cd4_l3_nfkb'  = "#a46c43ff",
    'cd4_l3_proliferating'  = "#9b3097ff", 
    'cd4_l3_temra'  = "#7d252aff",
    'cd4_l3_tfh'  = "#8e6a1dff",
    'cd4_l3_th1th17'  = "#a94e4eff",
    'cd4_l3_th2'  = "#b36672ff",
    'cd4_l3_treg'  = "#ba6d8dff",
    'cd8_l1_full'  = "#67934aff",
    'cd8_l2_nk'  = "#105292ff",
    'cd8_l2_subcluster' = "#67934aff", 
    'cd8_l2_unc'  = "#6d00c0ff",
    'cd8_l3_naive'  = "#004455ff",
    'cd8_l3_prolif'  = "#5f711aff",
    'cd8_l3_tcm'  = "#427c39ff",
    'cd8_l3_tem'  = "#004455ff",
    'cd8_l3_temra'  = "#16481fff",
    "grey" = "grey88"
)

In [ ]:
all_fcs_result  %>% 
dplyr::filter(gene %in% all_markers$gene)  %>% 
dplyr::filter(p_val_adj < 0.05)  %>% 
dplyr::filter(!(grepl(gene, pattern = "MTRN")) & 
              grepl(dataset, pattern = "cd4_l3"))  %>% 
dplyr::select(avg_log2FC, gene, dataset, comaprison)  %>% 
pivot_wider(names_from = "comaprison", values_from = avg_log2FC)  %>% 
mutate(gene_in_genes = ifelse(DiaT0_vs_Ctrl > 0.15 & DiaT1_vs_Ctrl > 0.15 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl < -0.2, 
                              gene, NA_character_))  %>% 
ggplot(aes(x = DiaT0_vs_Ctrl, y = DiaT1_vs_Ctrl, label = gene_in_genes, color = dataset))  +
scale_color_manual(values = colors_of_populations)  +
geom_point(aes(shape = dataset)) + 
scale_shape_manual(values = c(21:25,21:25)) +
ggrepel::geom_text_repel() +
theme_bw() +
ggtheme()
ggsave(filename = "../figures/DE_genes/corr_DE_cd4_pop_t0_t1.svg", width = 14, height = 12)

In [ ]:
all_fcs_result  %>% 
dplyr::filter(gene %in% all_markers$gene)  %>% 
dplyr::filter(p_val_adj < 0.05)  %>% 
dplyr::filter(!(grepl(gene, pattern = "MTRN")) & 
              grepl(dataset, pattern = "cd8_l3"))  %>% 
dplyr::select(avg_log2FC, gene, dataset, comaprison)  %>% 
pivot_wider(names_from = "comaprison", values_from = avg_log2FC)  %>% 
mutate(gene_in_genes = ifelse(DiaT0_vs_Ctrl > 0.15 & DiaT1_vs_Ctrl > 0.15 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl < -0.2, 
                              gene, NA_character_))  %>% 
ggplot(aes(x = DiaT0_vs_Ctrl, y = DiaT1_vs_Ctrl, label = gene_in_genes, color = dataset))  +
scale_color_manual(values = colors_of_populations)  +
geom_point(aes(shape = dataset)) + 
scale_shape_manual(values = c(21:25,21:25)) +
ggrepel::geom_text_repel() +
theme_bw() +
ggtheme()
ggsave(filename = "../figures/DE_genes/corr_DE_cd8_pop_t0_t1.svg", width = 14, height = 12)

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 13)

all_fcs_result  %>% 
dplyr::filter(gene %in% all_markers$gene)  %>% 
dplyr::filter(p_val_adj < 0.05)  %>% 
dplyr::filter(!(grepl(gene, pattern = "MTRN")) & 
              dataset %in% c("cd4_l1_full_filt", "cd8_l1_full_filt"))  %>% 
dplyr::select(avg_log2FC, gene, dataset, comaprison)  %>% 
pivot_wider(names_from = "comaprison", values_from = avg_log2FC)  %>% 
mutate(gene_in_genes = ifelse(DiaT0_vs_Ctrl > 0.15 & DiaT1_vs_Ctrl > 0.15 |
                              DiaT0_vs_Ctrl > 0.15 & DiaT1_vs_Ctrl < -0.2 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl > 0.15 |
                              DiaT0_vs_Ctrl < -0.2 & DiaT1_vs_Ctrl < -0.2, 
                              gene, NA_character_))  %>% 
ggplot(aes(x = DiaT0_vs_Ctrl, y = DiaT1_vs_Ctrl, label = gene_in_genes, color = dataset))  +
scale_color_manual(values = c("#ffa66aff","#67934aff"))  +
geom_point(aes(shape = dataset)) + 
scale_shape_manual(values = c(21,22)) +
ggrepel::geom_text_repel() +
theme_bw() +
ggtheme()

# DE Select genes

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 20)
all_markers  %>% 
dplyr::filter(gene %in% up_genesd | gene %in% down_genes &
              source == "scRNAseq_RNA" & test_type == "Dia T0 vs Ctrl T0")  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
all_markers  %>% 
dplyr::filter(source == "scRNAseq_RNA" & test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0")) 

In [ ]:
all_markers  %>% 
dplyr::filter(source == "scRNAseq_RNA" & test_type %in% c("Dia T0 vs Ctrl T0", "Dia T1 vs Ctrl T0"))  %>%


# DE genes and plots

### Global regulators - Dia T0 vs Ctrl T0

In [ ]:
all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Ctrl T0" & n>1)

In [ ]:
genes.global.rna <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Ctrl T0" & n>1)  %>% pull(gene)

In [ ]:
genes.global.rna.ctrl.diat0 <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Ctrl T0" & n>1)  %>% pull(gene_direction)

In [ ]:
genes.global.rna

In [ ]:
lvl  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(direction, gene, gene_direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Ctrl T0" & n>1)  %>% 
arrange(direction) %>% pull(gene)  %>% unique

In [ ]:
lvl

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 20)
all_markers  %>% 
dplyr::filter(gene %in% genes.global.rna &
              source == "scRNAseq_RNA" & test_type == "Dia T0 vs Ctrl T0")  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene, levels = lvl), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
genes_up  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(direction, gene, gene_direction, test_type)  %>% 
dplyr::filter(direction == "Up")  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Ctrl T0" & n>1)  %>% 
arrange(direction) %>% pull(gene)  %>% unique

In [ ]:
genes_up


### Global regulators - Dia T1 vs Ctrl T0

In [ ]:
all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T1 vs Ctrl T0" & n>1)

In [ ]:
genes.global.rna <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T1 vs Ctrl T0" & n>1)  %>% pull(gene)

In [ ]:
genes.global.rna.ctrl.diat1 <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T1 vs Ctrl T0" & n>1)  %>% pull(gene_direction)

In [ ]:
genes.global.rna

In [ ]:
lvl  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(direction, gene, gene_direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T1 vs Ctrl T0" & n>1)  %>% 
arrange(direction) %>% pull(gene)  %>% unique

In [ ]:
lvl

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 20)
all_markers  %>% 
dplyr::filter(gene %in% genes.global.rna &
              source == "scRNAseq_RNA" & test_type == "Dia T1 vs Ctrl T0")  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene, levels = lvl), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))


### Global regulators - Dia T1 vs Dia T0

In [ ]:
all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Dia T1" & n>1)

In [ ]:
genes.global.rna <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Dia T1" & n>1)  %>% pull(gene)

In [ ]:
genes.global.rna.diat0.t1 <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(gene, gene_direction, direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Dia T1" & n>1)  %>% pull(gene_direction)

In [ ]:
genes.global.rna

In [ ]:
lvl  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(gene_direction = paste(gene, direction))  %>% 
group_by(direction, gene, gene_direction, test_type)  %>% tally  %>% 
arrange(desc(n))  %>% dplyr::filter(test_type == "Dia T0 vs Dia T1" & n>1)  %>% 
arrange(direction) %>% pull(gene)  %>% unique

In [ ]:
lvl

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 20)
all_markers  %>% 
dplyr::filter(gene %in% genes.global.rna &
              source == "scRNAseq_RNA" & test_type == "Dia T0 vs Dia T1")  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene, levels = lvl), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

# Overlap Venn T0 T1 Ctrl Dia

In [ ]:
library("ggVennDiagram")

In [ ]:
x <- list(
  Dia_T0_vs_Ctrl_T0 = genes.global.rna.ctrl.diat0, 
  Dia_T1_vs_Ctrl_T1 = genes.global.rna.ctrl.diat1, 
  Dia_T0_vs_Dia_T1 = genes.global.rna.diat0.t1)

In [ ]:
ggVennDiagram(x)
library(svglite)
#ggsave(filename = "../figures/DE_genes/venn_overlap.svg", width = 3, height = 5)

In [ ]:
venn <- Venn(x)

In [ ]:

data <- process_data(venn)
dat  <- data@region$item
names(dat)  <- data@region$name

dat

# plot

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 20)
all_markers  %>% 
left_join(gene_annot)  %>% 
dplyr::filter(!is.na(category) &
              source == "scRNAseq_RNA" & !grepl(pattern = "Cpeptfff", x = test_type))  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
facet_grid(cols = vars(category), rows = vars(test_type),
           drop = TRUE, scales = "free_x", space='free')+
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_color_manual(values = c("steelblue1","indianred2"))+
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90), 
      axis.ticks = element_blank())

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 20)
all_markers  %>% 
left_join(gene_annot)  %>% 
dplyr::filter(!is.na(category) &
              source == "scRNAseq_RNA" & !grepl(pattern = "Cpeptfff", x = test_type))  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
facet_grid(cols = vars(category), rows = vars(test_type),
           drop = TRUE, scales = "free_x", space='free')+
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_color_manual(values = c("steelblue1","indianred2"))+
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90), 
      axis.ticks = element_blank())

In [ ]:
gene_annot$is_snp  <- gene_annot$gene %in% snp_genes

In [ ]:
options(repr.plot.height = 2, repr.plot.width = 20)

gene_annot  %>% 
ggplot(aes(factor(gene), y = 1)) + 
  geom_point(size = 2, aes(colour = is_snp)) + 
facet_grid(cols = vars(category),
           drop = F, scales = "free_x", space='free')+
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_color_manual(values = c("white","black")) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

## Function in signaling

In [ ]:
gene_annot_nfkb  <- read_xlsx("../data/de_genes_annotataions_nfkb.xlsx")

In [ ]:
gene_annot_nfkb

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 10)
all_markers  %>% 
left_join(gene_annot_nfkb)  %>% 
dplyr::filter(!is.na(category) &
              source == "scRNAseq_RNA" & !grepl(pattern = "Cpept", x = test_type))  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
facet_grid(cols = vars(category), rows = vars(test_type),
           drop = TRUE, scales = "free_x", space='free')+
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_color_manual(values = c("steelblue1","indianred2"))+
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90), 
      axis.ticks = element_blank())

## Naive vs cytotox

In [ ]:
gene_annot  <- read_xlsx("../data/de_genes_annotataions2.xlsx")

In [ ]:
gene_annot

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 20)
all_markers  %>% 
left_join(gene_annot)  %>% 
dplyr::filter(!is.na(category) &
              source == "scRNAseq_RNA" & !grepl(pattern = "Cpept", x = test_type))  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
facet_grid(cols = vars(category), rows = vars(test_type),
           drop = TRUE, scales = "free_x", space='free')+
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_color_manual(values = c("steelblue1","indianred2"))+
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90), 
      axis.ticks = element_blank())

In [ ]:
ggsave("../figures/DE_genes/big_gene_table.png", width = 40, height = 20, units = "cm")
ggsave("../figures/DE_genes/big_gene_table.svg", width = 40, height = 20, units = "cm")

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 20)
all_markers  %>% 
left_join(gene_annot)  %>% 
dplyr::filter(!is.na(category) &
              source == "scRNAseq_RNA" & !grepl(pattern = "Cpept", x = test_type))  %>%  
mutate(cluster_source = paste(cluster, source))  %>% 
ggplot(aes(factor(gene), dataset)) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
facet_grid(cols = vars(category), rows = vars(test_type),
           drop = TRUE, scales = "free_x", space='free')+
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_color_manual(values = c("steelblue1","indianred2"))+
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90), 
      axis.ticks = element_blank()) + coord_flip()

In [ ]:
ggsave("../figures/DE_genes/big_gene_table.png", width = 40, height = 40, units = "cm")
ggsave("../figures/DE_genes/big_gene_table.svg", width = 40, height = 40, units = "cm")

In [ ]:
gene_annot$is_snp  <- gene_annot$gene %in% snp_genes

In [ ]:
options(repr.plot.height = 2, repr.plot.width = 20)

gene_annot  %>% 
ggplot(aes(factor(gene), y = 1)) + 
  geom_point(size = 2, aes(colour = is_snp)) + 
facet_grid(cols = vars(category),
           drop = F, scales = "free_x", space='free')+
  theme_bw()  + scale_size_continuous(range=c(1,4)) +
scale_color_manual(values = c("white","black")) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
ggsave("../figures/DE_genes/big_gene_table_snp.svg", width = 35, height = 5, units = "cm")

# Heatmap all comparisons

In [ ]:
hm  <- c("LEF1","BACH2","NELL2","TCF7","CXCR4","ZFP36L2","IL7R","KLF2","CCR7","SELL",
            
             
            rev(c("TNFRSF9","PRF1","NKG7","GZMB","CST7","GNLY","GZMA","CX3CR1","CCL5","TNF","TBX21","IFNG")),
             "TNFAIP3","DUSP1","TSC22D3","NFKBIA","DDIT4","INPP4B",
             rev(c("OASL","IFI44L","ISG15","MX1","STAT1","IFI6","IFIT3")),
                  "BTN3A2","BTN3A3")

In [ ]:
get_fc_genes  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    if(grepl(paths_fc[j], pattern = "Ctrl")){
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia"))  %>% 
dplyr::filter(gene %in% hm) 
        
        } else {
        
   if(grepl(paths_fc[j], pattern = "PR_0_T0")){
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "PR_0 T0"))  %>% 
dplyr::filter(gene %in% hm) 
        } else {
       
        if(grepl(paths_fc[j], pattern = "keto")){
 fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Keto_1 T0"))   %>% 
dplyr::filter(gene %in% hm) 
    } else {
 fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia T0"))   %>% 
dplyr::filter(gene %in% hm) 
    }}}

    fc.df$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 return(fc.df)
    }

In [ ]:
paths_fc  <- list.files("../../240617_VN_Diabetes_V06/tables/fold_change/", full.names = T)
dataset_names  <- list.files("../../240617_VN_Diabetes_V06/tables/fold_change", full.names = F)

paths_fc  <- paths_fc[grepl(paths_fc, pattern = "l3")|grepl(paths_fc, pattern = "cd8_l2_unc")|
                      grepl(paths_fc, pattern = "cd4_l2_unc")|grepl(paths_fc, pattern = "cd8_l2_nk") ]

dataset_names  <- dataset_names[grepl(dataset_names, pattern = "l3")|grepl(dataset_names, pattern = "cd8_l2_unc")|
                      grepl(dataset_names, pattern = "cd4_l2_unc")|grepl(dataset_names, pattern = "cd8_l2_nk") ]

paths_fc  <- paths_fc[grepl(paths_fc, pattern = "Ctrl_vs_DiaT0")|grepl(paths_fc, pattern = "DiaT0_vs_DiaT1")|
                      grepl(paths_fc, pattern = "Ctrl_vs_DiaT1")|
                      grepl(paths_fc, pattern = "PR_0_T0_vs_PR_1_T0")|grepl(paths_fc, pattern = "ketoacidosis")]

dataset_names  <- dataset_names[grepl(dataset_names, pattern = "Ctrl_vs_DiaT0")|grepl(dataset_names, pattern = "DiaT0_vs_DiaT1")|
                      grepl(dataset_names, pattern = "Ctrl_vs_DiaT1")|
                      grepl(dataset_names, pattern = "PR_0_T0_vs_PR_1_T0")|grepl(dataset_names, pattern = "ketoacidosis")]


In [ ]:
fcs  <- purrr::map(1:length(paths_fc), get_fc_genes)

In [ ]:
fcs  <- bind_rows(fcs)

In [ ]:
fcs$population  <- substr(fcs$dataset, 1,11)

table(fcs$population)

fcs  <- fcs  %>% mutate(population2 = ifelse(grepl(population, pattern = "_nk_"), substr(population,1,9), population))

fcs$population2  %>% table

fcs  <- fcs  %>% mutate(n1 = as.numeric(nchar(dataset)))  %>% 
mutate(n2 = n1-12)  %>% 
mutate(test_type = substr(dataset,n2,n1))  


fcs$test_type  %>% table

In [ ]:
fcs2  <- fcs  %>% mutate(andjus_logFC = ifelse(p_val_adj<0.05,avg_log2FC,avg_log2FC))  %>% 
dplyr::select(andjus_logFC, gene, dataset)

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 24)
fcs  %>% 
ggplot(aes(x = factor(population2, levels = c(
'cd8_l3_prol',
'cd8_l3_tcm_',
'cd4_l3_th1t',
'cd4_l3_temr',
'cd8_l3_temr',
'cd4_l3_tfh_',
'cd8_l3_naiv',
'cd4_l3_naiv',
'cd4_l3_th2_',
'cd4_l3_nfkb',
'cd4_l2_unc_',
'cd4_l3_treg',
'cd4_l3_isag',
'cd4_l3_prol',
'cd8_l3_tem_',
'cd8_l2_nk',
'cd8_l2_unc_'    
))          
           , y = factor(gene, levels = rev(hm)))) +
geom_point(aes(size = -log(p_val_adj), color = avg_log2FC)) +
scale_alpha_manual(values = c(0.5,1)) +
scale_size_continuous(range = c(3,8))+
ylab("") + xlab("") +
facet_grid(cols = vars(test_type), space = "free", scales = "free") +
scale_color_gradientn(colours = c("blue", "blue","white", "red","red")) + 
theme_classic() + theme(axis.text.x = element_text(angle = 90)) + ggtheme()

In [ ]:
ggsave("../figures/DE_genes//heatmap_deg.svg", width = 45, height = 25, units = "cm", create.dir = TRUE)

In [ ]:
fcs2_mtx  <- fcs2  %>% pivot_wider(names_from = dataset, values_from = andjus_logFC,
                                  values_fill = 0)  %>% column_to_rownames("gene")  %>% 
as.matrix()

In [ ]:
order_data  <- paste(c(""))

In [ ]:
rownames(fcs2_mtx)[match(hm, rownames(fcs2_mtx))]

In [ ]:
c(seq(from = 1, to = 85, by = 5),seq(from = 2, to = 85, by = 5),
  seq(from = 3, to = 85, by = 5),seq(from = 4, to = 85, by = 5),
  seq(from = 5, to = 85, by = 5))

In [ ]:
fcs2_mtx_ordered  <- fcs2_mtx[match(hm, rownames(fcs2_mtx)),c(seq(from = 1, to = 85, by = 5),seq(from = 2, to = 85, by = 5),
  seq(from = 3, to = 85, by = 5),seq(from = 4, to = 85, by = 5),
  seq(from = 5, to = 85, by = 5))]

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 24)


In [ ]:
fcs2_mtx_ordered  <- fcs2_mtx[match(hm, rownames(fcs2_mtx)),c(seq(from = 1, to = 85, by = 5))]

# Correlations in populations - loading full dataset

In [ ]:
cd4_l1_full  <- readRDS("../data/processed/L1/cd4_l1_full_filt.rds")

In [ ]:
cd8_l1_full  <- readRDS("../data/processed/L1/cd8_l1_full_filt.rds")

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 4)
DimPlot(cd4_l1_full, group.by = "annotations_l3", raster = T)

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 20)

DimPlot(cd8_l1_full, group.by = "annotations_l3", raster = T)

In [ ]:
full  <- merge(cd4_l1_full, cd8_l1_full)

In [ ]:
rm(cd8_l1_full)
rm(cd4_l1_full)

In [ ]:
gc()

In [ ]:
full  <- readRDS("../data/processed/full.rds")

In [ ]:
full@meta.data  <- full@meta.data  %>% 
mutate(annotations_l2  = gsub(pattern = " T cells", replacement = "", x = annotations_l3)) 

In [ ]:
 %>% 
separate(annotations_l2, into = c("annotations_l1", "annotations_l2", "annotations_l3"), sep = "---") 

In [ ]:
 %>% 
mutate(annotations_l2  = gsub(pattern = "Unconventional_CD8 Unc", replacement = "CD8_Unc", x = annotations_l2))  %>% 
mutate(annotations_l2  = gsub(pattern = "Unconventional_Unc", replacement = "CD4_Unc_", x = annotations_l2))  

In [ ]:
 %>% 
separate(annotations_l2, into = c("Level1", "Level2",NA,NA), sep = "_")  

In [ ]:
full$annotations_l2   <- paste(full$Level1, full$Level2)

In [ ]:
full$annotations_l3  %>% table

In [ ]:
saveRDS(full, "../data/processed/full.rds")

## Correlation of Expression with Fast Cpeptide

In [ ]:
ds  <- readRDS("../data/processed/full.rds")

In [ ]:
ds  <- subset(full, fasting_cpept_1 >= 0 & Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))

In [ ]:
ds$Patient_Time  %>% table

In [ ]:
ds@meta.data  <- ds@meta.data  %>% separate(annotations_l2, into = c("annotations_l2_part1",
                                                                     "annotations_l2_part2",
                                                                     "annotations_l2_part3"),
                                            sep = "---", remove = F)  %>% 
mutate(annotations_l2_new = paste(annotations_l2_part1, annotations_l2_part2))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "Unconventional Unconventional", replacement = "Unc"))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "Unconventional", replacement = "Unc"))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "NK cells NK cells", replacement = "NK cells"))



In [ ]:
ds$annotations_l2_new  %>% table

In [ ]:
ds$Patient_Time_population  <- paste(ds$Patient_Time, ds$annotations_l2_new, sep = ".")

In [ ]:
de_genes  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% pull(gene)  %>% unique

In [ ]:
ds_mtx  <- AverageExpression(ds, group.by = "Patient_Time_population", return.seurat = F, assays = "RNA",
                            features = de_genes)

In [ ]:
fast  <- ds@meta.data  %>% dplyr::select(Patient_Time_population, fasting_cpept_1)  %>% unique

In [ ]:
fast

In [ ]:
rna  <- ds_mtx$RNA  %>% 
as.data.frame()  %>% 
rownames_to_column("gene")  %>% 
pivot_longer(!gene, names_to = "Patient_Time_population", values_to = "expression")  %>% 
 left_join(fast)  %>% separate(Patient_Time_population, into = c("Patient_Time", "Population"),
                              sep = "\\.", remove = F)

In [ ]:
rna

In [ ]:
populations_to_test  <- rna  %>% pull(Population)  %>% unique

In [ ]:
populations_to_test

In [ ]:
calc_one_pop  <- function(select_pop){
    
    pop  <- select_pop
    df  <- rna  %>% dplyr::filter(gene == select_gene & Population == select_pop)
    cor  <- cor.test(df$expression, df$fasting_cpept_1)
    res_df  <- data.frame(gene = select_gene,
                          population = pop,
                          cor = cor$estimate, 
                          pval = cor$p.value, 
                         padj  = ifelse(cor$p.value*length(de_genes)>1,1,cor$p.value*length(de_genes)))
    return(res_df)
}

calc_correlation  <- function(select_gene){
    populations  <- rna  %>% pull(Population)  %>% unique
    all_results  <- future_map(.x = populations, .f = calc_one_pop)
    test2  <- bind_rows(all_results)
    return(test2)
}

In [ ]:
library(future)
plan(multisession, workers = 8)

In [ ]:
rm(full)
gc()

In [ ]:
test  <- future_map(.x = de_genes, .f = calc_correlation)

In [ ]:
test3  <- bind_rows(test)

In [ ]:
test4  <- test3  %>% arrange(pval)

In [ ]:
test4

In [ ]:
options(repr.plot.width = 5, repr.plot.width = 5)
for(i in 1:10){
gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(gene, "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
print(p)
    }

In [ ]:
test5  <- test4  %>% dplyr::filter(population == "CD4 Treg")

In [ ]:
options(repr.plot.width = 5, repr.plot.width = 5)
for(i in 1:10){
gene  <- test5$gene[i]
population  <- "CD4 Treg"
df  <- rna  %>% dplyr::filter(gene == test5$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(gene, "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
print(p)
    }

In [ ]:
p

In [ ]:
df

In [ ]:
gene  <- "IL4R"
population  <- "CD4 Treg"
df  <- rna  %>% dplyr::filter(gene == gene & Population == population)  %>% unique

In [ ]:
df

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)
for(i in 96) {
   gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_T1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(test4$gene[i], "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
 ggsave(filename = paste0("../figures/correlation_gene_in_populations/",i,".svg"), width = 5, height = 4.5)

    print(p)
    }

In [ ]:
dir.create("../figures/correlation_gene_in_populations/")

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)
for(i in 1:30) {
   gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_T1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(test4$gene[i], "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
 ggsave(filename = paste0("../figures/correlation_gene_in_populations/",i,".svg"), width = 5, height = 4.5)

    print(p)
    }

## Correlation of Expression with Age

In [ ]:
full$age

In [ ]:
ds  <- subset(full, Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))

In [ ]:
ds$Patient_Time_population  <- paste(ds$Patient_Time, ds$annotations_l2, sep = "-")

In [ ]:
ds_mtx  <- AverageExpression(ds, group.by = "Patient_Time_population", return.seurat = F, assays = "RNA")

In [ ]:
fast  <- ds@meta.data  %>% dplyr::select(Patient_Time_population, age)  %>% unique

In [ ]:
fast

In [ ]:
rna  <- ds_mtx$RNA  %>% 
as.data.frame()  %>% 
rownames_to_column("gene")  %>% 
pivot_longer(!gene, names_to = "Patient_Time_population", values_to = "expression")  %>% 
 left_join(fast)  %>% 
separate(Patient_Time_population, into = c("Patient_Time", "Population"), sep = "-", remove = F)

In [ ]:
rna

In [ ]:
genes_to_test  <- test4$gene[1:30]  %>% unique

In [ ]:
genes_to_test

In [ ]:
populations_to_test  <- rna  %>% pull(Population)  %>% unique

In [ ]:
populations_to_test

In [ ]:
calc_one_pop  <- function(select_pop){
    
    pop  <- select_pop
    df  <- rna  %>% dplyr::filter(gene == select_gene & Population == select_pop)
    cor  <- cor.test(df$expression, df$fasting_cpept_T1)
    res_df  <- data.frame(gene = select_gene,
                          population = pop,
                          cor = cor$estimate, 
                          pval = cor$p.value, 
                         padj  = ifelse(cor$p.value*length(genes_to_test)>1,1,cor$p.value*length(genes_to_test)))
    return(res_df)
}

calc_correlation  <- function(select_gene){
    populations  <- rna  %>% pull(Population)  %>% unique
    all_results  <- future_map(.x = populations, .f = calc_one_pop)
    test2  <- bind_rows(all_results)
    return(test2)
}

In [ ]:
library(future)
plan(multisession, workers = 8)

In [ ]:
rm(cd4_l1_full)
rm(cd8_l1_full)
rm(full)
gc()

In [ ]:
test  <- future_map(.x = global_genes, .f = calc_correlation)

In [ ]:
test3  <- bind_rows(test)

In [ ]:
test4  <- test3  %>% arrange(pval)

In [ ]:
gc()

In [ ]:
i = 1

In [ ]:
rna

In [ ]:
gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)

In [ ]:
df

In [ ]:
test4

In [ ]:
gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_T1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(test4$gene[i], "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()

In [ ]:
p

In [ ]:
dir.create("../figures/correlation_gene_in_populations/")

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)
for(i in 1:30) {
   gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_T1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(test4$gene[i], "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
 ggsave(filename = paste0("../figures/correlation_gene_in_populations/",i,".svg"), width = 5, height = 4.5)

    print(p)
    }

## Correlation v05

### Cpeptide

In [ ]:
full  <- readRDS("../data/processed/full.rds")

In [ ]:
gc()

In [ ]:
full  <- subset(full, fasting_cpept_1 >= 0 & Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))

In [ ]:
full$Patient_Time  %>% table

In [ ]:
full@meta.data  <- full@meta.data  %>% separate(annotations_l2, into = c("annotations_l2_part1",
                                                                     "annotations_l2_part2",
                                                                     "annotations_l2_part3"),
                                            sep = "---", remove = F)  %>% 
mutate(annotations_l2_new = paste(annotations_l2_part1, annotations_l2_part2))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "Unconventional Unconventional", replacement = "Unc"))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "Unconventional", replacement = "Unc"))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "NK cells NK cells", replacement = "NK cells"))



In [ ]:
full$annotations_l2_new  %>% table

In [ ]:
full$Patient_Time_population  <- paste(full$Patient_Time, full$annotations_l2_new, sep = ".")

In [ ]:
markers_cd4  <- read_csv("../tables/de_genes/240319_cd4_all_markers_without_sex.csv")
markers_cd8  <- read_csv("../tables/de_genes/240319_cd8_all_markers_without_sex.csv")
all_markers  <- rbind(markers_cd4, markers_cd8)

In [ ]:
de_genes  <- all_markers  %>% dplyr::filter(source == "scRNAseq_RNA")  %>% pull(gene)  %>% unique

In [ ]:
length(de_genes)

In [ ]:
all_markers  %>% dplyr::filter(gene == "IL4R")

In [ ]:
which(de_genes == "IL4R")

In [ ]:
full_mtx  <- AverageExpression(full, group.by = "Patient_Time_population", return.seurat = F, assays = "RNA",
                            features = de_genes)

In [ ]:
fast  <- full@meta.data  %>% dplyr::select(Patient_Time_population, fasting_cpept_1)  %>% unique

In [ ]:
fast

In [ ]:
rna  <- full_mtx$RNA  %>% 
as.data.frame()  %>% 
rownames_to_column("gene")  %>% 
pivot_longer(!gene, names_to = "Patient_Time_population", values_to = "expression")  %>% 
 left_join(fast)  %>% separate(Patient_Time_population, into = c("Patient_Time", "Population"),
                              sep = "\\.", remove = F)

In [ ]:
populations_to_test  <- rna  %>% pull(Population)  %>% unique

In [ ]:
populations_to_test

In [ ]:
calc_one_pop  <- function(select_pop){
    
    pop  <- select_pop
    df  <- rna  %>% dplyr::filter(gene == select_gene & Population == select_pop)
    cor  <- cor.test(df$expression, df$fasting_cpept_1)
    res_df  <- data.frame(gene = select_gene,
                          population = pop,
                          cor = cor$estimate, 
                          pval = cor$p.value, 
                         padj  = ifelse(cor$p.value*length(de_genes)>1,1,cor$p.value*length(de_genes)))
    return(res_df)
}

calc_correlation  <- function(select_gene){
    populations  <- rna  %>% pull(Population)  %>% unique
    all_results  <- future_map(.x = populations, .f = calc_one_pop)
    test2  <- bind_rows(all_results)
    return(test2)
}

In [ ]:
library(future)
plan(multisession, workers = 12)

In [ ]:
rm(full)
gc()

In [ ]:
test  <- future_map(.x = de_genes, .f = calc_correlation)

In [ ]:
test3  <- bind_rows(test)

In [ ]:
test4  <- test3  %>% arrange(pval)

In [ ]:
test4  

In [ ]:
options(repr.plot.width = 5, repr.plot.width = 5)
for(i in 1:10){
gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(gene, "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
print(p)
    }

In [ ]:
write.csv(test4, "../tables/correlation_genes_in_populations_fast_cpept.csv")

In [ ]:
test4

In [ ]:
test4$abs_cor  <- abs(test4$cor)

In [ ]:
test4

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 10)
test4  %>%  
mutate(label = ifelse(abs_cor>0.4,gene,
                     NA_character_))  %>% 
mutate(color = ifelse(abs_cor>0.4,population,"grey"))   %>% 
ggplot(aes(x = cor, y = -log10(padj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2)) + 
        theme_minimal() +
        geom_text_repel(size = 6) +
        #scale_color_manual(values=colors_of_populations) +
        geom_vline(xintercept=c(-0.4, 0.4), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle("") + ggtheme()


### delta Cpeptide

In [ ]:
full  <- readRDS("../data/processed/full.rds")

In [ ]:
rm(full)

In [ ]:
gc()

In [ ]:
full$c_peptide_change_perc

In [ ]:
full  <- subset(full, c_peptide_change_perc >= 0 & Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))

In [ ]:
full$Patient_Time  %>% table

In [ ]:
full@meta.data  <- full@meta.data  %>% separate(annotations_l2, into = c("annotations_l2_part1",
                                                                     "annotations_l2_part2",
                                                                     "annotations_l2_part3"),
                                            sep = "---", remove = F)  %>% 
mutate(annotations_l2_new = paste(annotations_l2_part1, annotations_l2_part2))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "Unconventional Unconventional", replacement = "Unc"))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "Unconventional", replacement = "Unc"))  %>% 
mutate(annotations_l2_new = gsub(annotations_l2_new, pattern = "NK cells NK cells", replacement = "NK cells"))



In [ ]:
full$Patient_Time_population  <- paste(full$Patient_Time, full$annotations_l2_new, sep = ".")

In [ ]:
full_mtx  <- AverageExpression(full, group.by = "Patient_Time_population", return.seurat = F, assays = "RNA",
                            features = de_genes)

In [ ]:
fast  <- full@meta.data  %>% dplyr::select(Patient_Time_population, c_peptide_change, c_peptide_change_perc)  %>% unique

In [ ]:
fast

In [ ]:
rna  <- full_mtx$RNA  %>% 
as.data.frame()  %>% 
rownames_to_column("gene")  %>% 
pivot_longer(!gene, names_to = "Patient_Time_population", values_to = "expression")  %>% 
 left_join(fast)  %>% separate(Patient_Time_population, into = c("Patient_Time", "Population"),
                              sep = "\\.", remove = F)

In [ ]:
populations_to_test  <- rna  %>% pull(Population)  %>% unique

In [ ]:
populations_to_test

In [ ]:
calc_one_pop  <- function(select_pop){
    
    pop  <- select_pop
    df  <- rna  %>% dplyr::filter(gene == select_gene & Population == select_pop)
    cor  <- cor.test(df$expression, df$c_peptide_change)
    res_df  <- data.frame(gene = select_gene,
                          population = pop,
                          cor = cor$estimate, 
                          pval = cor$p.value, 
                         padj  = ifelse(cor$p.value*length(de_genes)>1,1,cor$p.value*length(de_genes)))
    cor2  <- cor.test(df$expression, df$c_peptide_change_perc)
    res_df2  <- data.frame(gene = select_gene,
                          population = pop,
                          cor = cor2$estimate, 
                          pval = cor2$p.value, 
                         padj  = ifelse(cor2$p.value*length(de_genes)>1,1,cor2$p.value*length(de_genes)))
    res_df$test  <- "c_peptide_change"
    res_df2$test  <- "c_peptide_change_perc"
    res_df  <- rbind(res_df, res_df2)
    return(res_df)
}

calc_correlation  <- function(select_gene){
    populations  <- rna  %>% pull(Population)  %>% unique
    all_results  <- future_map(.x = populations, .f = calc_one_pop)
    test2  <- bind_rows(all_results)
    return(test2)
}

In [ ]:
library(future)
plan(multisession, workers = 12)

In [ ]:
rm(full)
gc()

In [ ]:
test  <- future_map(.x = de_genes, .f = calc_correlation)

In [ ]:
test3  <- bind_rows(test)

In [ ]:
test4  <- test3  %>% arrange(pval)

In [ ]:
test4  

In [ ]:
options(repr.plot.width = 5, repr.plot.width = 5)
for(i in 1:10){
gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=c_peptide_change_perc)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(gene, "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
print(p)
    }

In [ ]:
write.csv(test4, "../tables/correlation_genes_in_populations_delta_cpept.csv")

In [ ]:
test4$abs_cor  <- abs(test4$cor)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 10)
test4  %>%  
dplyr::filter(test == "c_peptide_change_perc")  %>% 
mutate(label = ifelse(abs_cor>0.4,paste(gene, "in", population),
                     NA_character_))  %>% 
mutate(color = ifelse(abs_cor>0.4,population,"grey"))   %>% 
ggplot(aes(x = cor, y = -log2(padj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2)) + 
        theme_minimal() +
        geom_text_repel(size = 6) +
        #scale_color_manual(values=colors_of_populations) +
        geom_vline(xintercept=c(-0.4, 0.4), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle("") + ggtheme()



In [ ]:
options(repr.plot.width = 18, repr.plot.height = 10)
test4  %>%  
mutate(label = ifelse(abs_cor>0.4,gene,
                     NA_character_))  %>% 
mutate(color = ifelse(abs_cor>0.4,population,"grey"))   %>% 
ggplot(aes(x = cor, y = -log10(padj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2)) + 
        theme_minimal() +
        geom_text_repel(size = 6) +
        #scale_color_manual(values=colors_of_populations) +
        geom_vline(xintercept=c(-0.4, 0.4), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle("") + ggtheme() 

In [ ]:
options(repr.plot.width = 5, repr.plot.width = 5)
for(i in 1:10){
gene  <- test5$gene[i]
population  <- "CD4 Treg"
df  <- rna  %>% dplyr::filter(gene == test5$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(gene, "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
print(p)
    }

In [ ]:
df

In [ ]:
gene  <- "IL4R"
population  <- "CD4 Treg"
df  <- rna  %>% dplyr::filter(gene == gene & Population == population)  %>% unique

In [ ]:
df

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)
for(i in 96) {
   gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_T1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(test4$gene[i], "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
 ggsave(filename = paste0("../figures/correlation_gene_in_populations/",i,".svg"), width = 5, height = 4.5)

    print(p)
    }

In [ ]:
dir.create("../figures/correlation_gene_in_populations/")

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)
for(i in 1:30) {
   gene  <- test4$gene[i]
population  <- test4$population[i]
df  <- rna  %>% dplyr::filter(gene == test4$gene[i] & Population == population)
p <- df %>%  
 ggplot(aes(x=expression, y=fasting_cpept_T1)) +
 geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm, alpha = 0.2) + ggtitle(paste(test4$gene[i], "in", population)) + 
stat_cor(size = 7) + theme_classic() + ggtheme()
 ggsave(filename = paste0("../figures/correlation_gene_in_populations/",i,".svg"), width = 5, height = 4.5)

    print(p)
    }

# GSEA

### Global regulators UP

In [ ]:
all_markers2

In [ ]:
global_up_names  <- all_markers2  %>% 
                        dplyr::filter(source == "scRNAseq_RNA" & !is.na(entrezid) & direction == "Up")  %>% 
                        group_by(entrezid)  %>% 
                        summarise(avg_log2FC = mean(avg_log2FC))   %>% 
                        arrange(desc(avg_log2FC))  %>% 
                        pull(entrezid)


global_up_values  <- all_markers2  %>% 
                        dplyr::filter(source == "scRNAseq_RNA" & !is.na(entrezid) & direction == "Up")  %>% 
                        group_by(entrezid)  %>% 
                        summarise(avg_log2FC = mean(avg_log2FC))   %>% 
                        arrange(desc(avg_log2FC))  %>% 
                        pull(avg_log2FC)

In [ ]:
names(global_up_values)  <- global_up_names

In [ ]:
global_up_values  <- global_up_values[rev(order(global_up_values))]

In [ ]:
global_up_values

### C7 category

In [ ]:
msigdbr_collections()

In [ ]:
C7_t2g <- msigdbr(species = "Homo sapiens", category = "C7") %>% 
  dplyr::select(gs_name, entrez_gene)
head(C7_t2g)

In [ ]:
em2 <- GSEA(global_up_values, TERM2GENE = C7_t2g)
head(em2)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 16)
dotplot(em2, showCategory=30) + ggtitle("Global Up C7")

### C2 category

In [ ]:
C2_t2g <- msigdbr(species = "Homo sapiens", category = "C2") %>% 
  dplyr::select(gs_name, entrez_gene)
head(C2_t2g)

In [ ]:
em2 <- GSEA(global_up_values, TERM2GENE = C2_t2g)


In [ ]:
options(repr.plot.width = 8, repr.plot.height = 16)
dotplot(em2, showCategory=30) + ggtitle("Global Up C2")

## Global regulators Down

In [ ]:
global_down_names  <- all_markers2  %>% 
                        dplyr::filter(source == "scRNAseq_RNA" & !is.na(entrezid) & direction == "Down")  %>% 
                        group_by(entrezid)  %>% 
                        summarise(avg_log2FC = mean(avg_log2FC))   %>% 
                        arrange(desc(avg_log2FC))  %>% 
                        pull(entrezid)


global_down_values  <- all_markers2  %>% 
                        dplyr::filter(source == "scRNAseq_RNA" & !is.na(entrezid) & direction == "Down")  %>% 
                        group_by(entrezid)  %>% 
                        summarise(avg_log2FC = mean(avg_log2FC))   %>% 
                        arrange(desc(avg_log2FC))  %>% 
                        pull(avg_log2FC)

In [ ]:
names(global_down_values)  <- global_down_names

In [ ]:
global_down_values  <- global_down_values[rev(order(global_down_values))]

In [ ]:
global_down_values

### C7 category

In [ ]:
em2 <- GSEA(global_down_values, TERM2GENE = C7_t2g)
head(em2)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 16)
dotplot(em2, showCategory=30) + ggtitle("Global Down C7")

### C2 category

In [ ]:
em2 <- GSEA(global_down_values, TERM2GENE = C2_t2g)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 16)
dotplot(em2, showCategory=30) + ggtitle("Global Down C2")

## All categories up and down

In [ ]:
msigdbr_collections()

In [ ]:
pathways = msigdbr_collections()  %>% group_by(gs_cat)  %>% tally()  %>% pull(gs_cat)

In [ ]:
pathways

In [ ]:
t2g <- msigdbr(species = "Homo sapiens", category = "C7") %>% 
          dplyr::select(gs_name, human_gene_symbol)

In [ ]:
msigdbr(species = "Homo sapiens", category = "C7")

In [ ]:
get_pathway_df  <- function(pathway){
    t2g <- msigdbr(species = "Homo sapiens", category = pathway) %>% 
          dplyr::select(gs_name, entrez_gene)
    em2 <- GSEA(global_down_values, TERM2GENE = t2g)
    df_down  <- as.data.frame(em2)
            
    em3 <- GSEA(global_up_values, TERM2GENE = t2g)
    df_up  <- as.data.frame(em3)
            
    if(nrow(df_up)>0&nrow(df_down)>0){
        df_down$direction  <- "Down"
        df_up$direction  <- "Up"
        df_all  <- rbind(df_down, df_up)
        df_all$gs_cat  <- pathway
        
    } else {
        if(nrow(df_up)>0){
            df_up$direction  <- "Up"
            df_all  <- df_up
            df_all$gs_cat  <- pathway
    
    } else {
        if(nrow(df_down)>0){
            df_down$direction  <- "Down"
            df_all  <- df_down 
            df_all$gs_cat  <- pathway
    
    } else {
            df_all  <- NULL 
        } }}
    return(df_all)
}

In [ ]:
all_pathways  <- map(.x = pathways, .f = get_pathway_df)

In [ ]:
all_pathways2  <- bind_rows(all_pathways)

In [ ]:
all_pathways2$core_enrichment   %>% nchar  %>% table

In [ ]:
711/4

In [ ]:
dir.create("../tables/pathways/")

In [ ]:
write.csv(all_pathways2, "../tables/pathways/gsea_global.csv")

In [ ]:
enriched_genes  <- all_pathways2  %>% 
dplyr::select(core_enrichment)  %>% 
separate(core_enrichment, sep = "/",
        into = paste0("v",1:138))



In [ ]:
enriched_genes

In [ ]:
geneIDs2  <- geneIDs1  %>% group_by(entrezid)  %>% slice_head(n = 1)

In [ ]:
geneIDs2  <- geneIDs2  %>% filter(!is.na(entrezid))

In [ ]:
df_enriched_genes  <- enriched_genes

In [ ]:
for(i in 1:138){
    df1  <- dplyr::select(df_enriched_genes, i)
    colnames(df1)  <- "entrezid"
    df1$entrezid  <- as.integer(df1$entrezid)
    df2  <- left_join(df1, geneIDs2)
    df_enriched_genes[[paste0("v",i)]]  <- df2$gene
}

In [ ]:
df_enriched_genes

In [ ]:
df_enriched_genes$genes  <- ""

In [ ]:
for(i in 1:411){
   df_enriched_genes[i, 139]  <-  gsub(pattern =  " / NA",
        replacement = "",
        x = paste(df_enriched_genes[i,1:138], collapse=" / ")) 
}

In [ ]:
df_enriched_genes

In [ ]:
all_pathways2$genes  <- df_enriched_genes$genes

In [ ]:
write.csv(all_pathways2, "../tables/pathways/gsea_global.csv")

## Global enrichment

In [ ]:
up_dia  <- all_markers2  %>% 
                        dplyr::filter(test_type == "Ctrl T0 vs Dia T0" & cluster == "Dia T0" & source == "scRNAseq_RNA")  %>% 
                        group_by(gene)  %>% 
                        tally()  %>% 
                        arrange(desc(n))  %>% 
                        pull(gene)
down_dia  <- all_markers2  %>% 
                        dplyr::filter(test_type == "Ctrl T0 vs Dia T0" & cluster == "Ctrl T0" & source == "scRNAseq_RNA")  %>% 
                        group_by(gene)  %>% 
                        tally()  %>% 
                        arrange(desc(n))  %>% 
                        pull(gene)

In [ ]:
up_dia_t0_vs_ctrl_t0

In [ ]:
ego2 <- enrichGO(gene         = up_dia_t0_vs_ctrl_t0,
                OrgDb         = org.Hs.eg.db,
                keyType       = 'SYMBOL',
                ont           = "ALL",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.01,
                qvalueCutoff  = 0.05)
head(ego2, 3)         

In [ ]:
options(repr.plot.width = 23, repr.plot.height = 7)
edox2 <- pairwise_termsim(ego2)
p1 <- treeplot(edox2)
p2 <- treeplot(edox2, hclust_method = "average")
p1 + ggtitle("Enriched in Dia")

In [ ]:
down_dia_t0_vs_ctrl_t0

In [ ]:
ego3 <- enrichGO(gene         = down_dia_t0_vs_ctrl_t0,
                OrgDb         = org.Hs.eg.db,
                keyType       = 'SYMBOL',
                ont           = "ALL",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.01,
                qvalueCutoff  = 0.05)
head(ego3, 10)         

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5)

edox3 <- pairwise_termsim(ego3)
p1 <- treeplot(edox3)
p2 <- treeplot(edox3, hclust_method = "average")
p1 + ggtitle("Enriched in Ctrl")

### Fold Changes in Our Dataset

In [ ]:
rm(list = ls())

In [ ]:
cd8_l3_tem  <- readRDS("../data/processed/L3/cd8_l3_tem.rds")
cd8_l3_naive  <- readRDS("../data/processed/L3/cd8_l3_naive.rds")
cd8_l3_tcm  <- readRDS("../data/processed/L3/cd8_l3_tcm.rds")
cd8_l3_temra  <- readRDS("../data/processed/L3/cd8_l3_temra.rds")
cd8_l3_prolif  <- readRDS("../data/processed/L3/cd8_l3_prolif.rds")
cd8_l2_subcluster  <- readRDS("../data/processed/L2/cd8_l2_subcluster.rds")
cd8_l2_nk  <- readRDS("../data/processed/L2/cd8_l2_nk.rds")
cd8_l2_unc  <- readRDS("../data/processed/L2/cd8_l2_unc.rds")
cd8_l1_full_filt  <- readRDS("../data/processed/L1/cd8_l1_full_filt.rds")
cd4_l3_naive  <- readRDS("../data/processed/L3/cd4_l3_naive.rds")
cd4_l3_tfh  <- readRDS("../data/processed/L3/cd4_l3_tfh.rds")
cd4_l3_th1th17  <- readRDS("../data/processed/L3/cd4_l3_th1_17.rds")
cd4_l3_nfkb  <- readRDS("../data/processed/L3/cd4_l3_nfkb.rds")
cd4_l3_th2  <- readRDS("../data/processed/L3/cd4_l3_th2.rds")
cd4_l3_treg  <- readRDS("../data/processed/L3/cd4_l3_treg.rds")
cd4_l3_isaghi  <- readRDS("../data/processed/L3/cd4_l3_isaghi.rds")
cd4_l3_proliferating  <- readRDS("../data/processed/L3/cd4_l3_proliferating.rds")
cd4_l3_temra  <- readRDS("../data/processed/L3/cd4_l3_temra.rds")
cd4_l2_unc  <- readRDS("../data/processed/L2/cd4_l2_unc.rds")

In [ ]:
cd4_l2_subcluster  <- readRDS("../data/processed/L2/cd4_subcluster.rds")

In [ ]:
cd4_l1_full_filt  <- readRDS("../data/processed/L1/cd4_l1_full_filt.rds")
cd8_l2_subcluster  <- readRDS("../data/processed/L2/cd8_l2_subcluster.rds")

In [ ]:
dataset_list  <- list(cd8_l3_tem, cd8_l3_naive, cd8_l3_tcm, cd8_l3_temra, cd8_l3_prolif, 
                    cd8_l2_nk, cd8_l2_unc, cd8_l1_full_filt,
                     cd4_l3_naive, cd4_l3_tfh, cd4_l3_th1th17, cd4_l3_nfkb, cd4_l3_th2,
                     cd4_l3_treg, cd4_l3_isaghi, cd4_l3_proliferating, cd4_l3_temra,
                     cd4_l2_unc, cd4_l2_subcluster, cd4_l1_full_filt, cd8_l2_subcluster)

names(dataset_list) <- c("cd8_l3_tem", "cd8_l3_naive", "cd8_l3_tcm", "cd8_l3_temra", "cd8_l3_prolif",
                       "cd8_l2_nk", "cd8_l2_unc", "cd8_l1_full_filt",
                        "cd4_l3_naive", "cd4_l3_tfh", "cd4_l3_th1th17", "cd4_l3_nfkb", "cd4_l3_th2",
                     "cd4_l3_treg", "cd4_l3_isaghi", "cd4_l3_proliferating", "cd4_l3_temra",
                     "cd4_l2_unc", "cd4_l2_subcluster", "cd4_l1_full_filt", "cd8_l2_subcluster")

In [ ]:
options(future.globals.maxSize = 2000 * 1024^2)

In [ ]:
dir.create("../tables/fold_change/")

In [ ]:
#plan("multisession", workers = 4)

for(i in 1:length(dataset_list)){

seurat_object  <- dataset_list[[i]]
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Disease

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], ".csv"))
}

In [ ]:
cd8_l2_subcluster$Condition2  %>% table

In [ ]:
#plan("multisession", workers = 8)

for(i in 1:length(dataset_list)){
###### Ctrl T0 vs Dia T0
seurat_object  <- subset(dataset_list[[i]], Condition %in% c("Ctrl T0", "Dia T0"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_Ctrl_vs_DiaT0.csv"))
    
####### Ctrl T0 vs Dia T1
seurat_object  <- subset(dataset_list[[i]], Condition %in% c("Ctrl T0", "Dia T1"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_Ctrl_vs_DiaT1.csv"))
    
    ####### Dia T0 vs Dia T1
seurat_object  <- subset(dataset_list[[i]], Condition %in% c("Dia T0", "Dia T1"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_DiaT0_vs_DiaT1.csv"))
    
    ####### Part remission
seurat_object  <- subset(dataset_list[[i]], Condition2 %in% c("Part_remission_0", "Part_remission_1"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition2

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_remission.csv"))
}

In [ ]:
md_for_remission  <- cd4_l1_full_filt@meta.data  %>% 
dplyr::filter(Condition %in% c("Dia T0"))  %>% 
group_by(Patient_ID, part_remission_y_n)  %>% 
tally %>% mutate(Condition2 = paste0("PR_",part_remission_y_n))  %>% 
dplyr::select(-n, -part_remission_y_n)

for( i in 1:length(dataset_list)) {
    
    dataset_list[[i]]$Condition2  <- NULL
    dataset_list[[i]]@meta.data  <- dataset_list[[i]]@meta.data  %>% 
    left_join(md_for_remission)  %>% mutate(Condition2 = paste(Condition2, Time))  %>% 
    mutate(Condition2 = ifelse(grepl(Condition2, pattern = "NA"), NA_character_,Condition2))
    
    print(dataset_list[[i]]$Condition2  %>% table)
    rownames(dataset_list[[i]]@meta.data)  <- colnames(dataset_list[[i]])
}

In [ ]:
md_for_ketoacidosis  <- cd4_l1_full_filt@meta.data  %>% 
dplyr::filter(Condition %in% c("Dia T0"))  %>% 
group_by(Patient_ID, ph_man)  %>% 
tally %>% mutate(Ketoacidosis = ifelse(ph_man<7.3,"Keto_1","Keto_0"))  %>% 
dplyr::select(-ph_man, -n)

for( i in 1:length(dataset_list)) {
    
    #dataset_list[[i]]$Ketoacidosis  <- NULL
    dataset_list[[i]]@meta.data  <- dataset_list[[i]]@meta.data  %>% 
    left_join(md_for_ketoacidosis)   %>% mutate(Keto_Time = paste(Ketoacidosis, Time))  %>% 
    mutate(Keto_Time = ifelse(grepl(Keto_Time, pattern = "NA"), NA_character_,Keto_Time))
    
    print(dataset_list[[i]]$Ketoacidosis  %>% table)
    rownames(dataset_list[[i]]@meta.data)  <- colnames(dataset_list[[i]])
}

In [ ]:
plan("multisession", workers = 4)

for(i in 1:length(dataset_list)){
###### Keto_Time
seurat_object  <- subset(dataset_list[[i]], Keto_Time %in% c("Keto_1 T0", "Keto_0 T0"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Keto_Time

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_ketoacidosis.csv"))
    
####### PR0 T0 vs PR0 T1
seurat_object  <- subset(dataset_list[[i]], Condition2 %in% c("PR_0 T0", "PR_0 T1"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition2

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_PR_0_T0_vs_PR_0_T1.csv"))
    
####### PR1 T0 vs PR1 T1
seurat_object  <- subset(dataset_list[[i]], Condition2 %in% c("PR_1 T0", "PR_1 T1"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition2

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_PR_1_T0_vs_PR_1_T1.csv"))
    
####### PR0 T0 vs PR1 T0
seurat_object  <- subset(dataset_list[[i]], Condition2 %in% c("PR_0 T0", "PR_1 T0"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition2

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_PR_0_T0_vs_PR_1_T0.csv"))
    
    ####### PR0 T1 vs PR1 T1
seurat_object  <- subset(dataset_list[[i]], Condition2 %in% c("PR_0 T1", "PR_1 T1"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition2

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change/", names(dataset_list)[i], "_PR_0_T1_vs_PR_1_T1.csv"))
    }

## FoldChanges DESeq

In [ ]:
dir.create("../tables/fold_change_deseq/")

In [ ]:
dataset_list

In [ ]:
seu  <- dataset_list[[1]]

In [ ]:
seu$Sample_ID  <- paste0("Id",seu$Sample_ID)

In [ ]:
seu$Sample_ID  %>% table

In [ ]:
avgexp = AggregateExpression(seu,
                           return.seurat = T, group.by = "Sample_ID", 
                          assay = "RNA", slot = "counts")

In [ ]:
colnames(avgexp)

In [ ]:
patient_metadata2  <- dataset_list[[i]]@meta.data  %>% 
    dplyr::select(Sample_ID, Patient_ID, Condition, Disease, Time, Experiment_ID)  %>% 
    unique  %>% ungroup

In [ ]:
patient_metadata2$Sample_ID  <- paste0("Id",patient_metadata2$Sample_ID)

In [ ]:
patient_metadata2$Sample_ID  %>% table

In [ ]:
colnames(avgexp)

In [ ]:
avgexp$Sample_ID

In [ ]:
    avgexp@meta.data  <- avgexp@meta.data   %>% left_join(patient_metadata2)
    rownames(avgexp@meta.data)  <- avgexp$Sample_ID
    

In [ ]:
avgexp@meta.data

In [ ]:
rownames(avgexp@meta.data)

In [ ]:
colnames(avgexp)

In [ ]:
seurat_object  <- subset(avgexp, Condition %in% c("Ctrl T0", "Dia T0") & 
                  Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))
DefaultAssay(seurat_object)  <- "RNA"

In [ ]:
#plan("multisession", workers = 8)

for(i in 1:length(dataset_list)){
    
    ## DESeq2
    seu  <- dataset_list[[i]]
    patient_metadata2  <- dataset_list[[i]]@meta.data  %>% 
    dplyr::select(Sample_ID, Patient_ID, Condition, Disease, Time, Experiment_ID)  %>% 
    unique  %>% ungroup
    patient_metadata2$Sample_ID  <- paste0("Id",patient_metadata2$Sample_ID)
    seu$Sample_ID  <- paste0("Id",seu$Sample_ID)
    avgexp = AggregateExpression(seu,
                           return.seurat = T, group.by = "Sample_ID", 
                          assay = "RNA", slot = "counts")
    
    avgexp@meta.data  <- avgexp@meta.data   %>% left_join(patient_metadata2)
    rownames(avgexp@meta.data)  <- avgexp$Sample_ID
    
    
###### Ctrl T0 vs Dia T0
seurat_object  <- subset(avgexp, Condition %in% c("Ctrl T0", "Dia T0") & 
                  Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))
DefaultAssay(seurat_object)  <- "RNA"

    
    Idents(seurat_object)  <- seurat_object$Condition

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "DESeq2", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change_deseq/", names(dataset_list)[i], "_Ctrl_vs_DiaT0.csv"))
    
####### Ctrl T0 vs Dia T1
seurat_object  <- subset(avgexp, Condition %in% c("Ctrl T0", "Dia T1") & 
                  Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "DESeq2", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change_deseq/", names(dataset_list)[i], "_Ctrl_vs_DiaT1.csv"))
    
    ####### Dia T0 vs Dia T1
seurat_object  <- subset(avgexp, Condition %in% c("Dia T0", "Dia T1") & 
                  Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "DESeq2", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change_deseq/", names(dataset_list)[i], "_DiaT0_vs_DiaT1.csv"))
 
}

In [ ]:
md_for_remission  <- cd4_l1_full_filt@meta.data  %>% 
dplyr::filter(Condition %in% c("Dia T0"))  %>% 
group_by(Patient_ID, part_remission_y_n)  %>% 
tally %>% mutate(Condition2 = paste0("PR_",part_remission_y_n))  %>% 
dplyr::select(-n, -part_remission_y_n)

for( i in 1:length(dataset_list)) {
    
    dataset_list[[i]]$Condition2  <- NULL
    dataset_list[[i]]@meta.data  <- dataset_list[[i]]@meta.data  %>% 
    left_join(md_for_remission)  %>% mutate(Condition2 = paste(Condition2, Time))  %>% 
    mutate(Condition2 = ifelse(grepl(Condition2, pattern = "NA"), NA_character_,Condition2))
    
    print(dataset_list[[i]]$Condition2  %>% table)
    rownames(dataset_list[[i]]@meta.data)  <- colnames(dataset_list[[i]])
}

In [ ]:
md_for_ketoacidosis  <- cd4_l1_full_filt@meta.data  %>% 
dplyr::filter(Condition %in% c("Dia T0"))  %>% 
group_by(Patient_ID, ph_man)  %>% 
tally %>% mutate(Ketoacidosis = ifelse(ph_man<7.3,"Keto_1","Keto_0"))  %>% 
dplyr::select(-ph_man, -n)

for( i in 1:length(dataset_list)) {
    
    #dataset_list[[i]]$Ketoacidosis  <- NULL
    dataset_list[[i]]@meta.data  <- dataset_list[[i]]@meta.data  %>% 
    left_join(md_for_ketoacidosis)   %>% mutate(Keto_Time = paste(Ketoacidosis, Time))  %>% 
    mutate(Keto_Time = ifelse(grepl(Keto_Time, pattern = "NA"), NA_character_,Keto_Time))
    
    print(dataset_list[[i]]$Ketoacidosis  %>% table)
    rownames(dataset_list[[i]]@meta.data)  <- colnames(dataset_list[[i]])
}

In [ ]:
plan("multisession", workers = 4)

for(i in 1:length(dataset_list)){
    
       ## DESeq2
    seu  <- dataset_list[[i]]
    patient_metadata2  <- seu@meta.data  %>% 
    dplyr::select(Sample_ID, Patient_ID, Condition, Disease, Time, Experiment_ID, Keto_Time, Condition2)  %>% 
    unique  %>% ungroup
    patient_metadata2$Sample_ID  <- paste0("Id",patient_metadata2$Sample_ID)
    seu$Sample_ID  <- paste0("Id",seu$Sample_ID)
    avgexp = AggregateExpression(seu,
                           return.seurat = T, group.by = "Sample_ID", 
                          assay = "RNA", slot = "counts")
    
    avgexp@meta.data  <- avgexp@meta.data   %>% left_join(patient_metadata2)
    rownames(avgexp@meta.data)  <- avgexp$Sample_ID
    
###### Keto_Time
seurat_object  <- subset(avgexp, Keto_Time %in% c("Keto_1 T0", "Keto_0 T0") & 
                  Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Keto_Time

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "DESeq2", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change_deseq/", names(dataset_list)[i], "_ketoacidosis.csv"))
 
####### PR0 T0 vs PR1 T0
seurat_object  <- subset(avgexp, Condition2 %in% c("PR_0 T0", "PR_1 T0") & 
                  Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Condition2

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "DESeq2", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

write.csv(fc.seurat_object, paste0("../tables/fold_change_deseq/", names(dataset_list)[i], "_PR_0_T0_vs_PR_1_T0.csv"))
    
    }

In [ ]:
list.files("../tables/fold_change_deseq/")

## All categories up and down

In [ ]:
pathways = msigdbr_collections()  %>% group_by(gs_cat)  %>% tally()  %>% pull(gs_cat)

In [ ]:
t2g <- msigdbr(species = "Homo sapiens", category = "C7") %>% 
          dplyr::select(gs_name, human_gene_symbol)

In [ ]:
get_pathway_df  <- function(pathway){
    t2g <- msigdbr(species = "Homo sapiens", category = pathway) %>% 
          dplyr::select(gs_name, human_gene_symbol)
    
    
    
    em2 <- GSEA(global_down_values, TERM2GENE = t2g)
    df_down  <- as.data.frame(em2)
                
    if(nrow(df_up)>0&nrow(df_down)>0){
        df_down$direction  <- "Down"
        df_up$direction  <- "Up"
        df_all  <- rbind(df_down, df_up)
        df_all$gs_cat  <- pathway
        
    } else {
        if(nrow(df_up)>0){
            df_up$direction  <- "Up"
            df_all  <- df_up
            df_all$gs_cat  <- pathway
    
    } else {
        if(nrow(df_down)>0){
            df_down$direction  <- "Down"
            df_all  <- df_down 
            df_all$gs_cat  <- pathway
    
    } else {
            df_all  <- NULL 
        } }}
    return(df_all)
}

In [ ]:
all_pathways  <- map(.x = pathways, .f = get_pathway_df)

In [ ]:
all_pathways2  <- bind_rows(all_pathways)

In [ ]:
all_pathways  <- map(.x = pathways, .f = get_pathway_df)

In [ ]:
all_pathways2  <- bind_rows(all_pathways)

In [ ]:
all_pathways2$core_enrichment   %>% nchar  %>% table

In [ ]:
711/4

In [ ]:
dir.create("../tables/pathways/")

In [ ]:
write.csv(all_pathways2, "../tables/pathways/gsea_global.csv")

In [ ]:
enriched_genes  <- all_pathways2  %>% 
dplyr::select(core_enrichment)  %>% 
separate(core_enrichment, sep = "/",
        into = paste0("v",1:138))



In [ ]:
enriched_genes

In [ ]:
geneIDs2  <- geneIDs1  %>% group_by(entrezid)  %>% slice_head(n = 1)

In [ ]:
geneIDs2  <- geneIDs2  %>% filter(!is.na(entrezid))

In [ ]:
df_enriched_genes  <- enriched_genes

In [ ]:
for(i in 1:138){
    df1  <- dplyr::select(df_enriched_genes, i)
    colnames(df1)  <- "entrezid"
    df1$entrezid  <- as.integer(df1$entrezid)
    df2  <- left_join(df1, geneIDs2)
    df_enriched_genes[[paste0("v",i)]]  <- df2$gene
}

In [ ]:
df_enriched_genes

In [ ]:
df_enriched_genes$genes  <- ""

In [ ]:
for(i in 1:411){
   df_enriched_genes[i, 139]  <-  gsub(pattern =  " / NA",
        replacement = "",
        x = paste(df_enriched_genes[i,1:138], collapse=" / ")) 
}

In [ ]:
df_enriched_genes

In [ ]:
all_pathways2$genes  <- df_enriched_genes$genes

In [ ]:
write.csv(all_pathways2, "../tables/pathways/gsea_global.csv")

Cluster profiler pathway heatmap

In [ ]:
t2g <- msigdbr(species = "Homo sapiens", category = "C7") 

In [ ]:
fgsea_sets<- t2g %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
fc.df  <- read_csv("../tables/fold_change/cd4_l1_full_filt.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)

In [ ]:
edo <- fgsea(pathways=t2g, stats=ranks)

In [ ]:
edo

In [ ]:
edox <- setReadable(edo, 'org.Hs.eg.db', 'ENTREZID')

In [ ]:
p2 <- heatplot(edox, foldChange=geneList, showCategory=5)

# GSEA all pathways

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")


In [ ]:
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)


In [ ]:
paths_fc

In [ ]:
paths_fc  <- paths_fc[which(grepl(paths_fc, pattern = "Ctrl")|grepl(paths_fc, pattern = "DiaT0_vs_DiaT1"))]

In [ ]:
dataset_names  <- dataset_names[which(grepl(dataset_names, pattern = "Ctrl")|grepl(dataset_names, pattern = "DiaT0_vs_DiaT1"))]

In [ ]:
dataset_names

In [ ]:
paths_fc

In [ ]:
fc.df  <- read_csv(paths_fc[1])  %>% as.data.frame()

In [ ]:
fc.df %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(x = cluster, pattern = "Dia")) 

## C7 pathways

In [ ]:
gsea_all_pathways  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    if(grepl(paths_fc[j], pattern = "Ctrl")){
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
        } else {
 fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia T0"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)       
    }
fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 return(fgseaRes)
    }

In [ ]:
gsea  <- map(.x = c(1:length(paths_fc)), .f = gsea_all_pathways)

In [ ]:
gseas  <- bind_rows(gsea)

In [ ]:
gseas  %>% arrange(padj)

In [ ]:
dir.create("../tables/gsea")

In [ ]:
gseas_df  <- gseas  %>% as.data.frame

In [ ]:
gseas_df  <- gseas_df  %>% mutate(
    leadingEdge2 = paste(leadingEdge))

In [ ]:
gseas_df$leadingEdge  <- NULL

In [ ]:
write.csv(gseas_df, "../tables/gsea/gsea_fold_changes_for_heatmap.csv")

In [ ]:
gseas_df

## DESEQ

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")


In [ ]:
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
paths_fc  <- list.files("../tables/fold_change_deseq/", full.names = T)
dataset_names  <- list.files("../tables/fold_change_deseq/", full.names = F)


In [ ]:
paths_fc

In [ ]:
paths_fc  <- paths_fc[which(grepl(paths_fc, pattern = "Ctrl")|grepl(paths_fc, pattern = "DiaT0_vs_DiaT1"))]

In [ ]:
dataset_names  <- dataset_names[which(grepl(dataset_names, pattern = "Ctrl")|grepl(dataset_names, pattern = "DiaT0_vs_DiaT1"))]

In [ ]:
dataset_names

In [ ]:
paths_fc

In [ ]:
fc.df  <- read_csv(paths_fc[1])  %>% as.data.frame()

In [ ]:
fc.df %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(x = cluster, pattern = "Dia")) 

In [ ]:
fc.df %>%  arrange(p_val_adj) 


## C7 pathways

In [ ]:
gsea_all_pathways  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    if(grepl(paths_fc[j], pattern = "Ctrl")){
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
        } else {
 fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia T0"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)       
    }
fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 return(fgseaRes)
    }

In [ ]:
gsea  <- map(.x = c(1:length(paths_fc)), .f = gsea_all_pathways)

In [ ]:
gseas  <- bind_rows(gsea)

In [ ]:
gseas  %>% arrange(padj)

In [ ]:
dir.create("../tables/gsea")

In [ ]:
gseas_df  <- gseas  %>% as.data.frame

In [ ]:
gseas_df  <- gseas_df  %>% mutate(
    leadingEdge2 = paste(leadingEdge))

In [ ]:
gseas_df$leadingEdge  <- NULL

In [ ]:
write.csv(gseas_df, "../tables/gsea/gsea_fold_changes_for_heatmap.csv")

## Hallmark pathways

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "H")


fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
names(fgsea_sets)

In [ ]:
gsea_all_pathways  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    if(grepl(paths_fc[j], pattern = "Ctrl")){
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
        } else {
 fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Dia T0"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)       
    }
fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 return(fgseaRes)
    }

In [ ]:
gsea  <- map(.x = c(1:length(paths_fc)), .f = gsea_all_pathways)

In [ ]:
gseas  <- bind_rows(gsea)

In [ ]:
gseas  %>% arrange(padj)

In [ ]:
gseas_df  <- gseas  %>% as.data.frame

In [ ]:
gseas_df  <- gseas_df  %>% mutate(
    leadingEdge2 = paste(leadingEdge))

In [ ]:
gseas_df$leadingEdge  <- NULL

In [ ]:
write.csv(gseas_df, "../tables/gsea/gsea_fold_changes_for_heatmap_hallmark.csv")

In [ ]:
gseas_df

## Heatmap immune and hallmark

In [ ]:
gseas_hallamrk  <- read_csv("../tables/gsea/gsea_fold_changes_for_heatmap_hallmark.csv")

In [ ]:
gseas_immune  <- read_csv("../tables/gsea/gsea_fold_changes_for_heatmap.csv")

In [ ]:
gseas_both  <- rbind(gseas_immune, gseas_hallamrk)

In [ ]:
## Enriched at least in 15 population-test_type combinations, padj at least 10-5

In [ ]:
gseas_selected  <- gseas_both  %>% dplyr::filter(padj < 0.00001)  %>% group_by(pathway)  %>% tally

In [ ]:
gseas_selected

In [ ]:
gseas_selected  <- gseas_selected  %>% dplyr::filter(n>5)  %>% pull(pathway)  %>% unique

In [ ]:
gseas_selected

In [ ]:
gseas_selected  %>% length

In [ ]:
gseas_selected2  <- gseas_both  %>% dplyr::filter(pathway %in% gseas_selected)  %>% 
dplyr::select(NES, dataset, pathway)  %>% 
mutate(dataset = gsub(dataset, pattern = "_Ctrl_vs_DiaT0", replacement = ".Ctrl_vs_DiaT0"))  %>% 
mutate(dataset = gsub(dataset, pattern = "_Ctrl_vs_DiaT1", replacement = ".Ctrl_vs_DiaT1"))  %>% 
mutate(dataset = gsub(dataset, pattern = "_DiaT0_vs_DiaT1", replacement = ".DiaT0_vs_DiaT1"))  %>% 
separate(dataset, into = c("population", "test_type"), sep = "\\.", remove = F)  %>% 
pivot_wider(names_from = "pathway", values_from = "NES", values_fill = 0) 

In [ ]:
gseas_mtx  <- gseas_selected2  %>% arrange(test_type, population)  %>% 
dplyr::select(-dataset, -population, -test_type)  %>% 
as.matrix()

In [ ]:
rownames(gseas_mtx)  <- gseas_selected2  %>% arrange(test_type, population)  %>% pull(dataset)

In [ ]:
library(pheatmap)

In [ ]:
colnames(gseas_mtx)

In [ ]:
!which(grepl(rownames(gseas_mtx), pattern = "subcluster")|
                         grepl(rownames(gseas_mtx), pattern = "_l1_"))

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
pheatmap(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F)),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
pheatmap(gseas_mtx, main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 40, height = 30,
                  fontsize = 9, filename = "../figures/heatmaps/gsea_heatmap.pdf")

In [ ]:
hm_ctrl_diat0  <- pheatmap(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = T,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]

In [ ]:
rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
pheatmap(gseas_mtx[c(rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "DiaT0_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)

pheatmap(gseas_mtx[c(rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "DiaT0_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 30, height = 15,
                  fontsize = 9, filename = "../figures/heatmaps/gsea_heatmap_ordered_with_hallmark.pdf")

### Try with insignificant white

In [ ]:
gseas_hallamrk  <- read_csv("../tables/gsea/gsea_fold_changes_for_heatmap_hallmark.csv")

In [ ]:
gseas_immune  <- read_csv("../tables/gsea/gsea_fold_changes_for_heatmap.csv")

In [ ]:
gseas_both  <- rbind(gseas_immune, gseas_hallamrk)

In [ ]:
gseas_both

In [ ]:
## Enriched at least in 15 population-test_type combinations, padj at least 10-5

In [ ]:
gseas_selected  <- gseas_both  %>% dplyr::filter(padj < 0.01)  %>% group_by(pathway)  %>% tally

In [ ]:
gseas_selected

In [ ]:
gseas_selected  <- gseas_selected  %>% dplyr::filter(n>10)  %>% pull(pathway)  %>% unique

In [ ]:
gseas_selected

In [ ]:
gseas_selected  %>% length

In [ ]:
gseas_both

In [ ]:
gseas_selected2  <- gseas_both  %>% dplyr::filter(pathway %in% gseas_selected)  %>% 
mutate(NES = ifelse(padj>0.05,NES/5,NES))  %>% 
dplyr::select(NES, dataset, pathway)  %>% 
mutate(dataset = gsub(dataset, pattern = "_Ctrl_vs_DiaT0", replacement = ".Ctrl_vs_DiaT0"))  %>% 
mutate(dataset = gsub(dataset, pattern = "_Ctrl_vs_DiaT1", replacement = ".Ctrl_vs_DiaT1"))  %>% 
mutate(dataset = gsub(dataset, pattern = "_DiaT0_vs_DiaT1", replacement = ".DiaT0_vs_DiaT1"))  %>% 
separate(dataset, into = c("population", "test_type"), sep = "\\.", remove = F)  %>% 
pivot_wider(names_from = "pathway", values_from = "NES", values_fill = 0) 

In [ ]:
gseas_mtx  <- gseas_selected2  %>% arrange(test_type, population)  %>% 
dplyr::select(-dataset, -population, -test_type)  %>% 
as.matrix()

In [ ]:
rownames(gseas_mtx)  <- gseas_selected2  %>% arrange(test_type, population)  %>% pull(dataset)

In [ ]:
library(pheatmap)

In [ ]:
colnames(gseas_mtx)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
hm_ctrl_diat0  <- pheatmap(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = T,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]

In [ ]:
rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
pheatmap(gseas_mtx[c(rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "DiaT0_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
pheatmap(gseas_mtx[c(rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "DiaT0_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)

pheatmap(gseas_mtx[c(rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT0")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "Ctrl_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)],
                    rownames(gseas_mtx[which((grepl(rownames(gseas_mtx), pattern = "subcluster")==F) &
                         (grepl(rownames(gseas_mtx), pattern = "_l1_")==F) &
                        grepl(rownames(gseas_mtx), pattern = "DiaT0_vs_DiaT1")),])[
    c(14, 15, 8, 6, 17, 7, 13, 3, 9, 4, 1, 10, 2, 5, 16, 11, 12)]),], 
         main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 45, height = 20,
                  fontsize = 9, filename = "../figures/heatmaps/gsea_heatmap_ordered_with_hallmark_white.pdf")

## Heatmap

In [ ]:
gseas_selected  <- gseas_both  %>% dplyr::filter(padj < 0.00000001)  %>% pull(pathway)  %>% unique

In [ ]:
gseas_selected  %>% length

In [ ]:
gseas_selected2  <- gseas  %>% dplyr::filter(pathway %in% gseas_selected)  %>% 
dplyr::select(NES, dataset, pathway)  %>% 
mutate(dataset = gsub(dataset, pattern = "_Ctrl_vs_DiaT0", replacement = ".Ctrl_vs_DiaT0"))  %>% 
mutate(dataset = gsub(dataset, pattern = "_Ctrl_vs_DiaT1", replacement = ".Ctrl_vs_DiaT1"))  %>% 
mutate(dataset = gsub(dataset, pattern = "_DiaT0_vs_DiaT1", replacement = ".DiaT0_vs_DiaT1"))  %>% 
separate(dataset, into = c("population", "test_type"), sep = "\\.", remove = F)  %>% 
pivot_wider(names_from = "pathway", values_from = "NES", values_fill = 0) 

In [ ]:
gseas_mtx  <- gseas_selected2  %>% arrange(test_type, population)  %>% 
dplyr::select(-dataset, -population, -test_type)  %>% 
as.matrix()

In [ ]:
rownames(gseas_mtx)  <- gseas_selected2  %>% arrange(test_type, population)  %>% pull(dataset)

In [ ]:
library(pheatmap)

In [ ]:
colnames(gseas_mtx)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
pheatmap(gseas_mtx, main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 18)
pheatmap(gseas_mtx, main = "", scale = "none", cluster_cols = T, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 40, height = 30,
                  fontsize = 9, filename = "../figures/heatmaps/gsea_heatmap.pdf")

In [ ]:
write.csv(colnames(gseas_mtx), "../tables/gsea/gseas_top_in_ehatmap_picture.csv")

In [ ]:
annotation_row = annot1

In [ ]:
levels = gseas  %>% group_by(dataset)  %>% slice_max(order_by = NES, n = 5) %>% pull(pathway)  %>% unique

In [ ]:

options(repr.plot.width = 20, repr.plot.height = 25)
gseas  %>% group_by(dataset)  %>% slice_max(order_by = NES, n = 5) %>% 
ggplot(aes(x = factor(pathway, levels = levels), y = NES)) + 
geom_point(aes(size = size, color = dataset, alpha = -padj)) + coord_flip() +
facet_wrap(~dataset, ncol = 1, drop = TRUE, scales = "free")

In [ ]:
gseas

In [ ]:
Labels = gseas  %>% 
dplyr::filter(dataset == "cd4_l1_full_filt")  %>% 
pull(pathway)

In [ ]:
Labels = ifelse(grepl(pattern = "TREG", Labels), Labels, NA_character_)

In [ ]:
Labels

In [ ]:
library(ggrepel)

ggrepel::geom_text_repel

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 9)

gseas  %>% 
dplyr::filter(dataset == "cd4_l1_full_filt")  %>% 
ggplot(aes(x = NES, y = padj)) + 
geom_point(aes(size = size, color = grepl(pathway, pattern = "TREG"), alpha = -padj)) +
geom_text_repel(aes(label = Labels))

## GSEA heatmap v07

In [ ]:
gseas_mtx  <- gseas_selected2  %>% arrange(test_type, population)  %>% 
dplyr::select(-dataset, -population, -test_type)  %>% 
as.matrix()

In [ ]:
which(colnames(gseas_selected2) %in% c("GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP",
"GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN",
                                       "GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN",
                                       "GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP"                                       ))

In [ ]:
gseas_selected2

In [ ]:
gseas_selected3  <- gseas_selected2  %>% arrange(test_type, population)  %>% 
dplyr::select(1,2,3,21,22,39,40)  %>% 
dplyr::select(-dataset, -population, -test_type)  %>% 
as.matrix()


In [ ]:
gseas_selected3

In [ ]:
pheatmap(gseas_selected3, colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50))

# GSEA in KetoAcidosis and Remission

In [ ]:
library(msigdbr)

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")


In [ ]:
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)


In [ ]:
paths_fc

In [ ]:
paths_fc  <- paths_fc[which(grepl(paths_fc, pattern = "ketoacidosis")|grepl(paths_fc, pattern = "PR_0_T0_vs_PR_1_T0"))]

In [ ]:
dataset_names  <- dataset_names[which(grepl(dataset_names, pattern = "ketoacidosis")|grepl(dataset_names, pattern = "PR_0_T0_vs_PR_1_T0"))]

In [ ]:
dataset_names

In [ ]:
paths_fc

In [ ]:
fc.df  <- read_csv(paths_fc[2])  %>% as.data.frame()

In [ ]:
fc.df

## C7 pathways

In [ ]:
gsea_all_pathways  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    if(grepl(paths_fc[j], pattern = "PR_0_T0")){
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "PR_0 T0"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
        } else {
 fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Keto_1 T0"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)       
    }
fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 return(fgseaRes)
    }

In [ ]:
gsea  <- map(.x = c(1:length(paths_fc)), .f = gsea_all_pathways)

In [ ]:
gseas  <- bind_rows(gsea)

In [ ]:
gseas  %>% arrange(padj)

In [ ]:
dir.create("../tables/gsea")

In [ ]:
gseas_df  <- gseas  %>% as.data.frame

In [ ]:
gseas_df  <- gseas_df  %>% mutate(
    leadingEdge2 = paste(leadingEdge))

In [ ]:
gseas_df$leadingEdge  <- NULL

In [ ]:
write.csv(gseas_df, "../tables/gsea/gsea_fold_changes_for_heatmap_ketoPR.csv")

In [ ]:
gseas_df

## Hallmark pathways

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "H")


fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
names(fgsea_sets)

In [ ]:
gsea_all_pathways  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    if(grepl(paths_fc[j], pattern = "PR_0_T0")){
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "PR_0 T0"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
        } else {
 fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(grepl(cluster, pattern = "Keto_1 T0"))  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)       
    }
fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 return(fgseaRes)
    }

In [ ]:
gsea  <- map(.x = c(1:length(paths_fc)), .f = gsea_all_pathways)

In [ ]:
gseas  <- bind_rows(gsea)

In [ ]:
gseas  %>% arrange(padj)

In [ ]:
gseas_df  <- gseas  %>% as.data.frame

In [ ]:
gseas_df  <- gseas_df  %>% mutate(
    leadingEdge2 = paste(leadingEdge))

In [ ]:
gseas_df$leadingEdge  <- NULL

In [ ]:
write.csv(gseas_df, "../tables/gsea/gsea_fold_changes_for_heatmap_hallmark_ketoPR.csv")

In [ ]:
gseas_df

## Heatmap of selected GSEAS V07

In [ ]:
paths_fc  <- list.files("../tables/gsea/", full.names = T)


In [ ]:
paths_fc  <- paths_fc[grepl(paths_fc, pattern = "gsea_fold_changes_for_heatmap")]

In [ ]:
paths_fc

In [ ]:
test  <- fread(paths_fc[3])

In [ ]:
test

In [ ]:
gseas  <- map(paths_fc, .f = fread)

In [ ]:
gseas  <- bind_rows(gseas)

In [ ]:
gseas$population  <- substr(gseas$dataset, 1,11)

In [ ]:
table(gseas$population)

In [ ]:
gseas  <- gseas  %>% mutate(population2 = ifelse(grepl(population, pattern = "_nk_"), substr(population,1,9), population))

In [ ]:
gseas$population2  %>% table

In [ ]:
gseas  <- gseas  %>% mutate(n1 = as.numeric(nchar(dataset)))  %>% 
mutate(n2 = n1-12)  %>% 
mutate(test_type = substr(dataset,n2,n1))  


In [ ]:
gseas$test_type  %>% table

In [ ]:
gseas

In [ ]:
gs  <- gseas  %>% group_by(dataset)  %>% slice_max(order_by = -log10(padj), n = 10)

In [ ]:
write.csv(gs, "../tables/gsea/v07heatmap_top_pathways.csv")

In [ ]:
plotEnrichment(fgsea_sets[["GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN") +

plotEnrichment(fgsea_sets[["GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_DN"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_DN") 




plotEnrichment(fgsea_sets[["GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_DN"]],
               ranks) + labs(title="GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_DN") + 

plotEnrichment(fgsea_sets[["GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_UP"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_UP") +

plotEnrichment(fgsea_sets[["GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_UP"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP") 



plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN") 


options(repr.plot.width = 5, repr.plot.height = 3)
plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN") 

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP") 

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE13738_RESTING_VS_BYSTANDER_ACTIVATED_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE13738_RESTING_VS_BYSTANDER_ACTIVATED_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_UP") 


In [ ]:
selected_pathways  <- c('HALLMARK_TNFA_SIGNALING_VIA_NFKB',
'HALLMARK_OXIDATIVE_PHOSPHORYLATION',

'HALLMARK_INTERFERON_ALPHA_RESPONSE',
'HALLMARK_INTERFERON_GAMMA_RESPONSE',
                        "GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_UP",
                        "GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP",
                        
"GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP",
"GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_DN",
'GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP',
'GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN'
)

In [ ]:
names(gseas$population2  %>% table) %in% c(
'cd8_l3_prol',
'cd8_l3_tcm_',
'cd4_l3_th1t7',
'cd4_l3_temr',
'cd8_l3_temr',
'cd4_l3_tfh_',
'cd8_l3_naiv',
'cd4_l3_naiv',
'cd4_l3_th2_',
'cd4_l3_nfkb',
'cd4_l2_unc_',
'cd4_l3_treg',
'cd4_l3_isag',
'cd4_l3_prol',
'cd8_l3_tem_',
'cd8_l2_nk',
'cd8_l2_unc_'    
)

In [ ]:
names(gseas$population2  %>% table)[!(names(gseas$population2  %>% table) %in% c(
'cd8_l3_prol',
'cd8_l3_tcm_',
'cd4_l3_th1t7',
'cd4_l3_temr',
'cd8_l3_temr',
'cd4_l3_tfh_',
'cd8_l3_naiv',
'cd4_l3_naiv',
'cd4_l3_th2_',
'cd4_l3_nfkb',
'cd4_l2_unc_',
'cd4_l3_treg',
'cd4_l3_isag',
'cd4_l3_prol',
'cd8_l3_tem_',
'cd8_l2_nk',
'cd8_l2_unc_'    
))]

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 24)
gseas  %>% dplyr::filter(pathway %in% selected_pathways &
                        ((population2 %in% c("cd8_l1_full","cd4_l1_full","cd4_l2_subc","cd8_l2_subc")) == F))  %>% 
ggplot(aes(x = factor(population2, levels = c(
'cd8_l3_prol',
'cd8_l3_tcm_',
'cd4_l3_th1t',
'cd4_l3_temr',
'cd8_l3_temr',
'cd4_l3_tfh_',
'cd8_l3_naiv',
'cd4_l3_naiv',
'cd4_l3_th2_',
'cd4_l3_nfkb',
'cd4_l2_unc_',
'cd4_l3_treg',
'cd4_l3_isag',
'cd4_l3_prol',
'cd8_l3_tem_',
'cd8_l2_nk',
'cd8_l2_unc_'    
)), y = pathway)) +
geom_point(aes(size = -log(padj), color = NES, alpha = padj<0.05)) +
scale_alpha_manual(values = c(0,1)) +
facet_grid(cols = vars(test_type), space = "free", scales = "free") +
scale_color_gradient2(low = "blue", mid = "white", high = "red") + 
xlab("") + ylab("") +
theme_classic() + theme(axis.text.x = element_text(angle = 90),
                       axis.ticks.x = element_blank()) + ggtheme()
ggsave("../figures/gsea/heatmap_gsea.svg", width = 55, height = 16, units = "cm")

In [ ]:
selected_pathways  <- c(
 'HALLMARK_INTERFERON_ALPHA_RESPONSE',
'HALLMARK_INTERFERON_GAMMA_RESPONSE',
'GSE13485_DAY7_VS_DAY21_YF17D_VACCINE_PBMC_UP',
'GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP',
'GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN')

options(repr.plot.height = 5, repr.plot.width = 24)
gseas  %>% dplyr::filter(pathway %in% selected_pathways)  %>% 
ggplot(aes(x = population, y = factor(pathway, levels = selected_pathways))) +
geom_point(aes(size = -log(padj), color = NES, alpha = padj<0.05)) +
scale_alpha_manual(values = c(0,1)) +
facet_grid(cols = vars(test_type), space = "free", scales = "free") +
scale_color_gradient2(low = "blue", mid = "white", high = "red") + 
theme_classic() + theme(axis.text.x = element_text(angle = 90)) + ggtheme()

In [ ]:
selected_pathways  <- c(
    "GSE25087_FETAL_VS_ADULT_TCONV_UP",
                       "GSE36476_CTRL_VS_TSST_ACT_40H_MEMORY_CD4_TCELL_YOUNG_DN",
                       "GSE22886_NAIVE_CD4_TCELL_VS_48H_ACT_TH2_DN",
    "GSE24634_NAIVE_CD4_TCELL_VS_DAY7_IL4_CONV_TREG_UP",
"GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_UP",
    "GSE36476_CTRL_VS_TSST_ACT_72H_MEMORY_CD4_TCELL_YOUNG_UP")

options(repr.plot.height = 5, repr.plot.width = 24)
gseas  %>% dplyr::filter(pathway %in% selected_pathways)  %>% 
ggplot(aes(x = population, y = factor(pathway, levels = selected_pathways))) +
geom_point(aes(size = -log(padj), color = NES, alpha = padj<0.05)) +
scale_alpha_manual(values = c(0,1)) +
facet_grid(cols = vars(test_type), space = "free", scales = "free") +
scale_color_gradient2(low = "blue", mid = "white", high = "red") + 
theme_classic() + theme(axis.text.x = element_text(angle = 90)) + ggtheme()

In [ ]:
gseas  %>% dplyr::filter(test_type == "_ketoacidosis" & padj < 0.05 & NES > 0)  %>% 
group_by(pathway)  %>% tally()  %>% arrange(desc(n))

In [ ]:
gseas  %>% dplyr::filter(pathway %in% c("HALLMARK_INTERFERON_ALPHA_RESPONSE"))

### GSEA DESeq

### GSEA NFKB

In [ ]:
paths_pathways  <- list.files("../data/nfkb/", full.names = T)
pathway_names  <- list.files("../data/nfkb/", full.names = F)


In [ ]:
nfkb_genes1  <- str_to_upper(read_csv("../data/nfkb/Paper_I1_cluster.csv")$Genes)
nfkb_genes2  <- str_to_upper(read_csv("../data/nfkb/Paper_I2_cluster.csv")$Genes)
nfkb_genes3  <- str_to_upper(read_csv("../data/nfkb/Paper_I3_cluster.csv")$Genes)
nfkb_genes6  <- str_to_upper(read_csv("../data/nfkb/Paper_I6_cluster.csv")$Genes)

paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

In [ ]:
gsea_nfkb  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
fgseaRes <- fgsea(pathways = list(nfkb_genes1 = nfkb_genes1, nfkb_genes2 = nfkb_genes2, 
                                 nfkb_genes3 = nfkb_genes3, nfkb_genes6 = nfkb_genes6), 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")   
 return(fgseaRes)
    }


In [ ]:
paths_fc

In [ ]:
gsea  <- purrr::map(c(1:length(paths_fc)), .f = gsea_nfkb)

In [ ]:
gseas_nfkb  <- bind_rows(gsea)

In [ ]:
gseas_nfkb  %>% arrange(padj)

## GSEA Dia vs. Healthy

### CD4

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")

In [ ]:
fgsea_sets_hs_c7 <- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

In [ ]:
paths_fc

In [ ]:
paths_fc[10]

In [ ]:
fc.df  <- read_csv(paths_fc[10])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd4_t0 <- deframe(fc.df)

In [ ]:
ranks_cd4_t0  

In [ ]:
fgseaRes <- fgsea(pathways = fgsea_sets_hs_c7, 
                 stats = ranks_cd4_t0)

In [ ]:
fgseaRes <- fgsea(pathways = fgsea_sets_hs_c7, 
                 stats = ranks_cd4_t0)

fgseaRes  %>% dplyr::filter(grepl(pathway, pattern = "NAIVE_VS_EFF_MEMORY"))

In [ ]:
fgseaRes  %>% arrange(pval)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)
plotEnrichment(fgsea_sets_hs_c7[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]],
               ranks_cd4_t0) + labs(title="GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN") +
ylim(c(-0.9,0.4))
ggsave(filename = paste0("../figures/gsea/GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN.svg"), 
       width = 9.5, height = 8, units = "cm")


In [ ]:
plotEnrichment(fgsea_sets_hs_c7[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP"]],
               ranks_cd4_t0) + labs(title="GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP") +
ylim(c(-0.9,0.4))

ggsave(filename = paste0("../figures/gsea/GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP.svg"), 
       width = 9.5, height = 8, units = "cm")


In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C2")
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
plotEnrichment(fgsea_sets[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]],
               ranks_cd4_t0) + labs(title="REACTOME_INTERFERON_ALPHA_BETA_SIGNALING") +
ylim(c(-0.9,0.4))
ggsave(filename = paste0("../figures/gsea/REACTOME_INTERFERON_ALPHA_BETA_SIGNALING_CD4.svg"), 
       width = 9.5, height = 8, units = "cm")

### CD8

In [ ]:
grep(paths_fc, pattern = "cd8_l1_full_filt", value = T)

In [ ]:
fc.df  <- read_csv("../tables/fold_change/cd8_l1_full_filt.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd8_t0 <- deframe(fc.df)

ranks_cd8_t0  

In [ ]:
fgseaRes <- fgsea(pathways = fgsea_sets_hs_c7, 
                 stats = ranks_cd8_t0)

In [ ]:
fgseaRes  %>% dplyr::filter(grepl(pathway, pattern = "GOLDRATH"))

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)
plotEnrichment(fgsea_sets_hs_c7[["GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN"]],
               ranks_cd8_t0) + labs(title="GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN") +
ylim(c(-0.7,0.4))

ggsave(filename = paste0("../figures/gsea/GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN.svg"), 
       width = 9.5, height = 8, units = "cm")


In [ ]:
plotEnrichment(fgsea_sets_hs_c7[["GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_UP"]],
               ranks_cd8_t0) + labs(title="GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_UP") +
ylim(c(-0.7,0.4))

ggsave(filename = paste0("../figures/gsea/GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_UP.svg"), 
       width = 9.5, height = 8, units = "cm")


In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C2")
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
plotEnrichment(fgsea_sets[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]],
               ranks_cd8_t0) + labs(title="REACTOME_INTERFERON_ALPHA_BETA_SIGNALING") +
ylim(c(-0.7,0.4))

ggsave(filename = paste0("../figures/gsea/REACTOME_INTERFERON_ALPHA_BETA_SIGNALING_CD8.svg"), 
       width = 9.5, height = 8, units = "cm")


# GSEA v06

## C2

### Dia T0 vs Ctrl

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT0")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT0")]

dataset_names
paths_fc

In [ ]:
dataset_names[1]

In [ ]:
dataset_names[13]

In [ ]:
gsea_all_pathways  <- function(j, category) {
    
     m_df<- msigdbr(species = "Homo sapiens", category = category)
     fgsea_sets <- m_df %>% split(x = .$gene_symbol, f = .$gs_name)
    fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
    dplyr::filter(grepl(cluster, pattern = "Dia"))  %>% 
      dplyr::select(gene, avg_log2FC)
    ranks <- deframe(fc.df)
    fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
     fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
     return(fgseaRes)
    
    }


In [ ]:
gsea_t0_cd4_c2  <- gsea_all_pathways(1, "C2")

In [ ]:
gsea_t0_cd4_c2  <- bind_rows(gsea_t0_cd4_c2)
gsea_t0_cd4_c2  %>% arrange(padj)

In [ ]:
gsea_t0_cd4_c2$leadingEdge  <- NULL

In [ ]:
write.csv(gsea_t0_cd4_c2  %>% arrange(padj), "../tables/gsea/gsea_t0_cd4_c2.csv")

In [ ]:
gsea_t0_cd8_c2 <- gsea_all_pathways(13, "C2")

In [ ]:
gsea_t0_cd8_c2  <- bind_rows(gsea_t0_cd8_c2)
gsea_t0_cd8_c2  %>% arrange(padj)

In [ ]:
gsea_t0_cd8_c2$leadingEdge  <- NULL

write.csv(gsea_t0_cd8_c2  %>% arrange(padj), "../tables/gsea/gsea_t0_cd8_c2.csv")

### Dia T1 vs Ctrl

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT1")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT1")]

dataset_names
paths_fc

In [ ]:
dataset_names[1]

In [ ]:
dataset_names[13]

In [ ]:
gsea_t1_cd4_c2  <- gsea_all_pathways(1, "C2")

In [ ]:
gsea_t1_cd4_c2  <- bind_rows(gsea_t1_cd4_c2)
gsea_t1_cd4_c2  %>% arrange(padj)

In [ ]:
gsea_t1_cd4_c2$leadingEdge  <- NULL

In [ ]:
write.csv(gsea_t1_cd4_c2  %>% arrange(padj), "../tables/gsea/gsea_t1_cd4_c2.csv")

In [ ]:
gsea_t1_cd8_c2 <- gsea_all_pathways(13, "C2")

In [ ]:
gsea_t1_cd8_c2  <- bind_rows(gsea_t1_cd8_c2)
gsea_t1_cd8_c2  %>% arrange(padj)

In [ ]:
gsea_t1_cd8_c2$leadingEdge  <- NULL

write.csv(gsea_t1_cd8_c2  %>% arrange(padj), "../tables/gsea/gsea_t1_cd8_c2.csv")

## C7

### Dia T0 vs Ctrl

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT0")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT0")]

dataset_names
paths_fc

In [ ]:
dataset_names[1]

In [ ]:
dataset_names[13]

In [ ]:
gsea_all_pathways  <- function(j, category) {
    
     m_df<- msigdbr(species = "Homo sapiens", category = category)
     fgsea_sets <- m_df %>% split(x = .$gene_symbol, f = .$gs_name)
    fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
    fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
    dplyr::filter(grepl(cluster, pattern = "Dia"))  %>% 
      dplyr::select(gene, avg_log2FC)
    ranks <- deframe(fc.df)
    fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
     fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
     return(fgseaRes)
    
    }


In [ ]:
gsea_t0_cd4_c7  <- gsea_all_pathways(1, "C7")

In [ ]:
gsea_t0_cd4_c7  <- bind_rows(gsea_t0_cd4_c7)
gsea_t0_cd4_c7  %>% arrange(padj)

In [ ]:
gsea_t0_cd4_c7$leadingEdge  <- NULL

In [ ]:
write.csv(gsea_t0_cd4_c7  %>% arrange(padj), "../tables/gsea/gsea_t0_cd4_c7.csv")

In [ ]:
gsea_t0_cd8_c7 <- gsea_all_pathways(13, "C7")

In [ ]:
gsea_t0_cd8_c7  <- bind_rows(gsea_t0_cd8_c7)
gsea_t0_cd8_c7  %>% arrange(padj)

In [ ]:
gsea_t0_cd8_c7$leadingEdge  <- NULL

write.csv(gsea_t0_cd8_c7  %>% arrange(padj), "../tables/gsea/gsea_t0_cd8_c7.csv")

### Dia T1 vs Ctrl

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT1")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT1")]

dataset_names
paths_fc

In [ ]:
dataset_names[1]

In [ ]:
dataset_names[13]

In [ ]:
gsea_t1_cd4_c7  <- gsea_all_pathways(1, "C7")

In [ ]:
gsea_t1_cd4_c7  <- bind_rows(gsea_t1_cd4_c7)
gsea_t1_cd4_c7  %>% arrange(padj)

In [ ]:
gsea_t1_cd4_c7$leadingEdge  <- NULL

In [ ]:
write.csv(gsea_t1_cd4_c7  %>% arrange(padj), "../tables/gsea/gsea_t1_cd4_c7.csv")

In [ ]:
gsea_t1_cd8_c7 <- gsea_all_pathways(13, "C7")

In [ ]:
gsea_t1_cd8_c7  <- bind_rows(gsea_t1_cd8_c7)
gsea_t1_cd8_c7  %>% arrange(padj)

In [ ]:
gsea_t1_cd8_c7$leadingEdge  <- NULL

write.csv(gsea_t1_cd8_c7  %>% arrange(padj), "../tables/gsea/gsea_t1_cd8_c7.csv")

# Heatmap

In [ ]:
paths_fc2  <- list.files("../tables/fold_change", full.names = T)
dataset_names2  <- list.files("../tables/fold_change", full.names = F)


In [ ]:
paths_fc2

In [ ]:
paths_fc  <- paths_fc2[grepl(paths_fc2, pattern = "l3")|grepl(paths_fc2, pattern = "cd8_l2_unc")|
                      grepl(paths_fc2, pattern = "cd4_l2_unc")|grepl(paths_fc2, pattern = "cd8_l2_nk")]

In [ ]:
paths_fc

# GSEA figure

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C2")

fgsea_sets_hs_c2 <- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")

fgsea_sets_hs_c7 <- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

## Ranks T0

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)


paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT0")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT0")]

dataset_names

paths_fc

In [ ]:
fc.df  <- read_csv(paths_fc[1])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T0")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd4_t0 <- deframe(fc.df)

In [ ]:
fc.df  <- read_csv(paths_fc[13])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T0")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd8_t0 <- deframe(fc.df)

In [ ]:
ranks_cd8_t0

## T1

In [ ]:
paths_fc2  <- list.files("../tables/fold_change", full.names = T)
dataset_names2  <- list.files("../tables/fold_change", full.names = F)


paths_fc2  <- paths_fc2[grepl(paths_fc2, pattern = "_Ctrl_vs_DiaT1")]
dataset_names2  <- dataset_names2[grepl(dataset_names2, pattern = "_Ctrl_vs_DiaT1")]

In [ ]:
paths_fc2
dataset_names2


In [ ]:
fc.df  <- read_csv(paths_fc2[1])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T1")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd4_t1 <- deframe(fc.df)



In [ ]:
fc.df  <- read_csv(paths_fc2[13])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T1")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd8_t1 <- deframe(fc.df)

In [ ]:
paths_fc2[13]

In [ ]:
ranks_cd8_t1

In [ ]:
plot_four_gsea_condition  <- function(gsea, fgsea_set){
   fgsea_sets  <- fgsea_set
    p  <- plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd8_t0) + labs(title="CD8 Dia T0 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd8_t1) + labs(title="CD8 Dia T1 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd4_t0) + labs(title="CD4 Dia T0 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd4_t1) + labs(title="CD4 Dia T1 vs Ctrl.") + ylim(-0.8,0.8) +  ggtheme() + 
    plot_annotation(title = gsea)
    
    ggsave(p, filename = paste0("../figures/gsea/",gsea,".svg"), width = 24, height = 24, units = "cm")
    return(p)
}

In [ ]:
fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]]

In [ ]:
fgsea_sets_hs_c7[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]]

In [ ]:
intersect(fgsea_sets_hs_c7[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]], 
          fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]])

In [ ]:
plot_four_gsea_condition("REACTOME_INTERFERON_ALPHA_BETA_SIGNALING", fgsea_sets_hs_c2)

In [ ]:
plot_four_gsea_condition("GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP", fgsea_sets_hs_c7)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
plot_four_gsea_condition("WP_INSULIN_SIGNALING", fgsea_sets_hs_c2)


In [ ]:
plot_four_gsea_condition("GSE13485_CTRL_VS_DAY7_YF17D_VACCINE_PBMC_DN", fgsea_sets_hs_c7)


In [ ]:
plot_four_gsea_condition("GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN", fgsea_sets_hs_c7)


In [ ]:
plot_four_gsea_condition("GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE9650_NAIVE_VS_EFF_CD8_TCELL_UP", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD8_TCELL_VS_NKCELL_UP", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD8_TCELL_VS_NKCELL_DN", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD4_TCELL_VS_NKCELL_DN", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD4_TCELL_VS_NKCELL_UP", fgsea_sets_hs_c7)

In [ ]:
intersect(fgsea_sets_hs_c7[["GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN"]], 
          fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]])

In [ ]:

    which((fgsea_sets_hs_c7[["GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN"]] %in% 
      intersect(fgsea_sets_hs_c7[["GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN"]], 
          fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]])) == F)

In [ ]:
keep  <- 
    which((fgsea_sets_hs_c7[["GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN"]] %in% 
      intersect(fgsea_sets_hs_c7[["GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN"]], 
          fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]])) == F)

In [ ]:
fgsea_sets_hs_c7[["GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN"]][keep]                    

In [ ]:
plot_four_gsea_condition  <- function(genes){
   #fgsea_sets  <- fgsea_set
    p  <- plotEnrichment(genes,
               ranks_cd8_t0) + labs(title="CD8 Dia T0 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(genes,
               ranks_cd8_t1) + labs(title="CD8 Dia T1 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(genes,
               ranks_cd4_t0) + labs(title="CD4 Dia T0 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(genes,
               ranks_cd4_t1) + labs(title="CD4 Dia T1 vs Ctrl.") + ylim(-0.8,0.8) +  ggtheme() + 
    plot_annotation(title = "")
    
    #ggsave(p, filename = paste0("../figures/gsea/",gsea,".svg"), width = 24, height = 24, units = "cm")
    return(p)
}

In [ ]:
genes_cd8  <- fgsea_sets_hs_c7[["GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN"]][keep]                    

In [ ]:
plot_four_gsea_condition(genes_cd8)

In [ ]:
keep  <- 
    which((fgsea_sets_hs_c7[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]] %in% 
      intersect(fgsea_sets_hs_c7[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]], 
          fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]])) == F)

In [ ]:
genes_cd4  <- fgsea_sets_hs_c7[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]][keep]                    

In [ ]:
genes_cd4

In [ ]:
plot_four_gsea_condition(genes_cd4)

In [ ]:
keep  <- 
    which((fgsea_sets_hs_c7[["GSE13485_CTRL_VS_DAY7_YF17D_VACCINE_PBMC_DN"]] %in% 
      intersect(fgsea_sets_hs_c7[["GSE13485_CTRL_VS_DAY7_YF17D_VACCINE_PBMC_DN"]], 
          fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]])) == F)

genes_cd4  <- fgsea_sets_hs_c7[["GSE13485_CTRL_VS_DAY7_YF17D_VACCINE_PBMC_DN"]][keep]                    

genes_cd4

plot_four_gsea_condition(genes_cd4)

In [ ]:
fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]]

In [ ]:
fgsea_sets_hs_c7[["GSE13485_CTRL_VS_DAY7_YF17D_VACCINE_PBMC_DN"]]

In [ ]:
intersect(fgsea_sets_hs_c7[["GSE13485_CTRL_VS_DAY7_YF17D_VACCINE_PBMC_DN"]], 
          fgsea_sets_hs_c2[["REACTOME_INTERFERON_ALPHA_BETA_SIGNALING"]])

# GSEA figure DESeq

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C2")

fgsea_sets_hs_c2 <- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")

fgsea_sets_hs_c7 <- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

## Ranks T0

In [ ]:
paths_fc  <- list.files("../tables/fold_change_deseq/", full.names = T)
dataset_names  <- list.files("../tables/fold_change_deseq/", full.names = F)


paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT0")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT0")]

dataset_names

paths_fc

In [ ]:
fc.df  <- read_csv(paths_fc[1])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T0")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd4_t0 <- deframe(fc.df)

In [ ]:
ranks_cd4_t0  

In [ ]:
fgseaRes <- fgsea(pathways = fgsea_sets_hs_c7, 
                 stats = ranks_cd4_t0)

In [ ]:
fgseaRes  %>% arrange(pval)

In [ ]:
fc.df  <- read_csv(paths_fc[13])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T0")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd8_t0 <- deframe(fc.df)

In [ ]:
ranks_cd8_t0

In [ ]:
gseaRes <- fgsea(pathways = fgsea_sets_hs_c7, 
                 stats = ranks_cd8_t0)

In [ ]:
gseaRes  %>% arrange(pval)

## T1

In [ ]:
paths_fc2  <- list.files("../tables/fold_change_deseq/", full.names = T)
dataset_names2  <- list.files("../tables/fold_change_deseq/", full.names = F)


paths_fc2  <- paths_fc2[grepl(paths_fc2, pattern = "_Ctrl_vs_DiaT1")]
dataset_names2  <- dataset_names2[grepl(dataset_names2, pattern = "_Ctrl_vs_DiaT1")]

In [ ]:
paths_fc2
dataset_names2


In [ ]:
fc.df  <- read_csv(paths_fc2[1])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T1")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd4_t1 <- deframe(fc.df)



In [ ]:
fc.df  <- read_csv(paths_fc2[13])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T1")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks_cd8_t1 <- deframe(fc.df)

In [ ]:
paths_fc2[13]

In [ ]:
ranks_cd8_t1

In [ ]:
ggtheme <- function() {
  theme(
    axis.text = element_text(size = 20),
    axis.title = element_text(size = 20),
    text = element_text(size = 20, colour = "black"),
    legend.text = element_text(size = 20),
    legend.key.size =  unit(10, units = "points")
    
  )
}


In [ ]:
plot_four_gsea_condition  <- function(gsea, fgsea_set){
   fgsea_sets  <- fgsea_set
    p  <- plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd8_t0) + labs(title="CD8 Dia T0 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd8_t1) + labs(title="CD8 Dia T1 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd4_t0) + labs(title="CD4 Dia T0 vs Ctrl.") + ylim(-0.8,0.8) + ggtheme() +
    plotEnrichment(fgsea_sets[[gsea]],
               ranks_cd4_t1) + labs(title="CD4 Dia T1 vs Ctrl.") + ylim(-0.8,0.8) +  ggtheme() + 
    plot_annotation(title = gsea)
    
    ggsave(p, filename = paste0("../figures/gsea/",gsea,".svg"), width = 24, height = 24, units = "cm")
    return(p)
}

In [ ]:
plot_four_gsea_condition("REACTOME_INTERFERON_ALPHA_BETA_SIGNALING", fgsea_sets_hs_c2)

In [ ]:
plot_four_gsea_condition("GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP", fgsea_sets_hs_c7)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
plot_four_gsea_condition("WP_INSULIN_SIGNALING", fgsea_sets_hs_c2)


In [ ]:
plot_four_gsea_condition("GSE13485_CTRL_VS_DAY7_YF17D_VACCINE_PBMC_DN", fgsea_sets_hs_c7)


In [ ]:
plot_four_gsea_condition("GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN", fgsea_sets_hs_c7)


In [ ]:
plot_four_gsea_condition("GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE9650_NAIVE_VS_EFF_CD8_TCELL_UP", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE9650_NAIVE_VS_EFF_CD8_TCELL_DN", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD8_TCELL_VS_NKCELL_UP", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD8_TCELL_VS_NKCELL_DN", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD4_TCELL_VS_NKCELL_DN", fgsea_sets_hs_c7)

In [ ]:
plot_four_gsea_condition("GSE22886_NAIVE_CD4_TCELL_VS_NKCELL_UP", fgsea_sets_hs_c7)

# GSEA heatmap

## GSEA - Dia T0 vs Ctrl T0

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")


In [ ]:
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)


In [ ]:
paths_fc

In [ ]:
paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT0")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT0")]

In [ ]:
dataset_names

In [ ]:
paths_fc

In [ ]:
gsea_all_pathways  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T0")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
fgseaRes <- fgseaRes  %>% dplyr::filter(padj < 0.05)
 return(fgseaRes)
    }

In [ ]:
gsea_diat0_vs_ctrl  <- map(.x = c(1:length(paths_fc)), .f = gsea_all_pathways)

In [ ]:
gseas_diat0_vs_ctrl  <- bind_rows(gsea_diat0_vs_ctrl)

In [ ]:
gseas_diat0_vs_ctrl  %>% arrange(padj)

In [ ]:
gseas_diat0_vs_ctrl  %>% as.data.frame

In [ ]:
gseas_diat0_vs_ctrl_df  <- gseas_diat0_vs_ctrl  %>% as.data.frame

In [ ]:
gseas_diat0_vs_ctrl  <- gseas_diat0_vs_ctrl  %>% mutate(
    leadingEdge2 = paste(leadingEdge))

In [ ]:
gseas_diat0_vs_ctrl$leadingEdge  <- NULL

In [ ]:
write.csv(gseas_diat0_vs_ctrl, "../tables/gsea/gsea_fold_changes_diat0_vsctrl.csv")

## GSEA - Dia T1 vs Ctrl T1

In [ ]:
m_df<- msigdbr(species = "Homo sapiens", category = "C7")


In [ ]:
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)


In [ ]:
paths_fc  <- paths_fc[grepl(paths_fc, pattern = "_Ctrl_vs_DiaT1")]
dataset_names  <- dataset_names[grepl(dataset_names, pattern = "_Ctrl_vs_DiaT1")]

In [ ]:
dataset_names

In [ ]:
paths_fc

In [ ]:
gsea_all_pathways  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T1")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
fgseaRes <- fgsea(pathways = fgsea_sets, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
fgseaRes <- fgseaRes  %>% dplyr::filter(padj < 0.05)
 return(fgseaRes)
    }

In [ ]:
gsea_diat1_vs_ctrl  <- map(.x = c(1:length(paths_fc)), .f = gsea_all_pathways)

In [ ]:
gseas_diat1_vs_ctrl  <- bind_rows(gsea_diat1_vs_ctrl)

In [ ]:
gseas_diat1_vs_ctrl  %>% arrange(padj)

In [ ]:
gseas_diat1_vs_ctrl  %>% as.data.frame

In [ ]:
gseas_diat1_vs_ctrl_df  <- gseas_diat1_vs_ctrl  %>% as.data.frame

In [ ]:
gseas_diat1_vs_ctrl  <- gseas_diat1_vs_ctrl  %>% mutate(
    leadingEdge2 = paste(leadingEdge))

In [ ]:
gseas_diat1_vs_ctrl$leadingEdge  <- NULL

In [ ]:
write.csv(gseas_diat1_vs_ctrl, "../tables/gsea/gsea_fold_changes_diat1_vsctrl.csv")

In [ ]:
gseas  <- read_delim("../tables/gsea/gsea_fold_changes_dia_vsctrl.csv")

In [ ]:
gseas

## GSEA Naive vs effector



### CD4

In [ ]:
# CD4L1
fc.df  <- read_csv("../tables/fold_change/cd4_l1_full_filt.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)

In [ ]:
plotEnrichment(fgsea_sets[["GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP"]],
               ranks) + labs(title="GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP") + 

plotEnrichment(fgsea_sets[["GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_DN") +

plotEnrichment(fgsea_sets[["GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_DN"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_DN") 




In [ ]:
plotEnrichment(fgsea_sets[["GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_DN"]],
               ranks) + labs(title="GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_DN") + 

plotEnrichment(fgsea_sets[["GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_UP"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_EFF_CD8_TCELL_UP") +

plotEnrichment(fgsea_sets[["GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_UP"]],
               ranks) + labs(title="GOLDRATH_NAIVE_VS_MEMORY_CD8_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP") 



In [ ]:
plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_CENT_MEMORY_CD4_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_EFF_MEMORY_CD4_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN") 


In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)
plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_DN") 

In [ ]:
plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP") 

In [ ]:
plotEnrichment(fgsea_sets[["GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE11057_NAIVE_VS_MEMORY_CD4_TCELL_UP") +

plotEnrichment(fgsea_sets[["GSE13738_RESTING_VS_BYSTANDER_ACTIVATED_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE13738_RESTING_VS_BYSTANDER_ACTIVATED_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE17974_0H_VS_24H_IN_VITRO_ACT_CD4_TCELL_UP") 


## T1DM and Lupus signatures

In [ ]:
i = 11

In [ ]:
seurat_object  <- dataset_list[[i]]
DefaultAssay(seurat_object)  <- "RNA"
Idents(seurat_object)  <- seurat_object$Disease

fc.seurat_object <- FindAllMarkers(seurat_object, 
                          test.use = "wilcox", 
                          logfc.threshold = -Inf, min.pct = -Inf, min.diff.pct = -Inf,
                                   only.pos = F,  
                                   min.cells.feature = 1, return.thresh = 1)

In [ ]:
fc.seurat_object

In [ ]:
# CD4L1
fc.df  <- read_csv("../tables/fold_change/cd4_l1_full_filt.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)

In [ ]:
GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP	5.65E-05	0.001949647	0.557332239	-0.578150085	-1.721117732	198	cd4_l1_full_filt	c("GZMA", "GZMB", "DUSP2", "H2AX", "PLAC8", "KLRG1", "IFITM3", "CKS2", "SERPINB9", "ADAM8", "MYADM", "DBI", "EGR1", "MRPL18", "BHLHE40", "TMEM14C", "PDCD1", "GZMK", "TMEM50B", "MT2A", "LITAF", "TUBA1B", "IFNG", "NUP62", "NR4A1", "CASP3", "MIS18BP1", "LMNB1", "ANXA4", "PSMA5", "DHFR", "S100A4", "IRF4", "GMNN", "LAG3", "CAPG", "ISG20", "TYROBP", "LXN", "UBE2T", "CCR2", "STMN1", "DHRS1", "CISD1", "TAGLN2", "YBX3", "TSPAN32", "SNX3", "FGL2", "KPNA2", "NFYB")


In [ ]:
plotEnrichment(fgsea_sets[["GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP"]],
               ranks) + labs(title="GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP") 

In [ ]:
fc.df  <- read_csv("../tables/fold_change/cd4_l1_full_filt.csv")  %>% as.data.frame()
fc.df  %>% dplyr::filter(gene %in% c("GZMA", "GZMB", "DUSP2","H2AX","PLAC8","KLRG1"))

In [ ]:
plotEnrichment(fgsea_sets[["GSE9006_HEALTHY_VS_TYPE_1_DIABETES_PBMC_AT_DX_DN"]],
               ranks) + labs(title="GSE9006_HEALTHY_VS_TYPE_1_DIABETES_PBMC_AT_DX_DN") + 

plotEnrichment(fgsea_sets[["GSE9006_HEALTHY_VS_TYPE_1_DIABETES_PBMC_4MONTH_POST_DX_DN"]],
               ranks) + labs(title="GSE9006_HEALTHY_VS_TYPE_1_DIABETES_PBMC_4MONTH_POST_DX_DN") +

plotEnrichment(fgsea_sets[["GSE10325_CD4_TCELL_VS_LUPUS_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE10325_CD4_TCELL_VS_LUPUS_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE10325_CD4_TCELL_VS_LUPUS_CD4_TCELL_UP"]],
               ranks) + labs(title="GSE10325_CD4_TCELL_VS_LUPUS_CD4_TCELL_UP") 

In [ ]:
GSE10325_CD4_TCELL_VS_LUPUS_CD4_TCELL_DN


Healthy vs Lupus
UP in Lupus

In [ ]:
fc.df  %>% dplyr::filter(gene %in% c("XAF1", "UBB", "IRF7", "IFI44L", "PSME2", "OAS1", "ARHGEF3", "MX1", "OAS2", "UBE2L6", "GBP1", "PSMB9", "RALA", "MT2A", "IFNG", "MRPL42", "IFI27", "IFIT1", "ISG15", "RAB11FIP1", "HERC5", "TAPBP", "IFI16", "C1orf21", "CLUAP1", "RTP4", "TAP1", "SNAPC3", "TRIM21", "OASL", "SP110", "H2BC12", "MX2", "PTPN22", "ME2", "SP100", "YARS1", "IRF4", "CAPZA1", "RBCK1", "CASP7", "LAG3", "ISG20", "PCYT1A", "PTPN2", "TNIP2", "STAT1", "IFI44", "TULP4", "ATP5MF", "PLSCR1", "KLF10", "YBX3", "USP46", "SAMD9", "TMSB10", 
"CDC5L", "UBE2Q1", "MYD88", "LIPC", "APOL1", "TRAFD1", "GSE1", "CD200", "PSME1", "ATP6V1A", "CEACAM1", "TLR5", "PIK3R3"))

In [ ]:
GSE9006_HEALTHY_VS_TYPE_1_DIABETES_PBMC_AT_DX_DN
Healthy vs Dia
Up in Dia
c("IER2", "FOSB", "ZFP36", "CISH", "SOCS3", "MYC", "RGCC", "MCL1", 
  "PNP", "IVNS1ABP", "CCR7", "EGR1", "CSNK1D", "PIM1", "OSTF1", "BTG2", 
  "AKTIP", "INPP5D", "NFYA", "GIGYF2", "CWC25", "SLC25A20", "S1PR1", "CKAP2", 
  "THRAP3", "FRAT2", "PPP1R10", "NRBF2", "CHM", "ELL2", "IRF4", "ST8SIA4", 
  "SELL", "MCRS1", "CORO1A", "ADAM19", "BTN2A1")


In [ ]:
fc.df  %>% dplyr::filter(gene %in% c("IER2", "FOSB", "ZFP36", "CISH", "SOCS3", "MYC", "RGCC", "MCL1", 
  "PNP", "IVNS1ABP", "CCR7", "EGR1", "CSNK1D", "PIM1", "OSTF1", "BTG2", 
  "AKTIP", "INPP5D", "NFYA", "GIGYF2", "CWC25", "SLC25A20", "S1PR1", "CKAP2", 
  "THRAP3", "FRAT2", "PPP1R10", "NRBF2", "CHM", "ELL2", "IRF4", "ST8SIA4", 
  "SELL", "MCRS1", "CORO1A", "ADAM19", "BTN2A1"))

## Treg signatures

In [ ]:
plotEnrichment(fgsea_sets[["GSE14415_INDUCED_TREG_VS_FAILED_INDUCED_TREG_DN"]],
               ranks) + labs(title="GSE14415_INDUCED_TREG_VS_FAILED_INDUCED_TREG_DN") + 

plotEnrichment(fgsea_sets[["GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_DN"]],
               ranks) + labs(title="GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_DN") +

plotEnrichment(fgsea_sets[["GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_IL2_CULTURE_UP"]],
               ranks) + labs(title="GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_IL2_CULTURE_UP") +

plotEnrichment(fgsea_sets[["GSE14415_NATURAL_TREG_VS_FOXP3_KO_NATURAL_TREG_UP"]],
               ranks) + labs(title="GSE14415_NATURAL_TREG_VS_FOXP3_KO_NATURAL_TREG_UP") 

In [ ]:


plotEnrichment(fgsea_sets[["GSE14415_NATURAL_TREG_VS_TCONV_UP"]],
               ranks) + labs(title="GSE14415_NATURAL_TREG_VS_TCONV_UP") + 

plotEnrichment(fgsea_sets[["GSE14415_ACT_TCONV_VS_ACT_NATURAL_TREG_DN"]],
               ranks) + labs(title="GSE14415_ACT_TCONV_VS_ACT_NATURAL_TREG_DN") +

plotEnrichment(fgsea_sets[["GSE18893_TCONV_VS_TREG_2H_CULTURE_DN"]],
               ranks) + labs(title="GSE18893_TCONV_VS_TREG_2H_CULTURE_DN") +

plotEnrichment(fgsea_sets[["GSE27434_WT_VS_DNMT1_KO_TREG_DN"]],
               ranks) + labs(title="GSE27434_WT_VS_DNMT1_KO_TREG_DN") 

In [ ]:

plotEnrichment(fgsea_sets[["GSE37532_WT_VS_PPARG_KO_LN_TREG_DN"]],
               ranks) + labs(title="GSE37532_WT_VS_PPARG_KO_LN_TREG_DN") + 

plotEnrichment(fgsea_sets[["GSE37533_PPARG1_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE37533_PPARG1_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE37533_PPARG2_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE37533_PPARG2_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN"]],
               ranks) + labs(title="GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN") 

In [ ]:
plotEnrichment(fgsea_sets[["GSE40685_NAIVE_CD4_TCELL_VS_FOXP3_KO_TREG_PRECURSOR_UP"]],
               ranks) + labs(title="GSE40685_NAIVE_CD4_TCELL_VS_FOXP3_KO_TREG_PRECURSOR_UP") + 

plotEnrichment(fgsea_sets[["GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_DN"]],
               ranks) + labs(title="GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_DN") +

plotEnrichment(fgsea_sets[["GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_UP"]],
               ranks) + labs(title="GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_UP") +

plotEnrichment(fgsea_sets[["GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN"]],
               ranks) + labs(title="GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN") 

In [ ]:
gseas

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 15)
gseas  %>% arrange(ES)  %>% 
dplyr::filter(grepl(pathway, pattern = "TREG") & dataset == "cd4_l1_full_filt")  %>% 
ggplot(aes(x = pathway, y = ES)) + 
geom_point(aes(size = size, color = grepl(pathway, pattern = "UP"), alpha = -padj)) + coord_flip()

### IPEX Treg our Treg

In [ ]:
fc.df  <- read_csv("../tables/fold_change/cd4_l3_treg.csv")  %>% as.data.frame()


In [ ]:
fc.df

In [ ]:
zemmour  <- read_xlsx("../data/published_data/Zemmour_2018/zemmour2.xlsx")

In [ ]:
table(zemmour$gene %in% fc.df$gene)

zemmour is ipex vs healthy

In [ ]:
fc.df  %>% dplyr::filter(cluster == "Dia")  %>% left_join(zemmour) 

In [ ]:
ipex_in_our  <- fc.df  %>% dplyr::filter(cluster == "Dia")  %>% left_join(zemmour)  %>% 
dplyr::filter(!is.na(Average_FC_in_Treg))

In [ ]:
ipex_in_our

In [ ]:
ipex_in_our %>%  ggplot(aes(x=avg_log2FC, y=Average_FC_in_Treg)) +
  geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + NoLegend() +
geom_hline(yintercept = 1)

In [ ]:
treg_ipex_up  <- ipex_in_our  %>% dplyr::filter(Average_FC_in_Treg>1)  %>% pull(gene)

In [ ]:
treg_ipex_down  <- ipex_in_our  %>% dplyr::filter(Average_FC_in_Treg<1)  %>% pull(gene)

In [ ]:
# CD4L1
fc.df  <- read_csv("../tables/fold_change/cd4_l3_treg.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)

plotEnrichment(treg_ipex_up,
               ranks) + labs(title="treg_ipex_up") 

plotEnrichment(treg_ipex_down,
               ranks) + labs(title="treg_ipex_down") 


In [ ]:
fgseaRes <- fgsea(pathways = list(treg_ipex_up = treg_ipex_up, treg_ipex_down = treg_ipex_down), 
                 stats = ranks)

In [ ]:
fgseaRes

In [ ]:
ipex_in_our

In [ ]:
ipex_in_our_rank  <- ipex_in_our  %>% dplyr::select(avg_log2FC, Average_FC_in_Treg)  %>% 
mutate_each(funs(dense_rank(-.))) 

In [ ]:
ipex_in_our_rank$gene  <- ipex_in_our$gene

In [ ]:
ipex_in_our_rank

In [ ]:
ipex_in_our_rank %>% mutate(decile_ipex = ntile(Average_FC_in_Treg, 10),
                            dec_our = ntile(avg_log2FC, 10)
                            )  %>% group_by(decile_ipex, dec_our)  %>% tally

In [ ]:

options(repr.plot.width = 6, repr.plot.height = 5)
ipex_in_our_rank %>% mutate(decile_ipex = ntile(Average_FC_in_Treg, 8),
                            dec_our = ntile(avg_log2FC, 8)
                            )  %>% group_by(decile_ipex, dec_our)  %>% tally  %>% 
ggplot(aes(decile_ipex, dec_our)) + 
  geom_point(aes(size = n, colour = n)) + 
  theme_bw() + scale_size_continuous(range=c(7,12)) +
  geom_text(aes(label = n)) + 
  scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon", midpoint = 2) +
  theme_bw()


In [ ]:
ipex_in_our_rank %>% mutate(decile_ipex = ntile(Average_FC_in_Treg, 10),
                            decile_our = ntile(avg_log2FC, 10)
                            )  %>% dplyr::filter(decile_ipex == 10 & decile_our == 10)

In [ ]:
genes_first_decile  <- ipex_in_our_rank %>% mutate(decile_ipex = ntile(Average_FC_in_Treg, 10),
                            decile_our = ntile(avg_log2FC, 10)
                            )  %>% dplyr::filter(decile_ipex == 1 & decile_our == 1)  %>% pull(gene)

In [ ]:
ipex_in_our  %>% dplyr::filter(gene %in% genes_first_decile)

In [ ]:
genes_last_decile  <- ipex_in_our_rank %>% mutate(decile_ipex = ntile(Average_FC_in_Treg, 10),
                            decile_our = ntile(avg_log2FC, 10)
                            )  %>% dplyr::filter(decile_ipex == 10 & decile_our == 10)  %>% pull(gene)

In [ ]:
genes_last_zemmour  <- ipex_in_our_rank %>% mutate(
                            decile_ipex = ntile(Average_FC_in_Treg, 10),
                            )  %>% dplyr::filter(decile_ipex == 10)  %>% pull(gene)

In [ ]:
genes_first_zemmour  <- ipex_in_our_rank %>% mutate(
                            decile_ipex = ntile(Average_FC_in_Treg, 10),
                            )  %>% dplyr::filter(decile_ipex == 1)  %>% pull(gene)

In [ ]:
genes_first_zemmour

In [ ]:
genes_last_zemmour

In [ ]:
write.csv(genes_first_zemmour, "../data/published_data/Zemmour_2018/genes_first_zemmour.csv")
write.csv(genes_last_zemmour, "../data/published_data/Zemmour_2018/genes_last_zemmour.csv")

## IPEX CD4 in CD4

In [ ]:
# CD4L1
fc.df  <- read_csv("../tables/fold_change/cd4_l1_full_filt.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)

In [ ]:
zemmour  <- read_xlsx("../data/zemmour2.xlsx")

In [ ]:
table(zemmour$gene %in% fc.df$gene)

zemmour is ipex vs healthy

In [ ]:
ipex_in_our  <- fc.df  %>% left_join(zemmour)  %>% 
dplyr::filter(!is.na(Average_FC_in_Tconv))

In [ ]:
ipex_in_our

In [ ]:
ipex_in_our %>%  ggplot(aes(x=avg_log2FC, y=Average_FC_in_Tconv)) +
  geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + NoLegend() +
geom_hline(yintercept = 1)

In [ ]:
cd4_ipex_up  <- ipex_in_our  %>% dplyr::filter(Average_FC_in_Tconv>1)  %>% pull(gene)

In [ ]:
cd4_ipex_down  <- ipex_in_our  %>% dplyr::filter(Average_FC_in_Tconv<1)  %>% pull(gene)

In [ ]:

plotEnrichment(treg_ipex_up,
               ranks) + labs(title="cd4_ipex_up") 

ggsave(filename = "../figures/gsea/pathway_plot_ipex_in_treg_up.png", width = 15, height = 9, units = "cm")
ggsave(filename = "../figures/gsea/pathway_plot_ipex_in_treg_up.svg", width = 15, height = 9, units = "cm")


plotEnrichment(treg_ipex_down,
               ranks) + labs(title="cd4_ipex_down") 

ggsave(filename = "../figures/gsea/pathway_plot_ipex_in_treg_dn.png", width = 15, height = 9, units = "cm")
ggsave(filename = "../figures/gsea/pathway_plot_ipex_in_treg_dn.svg", width = 15, height = 9, units = "cm")


In [ ]:
fgseaRes <- fgsea(pathways = list(cd4_ipex_up = cd4_ipex_up, cd4_ipex_down = cd4_ipex_down), 
                 stats = ranks)

In [ ]:
fgseaRes$leadingEdge

### Correlation plot

In [ ]:
ipex_in_our

In [ ]:
ipex_in_our_rank  <- ipex_in_our  %>% dplyr::select(-gene)  %>% 
mutate_each(funs(dense_rank(-.))) 

In [ ]:
ipex_in_our_rank$gene  <- ipex_in_our$gene

In [ ]:
ipex_in_our_rank

In [ ]:
ipex_in_our_rank %>%  ggplot(aes(x=avg_log2FC, y=Average_FC_in_Tconv)) +
  geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + NoLegend() +
geom_hline(yintercept = 1)

In [ ]:
ipex_in_our_rank %>% mutate(eightile_ipex = ntile(Average_FC_in_Tconv, 10),
                            eight_our = ntile(avg_log2FC, 10)
                            )  %>% group_by(eightile_ipex, eight_our)  %>% tally

In [ ]:

options(repr.plot.width = 6, repr.plot.height = 5)
ipex_in_our_rank %>% mutate(eightile_ipex = ntile(Average_FC_in_Tconv, 8),
                            eight_our = ntile(avg_log2FC, 8)
                            )  %>% group_by(eightile_ipex, eight_our)  %>% tally  %>% 
ggplot(aes(eightile_ipex, eight_our)) + 
  geom_point(aes(size = n, colour = n)) + 
  theme_bw() + scale_size_continuous(range=c(7,12)) +
  geom_text(aes(label = n)) + 
  scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon", midpoint = 2) +
  theme_bw()


In [ ]:
ipex_in_our_rank %>% mutate(eightile_ipex = ntile(Average_FC_in_Tconv, 8),
                            eight_our = ntile(avg_log2FC, 8)
                            )  %>% dplyr::filter(eightile_ipex == 8 & eight_our == 8)

In [ ]:
genes_first_decile  <- ipex_in_our_rank %>% mutate(decile_ipex = ntile(Average_FC_in_Tconv, 8),
                            decile_our = ntile(avg_log2FC, 8)
                            )  %>% dplyr::filter(decile_ipex == 1 & decile_our == 1)  %>% pull(gene)

In [ ]:
ipex_in_our  %>% dplyr::filter(gene %in% genes_first_decile)

In [ ]:
genes_last_decile  <- ipex_in_our_rank %>% mutate(decile_ipex = ntile(Average_FC_in_Tconv, 10),
                            decile_our = ntile(avg_log2FC, 10)
                            )  %>% dplyr::filter(decile_ipex == 10 & decile_our == 10)  %>% pull(gene)

In [ ]:
ipex_in_our  %>% dplyr::filter(gene %in% genes_last_decile)  %>% dplyr::select(gene) 

### Treg in Treg

In [ ]:
library(fgsea)

In [ ]:
# CD4L1
fc.df  <- read_csv("../tables/fold_change/cd4_l3_treg.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)

In [ ]:
gseas  <- read_csv("../tables/gsea/gsea_fold_changes_dia_vsctrl.csv")

In [ ]:
gseas  %>% 
dplyr::filter(dataset == "cd4_l3_treg" & grepl(pathway, pattern = "TREG"))  %>% 
dplyr::select(-leadingEdge2)  %>% 
arrange(padj)

In [ ]:
gseas  %>% 
dplyr::filter(dataset == "cd4_l3_treg" & grepl(pathway, pattern = "GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR")) 

In [ ]:
gseas  %>% 
dplyr::filter(dataset == "cd4_l3_treg" & grepl(pathway, pattern = "GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_UP"))  %>% pull(leadingEdge2) 

In [ ]:
gseas  %>% 
dplyr::filter(dataset == "cd4_l3_treg" & grepl(pathway, pattern = "GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_DN"))  %>% pull(leadingEdge2) 

In [ ]:
intersect(c("HLA-DQB1", "IFI44L", "SOCS3", "EPSTI1", "TNFSF10", "HLA-DRA", "MYO1G", "RNF114", "OAS2", "IL2RA", "GLRX", "UBB", "GBP5", "CCDC69", "IFITM2", "TRANK1", "IFIT1", "C5orf15", "IFIH1", "PLAAT4", "GIMAP4", "JAK3", "TNF", "STAT2", "TBC1D2B", "DPYD", "GBP4", "TRIM69", "ORAI2", "CISH", "IGFLR1", "HAPLN3", "SP110", "SNX10", "IL12RB1", "WDR48", "CD38", "MTO1", "HELB", "NUP58", "DAPP1", "PARP14", "PSME1", "APOL3", "GCH1", "C1RL", "LGMN", "BATF", "IFIT2", "HIVEP2", "RNF20", "CRK", "APOL6", "PECAM1", "ZNFX1", "AKAP7", "SNRPC", "CNIH4", "SETBP1", "RSAD2", "PDE7A", "CD74"),
         c("BTN3A2", "MX1", "XAF1", "IFI44L", "EPSTI1", "LY6E", "TNFSF10", "RNF114", "OAS2", "SAMD9L", "RCSD1", "GBP5", "TRANK1", "PSME2", "GIMAP6", "TAP1", "PLAAT4", "HLA-A", "GIMAP4", "PCYT1A", "STAT2", "UBE2L6", "GBP4", "SEC14L1", "HAPLN3", "KDM6A", "VAMP5", "SP110", "SNX10", "ACSS1", "OASL", "APOL1", "PARP14", "PSME1", "ASPHD2", "IL17RA", "GCH1", "TRIM21", "RTP4", "WDFY1", "SCO2", "GRIPAP1", "IFIT2", "ADAR", "SH3GLB1", "APOL6", "SMCO4", "ZNFX1", "PIK3R3", "IRF9", "IFIT3", "PARP9", "PPP2R2A", "RSAD2", "CASP7", "THEMIS2"))

In [ ]:
gseas  %>% dplyr::filter()

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)


In [ ]:
plotEnrichment(fgsea_sets[["GSE14415_INDUCED_TREG_VS_FAILED_INDUCED_TREG_DN"]],
               ranks) + labs(title="GSE14415_INDUCED_TREG_VS_FAILED_INDUCED_TREG_DN") + 

plotEnrichment(fgsea_sets[["GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_DN"]],
               ranks) + labs(title="GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_DN") +

plotEnrichment(fgsea_sets[["GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_IL2_CULTURE_UP"]],
               ranks) + labs(title="GSE14415_INDUCED_TREG_VS_FOXP3_KO_INDUCED_TREG_IL2_CULTURE_UP") +

plotEnrichment(fgsea_sets[["GSE14415_NATURAL_TREG_VS_FOXP3_KO_NATURAL_TREG_UP"]],
               ranks) + labs(title="GSE14415_NATURAL_TREG_VS_FOXP3_KO_NATURAL_TREG_UP") 

In [ ]:
plotEnrichment(fgsea_sets[["GSE14415_NATURAL_TREG_VS_TCONV_UP"]],
               ranks) + labs(title="GSE14415_NATURAL_TREG_VS_TCONV_UP") + 

plotEnrichment(fgsea_sets[["GSE14415_ACT_TCONV_VS_ACT_NATURAL_TREG_DN"]],
               ranks) + labs(title="GSE14415_ACT_TCONV_VS_ACT_NATURAL_TREG_DN") +

plotEnrichment(fgsea_sets[["GSE18893_TCONV_VS_TREG_2H_CULTURE_DN"]],
               ranks) + labs(title="GSE18893_TCONV_VS_TREG_2H_CULTURE_DN") +

plotEnrichment(fgsea_sets[["GSE27434_WT_VS_DNMT1_KO_TREG_DN"]],
               ranks) + labs(title="GSE27434_WT_VS_DNMT1_KO_TREG_DN") 

In [ ]:
plotEnrichment(fgsea_sets[["GSE37532_WT_VS_PPARG_KO_LN_TREG_DN"]],
               ranks) + labs(title="GSE37532_WT_VS_PPARG_KO_LN_TREG_DN") + 

plotEnrichment(fgsea_sets[["GSE37533_PPARG1_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE37533_PPARG1_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE37533_PPARG2_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN"]],
               ranks) + labs(title="GSE37533_PPARG2_FOXP3_VS_FOXP3_TRANSDUCED_CD4_TCELL_DN") +

plotEnrichment(fgsea_sets[["GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN"]],
               ranks) + labs(title="GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN") 

In [ ]:
plotEnrichment(fgsea_sets[["GSE40685_NAIVE_CD4_TCELL_VS_FOXP3_KO_TREG_PRECURSOR_UP"]],
               ranks) + labs(title="GSE40685_NAIVE_CD4_TCELL_VS_FOXP3_KO_TREG_PRECURSOR_UP") + 

plotEnrichment(fgsea_sets[["GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_DN"]],
               ranks) + labs(title="GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_DN") +

plotEnrichment(fgsea_sets[["GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_UP"]],
               ranks) + labs(title="GSE40685_TREG_VS_FOXP3_KO_TREG_PRECURSOR_UP") +

plotEnrichment(fgsea_sets[["GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN"]],
               ranks) + labs(title="GSE37605_C57BL6_VS_NOD_FOXP3_FUSION_GFP_TREG_DN") 

### GSEA with GeneKitR

In [ ]:
library("genekitr")

In [ ]:
gs <- geneset::getMsigdb(org = "human",category = 'C7-IMMUNESIGDB')

In [ ]:
ranks4  <- unlist(ranks, use.names=TRUE)

In [ ]:
gse <- genGSEA(genelist = ranks4, geneset = gs)

In [ ]:
options(repr.plot.height = 20)
pathways <- c("GSE9006_HEALTHY_VS_TYPE_1_DIABETES_PBMC_AT_DX_DN", 
              #"GSE9006_HEALTHY_VS_TYPE_1_DIABETES_PBMC_4MONTH_POST_DX_DN",
              "GSE10325_CD4_TCELL_VS_LUPUS_CD4_TCELL_DN",
              "GSE10325_CD4_TCELL_VS_LUPUS_CD4_TCELL_UP")
genes <- c("CXCR4", "NFKBIA", "RBM3", "TSC22D3")
plotGSEA(gse, plot_type = "classic",  
         show_pathway = pathways,
            ) + ggtheme()

## NfKB

In [ ]:
paths_pathways  <- list.files("../data/nfkb/", full.names = T)
pathway_names  <- list.files("../data/nfkb/", full.names = F)


In [ ]:
nfkb_genes1  <- str_to_upper(read_csv("../data/nfkb/Paper_I1_cluster.csv")$Genes)
nfkb_genes2  <- str_to_upper(read_csv("../data/nfkb/Paper_I2_cluster.csv")$Genes)
nfkb_genes3  <- str_to_upper(read_csv("../data/nfkb/Paper_I3_cluster.csv")$Genes)
nfkb_genes6  <- str_to_upper(read_csv("../data/nfkb/Paper_I6_cluster.csv")$Genes)

paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

In [ ]:
gsea_nfkb  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
fgseaRes <- fgsea(pathways = list(nfkb_genes1 = nfkb_genes1, nfkb_genes2 = nfkb_genes2, 
                                 nfkb_genes3 = nfkb_genes3, nfkb_genes6 = nfkb_genes6), 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")   
 return(fgseaRes)
    }


In [ ]:
paths_fc

In [ ]:
gsea  <- map(.x = c(1:length(paths_fc)), .f = gsea_nfkb)

In [ ]:
gseas  <- bind_rows(gsea)

In [ ]:
gseas  %>% arrange(padj)

In [ ]:
nfkb_genes  <- read_csv("../data/nfkb/Paper_I3_cluster.csv")
nfkb_genes  <- str_to_upper(nfkb_genes$Genes)

fc.df  <- read_csv("../tables/fold_change/cd8_l3_temra.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)


fgseaRes.nfkb1 <- fgsea(pathways = list(our_list = nfkb_genes), 
                  stats    = ranks)


p3 <- plotEnrichment(nfkb_genes, ranks) + 
labs(title="\nNFkB List3, cd8_l3_temra", 
     subtitle = paste0("p-value = ",format.pval(fgseaRes.nfkb1$pval, digits = 2))) 

p3

In [ ]:

fc.df  <- read_csv("../tables/fold_change/cd4_l3_temra.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)


fgseaRes.nfkb1 <- fgsea(pathways = list(our_list = nfkb_genes), 
                  stats    = ranks)


p3 <- plotEnrichment(nfkb_genes, ranks) + 
labs(title="\nNFkB List3, cd4_l3_temra", 
     subtitle = paste0("p-value = ",format.pval(fgseaRes.nfkb1$pval, digits = 2))) 

p3

In [ ]:

fc.df  <- read_csv("../tables/fold_change/cd4_l3_treg.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)


fgseaRes.nfkb1 <- fgsea(pathways = list(our_list = nfkb_genes6), 
                  stats    = ranks)


p3 <- plotEnrichment(nfkb_genes, ranks) + 
labs(title="\nNFkB List6, cd4_l3_treg", 
     subtitle = paste0("p-value = ",format.pval(fgseaRes.nfkb1$pval, digits = 2))) 

p3

## Akt/Erk

In [ ]:
paths  <- list.files("../data/pathways/", full.names = T)

In [ ]:
paths

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

In [ ]:
read_pathway  <- function(path){
    pathway  <- fread(path, header = F)
    df  <- data.frame(name = pathway$V2[1],
                     genes = str_to_upper(pathway$V2[which(pathway$V1 == "GENE_SYMBOLS")]))
    genes_vector  <- str_split(str_to_upper(pathway$V2[which(pathway$V1 == "GENE_SYMBOLS")]),simplify = FALSE,
                               pattern = ",",) %>% unique 
    genes_vector  <- genes_vector  %>% unlist()
    return(genes_vector)
}

In [ ]:
read_pathway_name  <- function(path){
    pathway  <- fread(path, header = F)
    df  <- data.frame(name = pathway$V2[1],
                     genes = str_to_upper(pathway$V2[which(pathway$V1 == "GENE_SYMBOLS")]))
    genes_vector  <- str_split(str_to_upper(pathway$V2[which(pathway$V1 == "GENE_SYMBOLS")]),simplify = FALSE,
                               pattern = ",",) %>% unique 
    genes_vector  <- genes_vector  %>% unlist()
    name = pathway$V2[1]
    return(name)
}

In [ ]:
pathways  <- map(paths, .f = read_pathway)

In [ ]:
names_pathways  <- map(paths, .f = read_pathway_name)

In [ ]:
names_pathways  %>% unlist()

In [ ]:
names(pathways)  <- unlist(names_pathways)

In [ ]:
pathways

In [ ]:
gsea_akt_erkb  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia T0")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)
fgseaRes <- fgsea(pathways = pathways, 
                 stats = ranks)
 fgseaRes$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")   
 return(fgseaRes)
    }

In [ ]:
paths_fc  <- paths_fc[which(grepl(paths_fc, pattern = "Ctrl_vs_DiaT0"))]

In [ ]:
dataset_names = paths_fc

In [ ]:
fc.df  <- read_csv(paths_fc[1])  %>% as.data.frame()


In [ ]:
fc.df

In [ ]:
gsea  <- map(.x = c(1:length(paths_fc)), .f = gsea_akt_erkb)

In [ ]:
gseas  <- bind_rows(gsea)

gseas  %>% arrange(padj)

In [ ]:

fc.df  <- read_csv("../tables/fold_change/cd4_l1_full_filt.csv")  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) %>% 
dplyr::filter(cluster == "Dia")  %>% 
  dplyr::select(gene, avg_log2FC)
ranks<- deframe(fc.df)


fgseaRes <- fgsea(pathways = list(pathways = pathways$WP_FOCAL_ADHESION_PI3K_AKT_MTOR_SIGNALING_PATHWAY), 
                  stats    = ranks)


p3 <- plotEnrichment(pathways$WP_FOCAL_ADHESION_PI3K_AKT_MTOR_SIGNALING_PATHWAY, ranks) + 
labs(title="\nWP_FOCAL_ADHESION_PI3K_AKT_MTOR_SIGNALING_PATHWAY, \ncd4_l1_full_filt.csv", 
     subtitle = paste0("p-value = ",format.pval(fgseaRes$pval, digits = 2))) 

p3

# PI3K

In [ ]:
plotEnrichment(treg_ipex_up,
               ranks) + labs(title="cd4_ipex_up") 

In [ ]:
plotEnrichment(treg_ipex_up,
               ranks) + labs(title="cd4_ipex_up") 

## Spider plot

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)

In [ ]:
load_fold_changes  <- function(j) {
  print(j)
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC)) 
 fc.df$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
 return(fc.df)
    }

In [ ]:
fold_changes  <- map(1:210, .f = load_fold_changes)

In [ ]:
fold_changes2  <- bind_rows(fold_changes)

In [ ]:
fold_changes2

In [ ]:
fc_selected  <- fold_changes2  %>% dplyr::filter(gene %in% c("IER2","GADD45B","BTN3A2", "GZMB","XAF1",
                                            "RBM3","CD69","CXCR4","DDIT4","ACTG1"))  %>% 
dplyr::filter(grepl(dataset, pattern = "_Ctrl_vs_DiaT0")|
              grepl(dataset, pattern = "_Ctrl_vs_DiaT1"))  %>% 
dplyr::filter((grepl(dataset, pattern = "_Ctrl_vs_DiaT1") & cluster == "Dia T1") |
       (grepl(dataset, pattern = "_Ctrl_vs_DiaT0"))) 

In [ ]:
fc_selected$cluster  %>% table

In [ ]:
fc_for_spider  <-  fc_selected  %>% dplyr::select(dataset, cluster, gene, avg_log2FC)  %>% 
mutate(dataset = paste(cluster, gsub(dataset, pattern = "_Ctrl_vs_DiaT0", replacement = ""), sep = "---"))  %>% 
mutate(dataset = gsub(dataset, pattern = "_Ctrl_vs_DiaT1", replacement = ""))  %>% 
       dplyr::select(-cluster)  %>% pivot_wider(names_from = gene, values_from = avg_log2FC, values_fill = 0)  %>% 
column_to_rownames("dataset")

In [ ]:
fc_for_spider 

In [ ]:
colors_in = ifelse(grepl(rownames(fc_for_spider), pattern = "Ctrl T0"), "dodgerblue2", 
                   ifelse(grepl(rownames(fc_for_spider), pattern = "Dia T0"),"indianred3","goldenrod"))

In [ ]:
colors_in

In [ ]:
mins = (fc_for_spider  %>% mutate_all(.funs = min))[1,]

In [ ]:
maxs = (fc_for_spider  %>% mutate_all(.funs = max))[1,]

In [ ]:
fc_for_spider2  <- rbind(mins, maxs, fc_for_spider)

In [ ]:
rownames(fc_for_spider2)[c(1,2)]  <- c("min","max")

In [ ]:
fc_for_spider2

In [ ]:
radarchart(fc_for_spider2, pcol = colors_in)
legend(x=2, y=1, legend = rownames(fc_for_spider), 
       col = colors_in,
       bty = "n", pch=20 , text.col = "black", cex=1.2, pt.cex=3)

In [ ]:
radarchart(fc_for_spider2, pcol = colors_in)
legend(x=2, y=1, legend = rownames(fc_for_spider), 
       col = colors_in,
       bty = "n", pch=20 , text.col = "black", cex=1.2, pt.cex=3)

In [ ]:
fc_for_spider3  <- fc_for_spider2[3:nrow(fc_for_spider3),]

In [ ]:
fc_for_spider3$group  <- ifelse(grepl(rownames(fc_for_spider), pattern = "Ctrl T0"), "Ctrl T0", 
                   ifelse(grepl(rownames(fc_for_spider), pattern = "Dia T0"),"Dia T0","Tia T1"))

In [ ]:
fc_for_spider3  %>% ncol

In [ ]:
fc_for_spider3

In [ ]:
library(GGally)
data <- fc_for_spider3

data %>% 
  ggparcoord(
    columns = 1:10, groupColumn = 11, 
    showPoints = TRUE, 
    title = "Parallel Coordinate Plot for the Iris Data",
    alphaLines = 0.3
    ) + 
  scale_color_manual(values = c("dodgerblue2","red3","goldenrod")) 

In [ ]:
gene	gene_direction	direction	test_type	n
IER2 	IER2 Down 	Down 	Dia T0 vs Ctrl T0	19
GADD45B 	GADD45B Down 	Down 	Dia T0 vs Ctrl T0	18
BTN3A2 	BTN3A2 Down 	Down 	Dia T0 vs Ctrl T0	13
CENPK 	CENPK Down 	Down 	Dia T0 vs Ctrl T0	13
XAF1 	XAF1 Down 	Down 	Dia T0 vs Ctrl T0	13
RBM3 	RBM3 Up 	Up 	Dia T0 vs Ctrl T0	10
SOCS3 	SOCS3 Down 	Down 	Dia T0 vs Ctrl T0	8
ACTG1 	ACTG1 Up 	Up 	Dia T0 vs Ctrl T0	7
CD69 	CD69 Up 	Up 	Dia T0 vs Ctrl T0	7
CXCR4 	CXCR4 Up 	Up 	Dia T0 vs Ctrl T0	7
ISG15 	ISG15 Down 	Down 	Dia T0 vs Ctrl T0	7
JUN 	JUN Down 	Down 	Dia T0 vs Ctrl T0	7
MX1 	MX1 Down 	Down 	Dia T0 vs Ctrl T0	7
DDIT4 	DDIT4 Up 	Up 	Dia T0 vs Ctrl T0	6
DUSP1 	DUSP1 Up 	Up 	Dia T0 vs Ctrl T0	6
EPSTI1 	EPSTI1 Down 	Down 	Dia T0 vs Ctrl T0	6
IFITM2 	IFITM2 Down 	Down 	Dia T0 vs Ctrl T0	6
CDK11A 	CDK11A Up 	Up 	Dia T0 vs Ctrl T0	5
CRIP1 	CRIP1 Down 	Down 	Dia T0 vs Ctrl T0	5
DUSP2 	DUSP2 Down 	Down 	Dia T0 vs Ctrl T0	5
FOS 	FOS Down 	Down 	Dia T0 vs Ctrl T0	5
FOSB 	FOSB Down 	Down 	Dia T0 vs Ctrl T0	5
HLA-DQB1 	HLA-DQB1 Down 	Down 	Dia T0 vs Ctrl T0	5
IFI44L 	IFI44L Down 	Down 	Dia T0 vs Ctrl T0	5
IFITM3 	IFITM3 Down 	Down 	Dia T0 vs Ctrl T0	5
PTGDR 	PTGDR Up 	Up 	Dia T0 vs Ctrl T0	5
BCL2 	BCL2 Up 	Up 	Dia T0 vs Ctrl T0	4
CCL5 	CCL5 Down 	Down 	Dia T0 vs Ctrl T0	4
GSTM1 	GSTM1 Down 	Down 	Dia T0 vs Ctrl T0	4 

# DE genes visualization

In [ ]:
full  <- readRDS("../data/processed/full.rds")

In [ ]:
full@meta.data  <- full@meta.data  %>% 
mutate(annotations_l2  = gsub(pattern = " T cells", replacement = "", x = annotations_l3))  %>% 
separate(annotations_l2, into = c("annotations_l1", "annotations_l2", "annotations_l3"), sep = "---") %>% 
mutate(annotations_l2  = gsub(pattern = "Unconventional_CD8 Unc", replacement = "CD8_Unc", x = annotations_l2))  %>% 
mutate(annotations_l2  = gsub(pattern = "Unconventional_Unc", replacement = "CD4_Unc_", x = annotations_l2)) %>% 
separate(annotations_l2, into = c("Level1", "Level2",NA,NA), sep = "_")  

In [ ]:
cd4_l1_full  <- readRDS("../data/processed/L1/cd4_l1_full_filt.rds")

In [ ]:
cd8_l1_full  <- readRDS("../data/processed/L1/cd8_l1_full_filt.rds")

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 4)
DimPlot(cd4_l1_full, group.by = "annotations_l3", raster = T)

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 20)

DimPlot(cd8_l1_full, group.by = "annotations_l3", raster = T)

In [ ]:
rm(full)

In [ ]:
markers  <- rev(c("BTN3A2", "IL32","AMICA1"))

### All cell types CD8

In [ ]:
md  <- cd8_l1_full@meta.data  %>% dplyr::select(Patient_ID, Time, 
                                                Sample_ID, Disease, 
                                                Condition, Experiment_ID, Age_group)  %>% unique  %>% 
mutate(prelim_final = ifelse(Experiment_ID %in% c("Exp08", "Exp10","Exp11"), "Prelim","Final"))

In [ ]:
md

In [ ]:
avgexp = AverageExpression(cd8_l1_full, features = markers,
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

In [ ]:
btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% mutate(Condition = paste(Disease, Time))

In [ ]:
btn_etc

In [ ]:
options(repr.plot.width = 9.5, repr.plot.height = 3.5)

btn_etc  %>% ggplot(aes(x = Condition,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression all cell types CD8 T cells")

In [ ]:
cd8_l1_full$annotations_l2  %>% table

### CD8 types by annotation

In [ ]:
cd8_l1_full$SampleID_Annotation  <- paste(cd8_l1_full$Sample_ID, gsub(cd8_l1_full$annotations_l2, pattern = " ", replacement = "_"))
avgexp = AverageExpression(cd8_l1_full, features = markers,
                           return.seurat = F, group.by = "SampleID_Annotation", 
                          assay = "RNA")

In [ ]:
btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("SampleID_Annotation")  %>% 
pivot_longer(!SampleID_Annotation, names_to = "gene", values_to = "expression")  %>% 
separate(SampleID_Annotation, into = c("Sample_ID","Annot"), sep = " ", remove = F)  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% mutate(Condition = paste(Disease, Time))

In [ ]:
btn_etc

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 5)

btn_etc  %>% ggplot(aes(x = Condition,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Condition)) + 
    facet_grid(rows = vars(gene), cols = vars(Annot), scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression in CD8 T cell subtypes")

### All cell types CD4

In [ ]:
md  <- cd4_l1_full@meta.data  %>% dplyr::select(Patient_ID, Time, 
                                                Sample_ID, Disease, 
                                                Condition, Experiment_ID, Age_group)  %>% unique  %>% 
mutate(prelim_final = ifelse(Experiment_ID %in% c("Exp08", "Exp10","Exp11"), "Prelim","Final"))

In [ ]:
md

In [ ]:
avgexp = AverageExpression(cd4_l1_full, features = markers,
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

In [ ]:
btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% mutate(Condition = paste(Disease, Time))

In [ ]:
btn_etc

In [ ]:
options(repr.plot.width = 9.5, repr.plot.height = 3.5)

btn_etc  %>% ggplot(aes(x = Condition,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression all cell types CD4 T cells")

In [ ]:
cd4_l1_full$annotations_l2  %>% table

### CD4 types by annotation

In [ ]:
cd4_l1_full$SampleID_Annotation  <- paste(cd4_l1_full$Sample_ID, gsub(cd4_l1_full$annotations_l2, pattern = " ", replacement = "_"))
avgexp = AverageExpression(cd4_l1_full, features = markers,
                           return.seurat = F, group.by = "SampleID_Annotation", 
                          assay = "RNA")

In [ ]:
btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("SampleID_Annotation")  %>% 
pivot_longer(!SampleID_Annotation, names_to = "gene", values_to = "expression")  %>% 
separate(SampleID_Annotation, into = c("Sample_ID","Annot"), sep = " ", remove = F)  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% mutate(Condition = paste(Disease, Time))

In [ ]:
btn_etc

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 5)

btn_etc  %>% ggplot(aes(x = Condition,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Condition)) + 
    facet_grid(rows = vars(gene), cols = vars(Annot), scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression in CD4 T cell subtypes")

### CD4 by HLA

In [ ]:
md  <- cd4_l1_full@meta.data  %>% dplyr::select(Patient_ID, Time, 
                                                Sample_ID, Disease, 
                                                Condition, Experiment_ID, Age_group)  %>% unique  %>% 
mutate(prelim_final = ifelse(Experiment_ID %in% c("Exp08", "Exp10","Exp11"), "Prelim","Final"))

In [ ]:
md5  <- read_xlsx("../data/metadata_v05.xlsx")

In [ ]:
md5  <- md5  %>% mutate(dq2 = ifelse((HLA_DQA11 == "DQA1*05:01:01" | HLA_DQA12 == "DQA1*05:01:01") &
                                   (HLA_DQB11 == "DQB1*02:01:01" | HLA_DQB12 == "DQB1*02:01:01"),"DQ2",
                                   "Other"),
                      dq8 = ifelse((HLA_DQA11 == "DQA1*03:01:01" | HLA_DQA12 == "DQA1*03:01:01") &
                                   (HLA_DQB11 == "DQB1*03:02:01" | HLA_DQB12 == "DQB1*03:02:01"),"DQ8",
                                   "Other"))  %>% mutate(
                      dq2_8 = ifelse(dq2 == "DQ2" & dq8 == "DQ8","DQ2_8",
                                     ifelse(dq2 == "DQ2","DQ2", ifelse(dq8 == "DQ8","DQ8","Other"))))  %>% 
mutate(c7_01 = ifelse((HLA_C1 == "C*07:01:01" & HLA_C2 == "C*07:01:01"), "c7_01_hom",
                                   ifelse(HLA_C1 == "C*07:01:01" | HLA_C2 == "C*07:01:01","c7_01_het",
                                   "Other")),
                      b8_01 = ifelse((HLA_B1 == "B*08:01:01" & HLA_B2 == "B*08:01:01"), "b8_01_hom",
                                   ifelse(HLA_B1 == "B*08:01:01" | HLA_B2 == "B*08:01:01","b8_01_het",
                                   "Other")))   %>% mutate(
                      c7_b8 = ifelse(b8_01 == "b8_01_hom" & c7_01 == "c7_01_hom",
                                     "c7_b8_hom", ifelse(c7_01 == "c7_01_hom", "c7_01_hom",
                              ifelse(b8_01 == "b8_01_hom", "b8_01_hom", 
                              ifelse(b8_01 == "b8_01_het" & c7_01 == "c7_01_het",
                                     "c7_b8_het", 
                                     
                              ifelse(b8_01 == "b8_01_het", "b8_01_het", 
                              ifelse(c7_01 == "c7_01_het", "c7_01_het", "Other" )))))))

In [ ]:
md5  %>% colnames

In [ ]:
md_HLA  <- md5  %>% dplyr::select(Patient_Time, dq2_8)

In [ ]:
avgexp = AverageExpression(cd4_l1_full, features = markers,
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))  %>% 
left_join(md_HLA)  %>% 
mutate(Condition = paste(Disease, Time))

In [ ]:
btn_etc

In [ ]:
options(repr.plot.width = 9.5, repr.plot.height = 3.5)

btn_etc  %>% ggplot(aes(x = dq2_8,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Disease, shape = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA CD4 T cells")

In [ ]:
md5  %>% dplyr::select(starts_with("HLA"))

In [ ]:
md_HLA  <- md5  %>% dplyr::select(Patient_Time, c7_b8, b8_01, c7_01)

avgexp = AverageExpression(cd4_l1_full, features = markers,
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))  %>% 
left_join(md_HLA)  %>% 
mutate(Condition = paste(Disease, Time))

btn_etc

In [ ]:
options(repr.plot.width = 9.5, repr.plot.height = 3.5)

btn_etc  %>% ggplot(aes(x = c7_b8,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Disease, shape = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA CD4 T cells")

In [ ]:
md_HLA  <- md5  %>% dplyr::select(Patient_Time, c7_b8, b8_01, c7_01, dq2_8)

avgexp = AverageExpression(cd4_l1_full, features = c("NOTCH4","TABP","TAP2","LMP7","TAP1","LMP2","LST1",
                                                     "LTB","TNF","LTA","NFKBIL1", "BTN3A2","BTN3A1"),
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))  %>% 
left_join(md_HLA)  %>% 
mutate(Condition = paste(Disease, Time))

btn_etc

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 12)

btn_etc  %>% ggplot(aes(x = dq2_8,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Disease, shape = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA CD4 T cells")

In [ ]:
btn_etc  %>% ggplot(aes(x = c7_b8,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Disease, shape = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA CD4 T cells")

### CD8 by HLA

In [ ]:
md  <- cd8_l1_full@meta.data  %>% dplyr::select(Patient_ID, Time, 
                                                Sample_ID, Disease, 
                                                Condition, Experiment_ID, Age_group)  %>% unique  %>% 
mutate(prelim_final = ifelse(Experiment_ID %in% c("Exp08", "Exp10","Exp11"), "Prelim","Final"))

In [ ]:
avgexp = AverageExpression(cd8_l1_full, features = markers,
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))  %>% 
left_join(md_HLA)  %>% 
mutate(Condition = paste(Disease, Time))

In [ ]:
btn_etc

In [ ]:
options(repr.plot.width = 9.5, repr.plot.height = 3.5)

btn_etc  %>% ggplot(aes(x = dq2_8,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Disease, shape = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA CD8 T cells")

In [ ]:
md_HLA  <- md5  %>% dplyr::select(Patient_Time, c7_b8, b8_01, c7_01)

avgexp = AverageExpression(cd4_l1_full, features = markers,
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
mutate(Sample_ID = as.numeric(Sample_ID))  %>% 
left_join(md)  %>% 
mutate(Patient_Time = paste(Patient_ID, Time))  %>% 
left_join(md_HLA)  %>% 
mutate(Condition = paste(Disease, Time))

btn_etc

In [ ]:
options(repr.plot.width = 9.5, repr.plot.height = 3.5)

btn_etc  %>% ggplot(aes(x = c7_b8,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = Disease, shape = prelim_final)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA CD4 T cells")

In [ ]:
colnames(md5)

In [ ]:
md5  %>% mutate(group = substr(patient,1,1))  %>% group_by(HLA_DQA11, HLA_DQA12, HLA_DQB11, HLA_DQB12, group, dq2_8)  %>% tally

# DE genes visualization 3 VOLCANO

In [ ]:
library(tidytext)

In [ ]:
all_markers$dataset  %>% table

In [ ]:
all_markers  %>% 
dplyr::filter(source == "scRNAseq_RNA")  %>% 
dplyr::filter(test_type == "Dia T0 vs Ctrl T0" &
             dataset == "cd4_l3_proliferating")  %>% 
mutate(abs_avg_log2fc = abs(avg_log2FC))  %>% 
group_by( direction)  %>% 
slice_max(n = 10, order_by = abs_avg_log2fc) 

In [ ]:
test  <- all_markers  %>% 
dplyr::filter(source == "scRNAseq_RNA")  %>% 
group_by(test_type, dataset)  %>% tally  %>% 
pivot_wider(names_from = dataset, values_from = n)

In [ ]:
write.csv(test, "../tables/fc_test_counts.csv")

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 68)
all_markers  %>% 
dplyr::filter(source == "scRNAseq_RNA")  %>% 
mutate(abs_avg_log2fc = abs(avg_log2FC))  %>% 
group_by(test_type, direction)  %>% 
slice_max(n = 50, order_by = abs_avg_log2fc)  %>% 
ggplot(aes(x = dataset, y = reorder_within(gene, avg_log2FC, test_type))) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = direction)) + 
facet_grid(rows = vars(test_type), cols = vars(dataset),
           scales = "free", space = "free") +
scale_x_reordered() +
      theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_text(angle = 90))

In [ ]:
list.files("../tables/fold_change/")

In [ ]:
paths_fc  <- list.files("../tables/fold_change", full.names = T)
dataset_names  <- list.files("../tables/fold_change", full.names = F)


In [ ]:
read_fc  <- function(j) {
fc.df  <- read_csv(paths_fc[j])  %>% as.data.frame()
fc.df  <- fc.df  %>%  arrange(desc(avg_log2FC))
fc.df$dataset  <-   gsub(dataset_names[j], pattern = ".csv", replacement = "")
   
    return(fc.df)
    }

In [ ]:
paths_fc

In [ ]:
all_fcs  <- map(1:length(paths_fc), .f = read_fc)

In [ ]:
all_fcs2  <- bind_rows(all_fcs)

In [ ]:
all_fcs2  %>% head

In [ ]:
all_fcs2$`...1`  <- NULL

In [ ]:
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "full_filt", replacement = "full")

In [ ]:
all_fcs2  %>% dplyr::filter(grepl(dataset, pattern = "cd4_l1_full"))  %>% 
group_by(dataset) %>% tally

In [ ]:
all_fcs3

In [ ]:
test2  <- all_fcs3  %>% 
group_by(Comparison, Dataset)  %>% tally  %>% 
pivot_wider(names_from = Dataset, values_from = n)

write.csv(test2, "../tables/fc_test_count2.csv")

In [ ]:
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_Ctrl_vs_DiaT0", replacement = "--DiaT0 vs. Ctrl")

In [ ]:
all_fcs2  %>% dplyr::filter(grepl(dataset, pattern = "cd4_l1_full"))  %>% 
group_by(dataset) %>% tally

In [ ]:
gsub(grep(all_fcs2$dataset,pattern = "0_T1_vs_PR_1_T1", value = T)  %>% head, 
     pattern = "_PR_0_T1_vs_PR_1_T1", replacement = "--PR0_T1 vs. PR1_T1")

In [ ]:
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_Ctrl_vs_DiaT1", replacement = "--DiaT1 vs. Ctrl")
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_ketoacidosis", replacement = "--Keto+ vs. Keto-")
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_PR_0_T0_vs_PR_0_T1", replacement = "--PR0_T0 vs. PR0_T1")
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_PR_0_T0_vs_PR_1_T0", replacement = "--PR0_T0 vs. PR1_T0")
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_PR_1_T0_vs_PR_1_T1", replacement = "--PR1_T0 vs. PR1_T1")
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_Ctrl_vs_DiaT0", replacement = "--DiaT0 vs. Ctrl")
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_DiaT0_vs_DiaT1", replacement = "--DiaT0 vs. DiaT1")

In [ ]:
all_fcs2$dataset   <- gsub(all_fcs2$dataset, pattern = "_PR_0_T1_vs_PR_1_T1", replacement = "--PR0_T1 vs. PR1_T1")

In [ ]:
all_fcs2  %>% dplyr::filter(grepl(dataset, pattern = "cd4_l1_full"))  %>% 
group_by(dataset) %>% tally

In [ ]:
all_fcs2  <- all_fcs2  %>% separate(dataset, into = c("Dataset", "Comparison"), sep = "--", remove = F)

In [ ]:
all_fcs2  <- all_fcs2  %>% dplyr::filter(!is.na(Comparison))

In [ ]:
dataset_comparison  <- levels(factor(all_fcs2$dataset))

In [ ]:
all_fcs2$Comparison  %>% table

In [ ]:
all_fcs2$dataset  %>% table

In [ ]:
all_fcs2$Comparison  %>% table

In [ ]:
all_fcs2  <- all_fcs2  %>% mutate(
direction = case_when(Comparison == "DiaT1 vs. Ctrl" & cluster == "Ctrl T0" ~ "Down",
                      Comparison == "DiaT1 vs. Ctrl" & cluster == "Dia T1" ~ "Up",
                      
                      Comparison == "DiaT0 vs. Ctrl" & cluster == "Ctrl T0" ~ "Down",
                      Comparison == "DiaT0 vs. Ctrl" & cluster == "Dia T0" ~ "Up",
                      
                      Comparison == "DiaT0 vs. DiaT1" & cluster == "Dia T0" ~ "Up",
                      Comparison == "DiaT0 vs. DiaT1" & cluster == "Dia T1" ~ "Down",
                      
                      Comparison == "PR0_T0 vs. PR0_T1" & cluster == "PR_0 T0" ~ "Up",
                      Comparison == "PR0_T0 vs. PR0_T1" & cluster == "PR_0 T1" ~ "Down",
                      
                      Comparison == "PR0_T0 vs. PR1_T0" & cluster == "PR_0 T0" ~ "Up",
                      Comparison == "PR0_T0 vs. PR1_T0" & cluster == "PR_1 T0" ~ "Down",
                      
                      Comparison == "PR0_T1 vs. PR1_T1" & cluster == "PR_0 T1" ~ "Up",
                      Comparison == "PR0_T1 vs. PR1_T1" & cluster == "PR_1 T1" ~ "Down",
                      
                      Comparison == "PR1_T0 vs. PR1_T1" & cluster == "PR_1 T0" ~ "Up",
                      Comparison == "PR1_T0 vs. PR1_T1" & cluster == "PR_1 T1" ~ "Down",
                      
                      Comparison == "Keto+ vs. Keto-" & cluster == "Keto_0 T0" ~ "Down",
                      Comparison == "Keto+ vs. Keto-" & cluster == "Keto_1 T0" ~ "Up"
                      
                     ))

In [ ]:
gc()

In [ ]:
head(all_fcs2)

In [ ]:
dataset_list  <- levels(factor(all_fcs2$dataset))


In [ ]:
dataset_list

In [ ]:
i = dataset_list[1]

In [ ]:
all_fcs2  %>% dplyr::filter(dataset == i & p_val_adj < 0.01)  %>% 
    group_by(direction)  

In [ ]:
Comparison_list  <- levels(factor(all_fcs2$Comparison))

In [ ]:
i = Comparison_list[1]

In [ ]:
genes  <- all_fcs2  %>% dplyr::filter(Comparison == i & p_val_adj < 0.01)  %>% 
    group_by(direction, Dataset)  %>% slice_max(n = 10, order_by = avg_log2FC)   %>% pull(gene)

In [ ]:
genes

In [ ]:
all_fcs2$signif = ifelse(all_fcs2$p_val_adj < 0.05, "y", "n")

In [ ]:
all_fcs2  <- all_fcs2  %>% mutate(p_val_adj = if_else(p_val_adj == 0, 1e-294,p_val_adj))

In [ ]:
library(tidytext)

In [ ]:
Datasets_list  <- levels(factor(all_fcs2$Dataset))

In [ ]:
i = Datasets_list[1]

In [ ]:
all_fcs2  %>% head

In [ ]:
 genes  <- all_fcs2  %>% dplyr::filter(Dataset == i & Comparison == "DiaT0 vs. Ctrl")  %>% 
    group_by(direction)  %>% slice_max(n = 10, order_by = avg_log2FC)   %>% pull(gene)
    df  <- all_fcs2  %>% dplyr::filter(Dataset == i & direction == "Up" & gene %in% genes & 
                                       Comparison == "DiaT0 vs. Ctrl"                                      )

In [ ]:
genes

In [ ]:
df

In [ ]:
j = 0


In [ ]:
plot_list[[i]]

In [ ]:
plot_list

In [ ]:
Comaprisons_list  <- levels(factor(all_fcs2$Comparison  ))

In [ ]:
dir.create("../figures/DE_genes/pdf/")

In [ ]:
j = Comparison_list[2]
    
plot_list  <- list()    
  
for(i in Datasets_list){
    genes  <- all_fcs2  %>% dplyr::filter(Dataset == i & Comparison == j)  %>% 
    group_by(direction)  %>% slice_max(n = 10, order_by = avg_log2FC)   %>% pull(gene)
    df  <- all_fcs2  %>% dplyr::filter(Dataset == i & direction == "Up" & 
                                       gene %in% genes & Comparison == j)
    p  <- df  %>% ggplot(aes(x = Dataset, y = reorder(gene, avg_log2FC))) + 
  geom_point(aes(size = -log(p_val_adj, base = 10), colour = avg_log2FC < 0)) + 
    scale_size_manual() + 
#facet_grid(rows = vars(test_type), cols = vars(dataset),
#           scales = "free", space = "free") +
scale_x_reordered() +
      theme_bw()  + scale_size_continuous(range=c(1,4)) +
  #scale_colour_gradient2(low = "lightskyblue", mid = "lightsteelblue2", high = "salmon") + 
 theme(axis.text.x = element_blank(),
       axis.ticks.x = element_blank()) + NoLegend() + ylab("") + xlab("") +
    ggtitle(i)
    plot_list[[i]]  <- p
}  
    
pdf(file = paste0("../figures/DE_genes/pdf/",j,".pdf"), width = 15, height = 18)
title <- ggdraw() + 
  draw_label(
   j,
    fontface = 'bold',
    x = 0,
    hjust = 0
  ) +
  theme(
    # add margin on the left of the drawing canvas,
    # so title is aligned with left edge of first plot
    plot.margin = margin(0, 0, 0, 7)
  )
plot_grid(
  title, cowplot::plot_grid(plotlist = plot_list, ncol = 7) ,
  ncol = 1,
  # rel_heights values control vertical title margins
  rel_heights = c(0.05,1)
)
dev.off()
    



In [ ]:
all_fcs2  %>% dplyr::filter(Comparison == j)  %>% 
  ggplot(aes(x = avg_log2FC, y = -log10(p_val_adj), color = Dataset)) + 
  geom_point() + 
        theme_minimal() +
        #geom_text_repel() +
        #scale_color_manual(values=c("blue", "black", "red")) +
        geom_vline(xintercept=c(-0.6, 0.6), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red")

In [ ]:
library(ggrepel)
# plot adding up all layers we have seen so far
       

In [ ]:
head(all_fcs2)

In [ ]:
gc()

In [ ]:
rm(all_fcs)

In [ ]:
ls()

In [ ]:
all_fcs3  <- all_fcs2  %>% 
dplyr::filter(avg_log2FC)>0  %>% 
mutate(avg_log2FC2 = ifelse(direction == "Down",avg_log2FC*-1,avg_log2FC))  %>% 
mutate(abs_avg_log2FC2 = abs(avg_log2FC2))  %>% 
mutate(label = ifelse(abs_avg_log2FC2>0.9&p_val_adj<0.05,gene,NA_character_))  %>% 
mutate(color = ifelse(abs_avg_log2FC2>0.6&p_val_adj<0.05,Dataset,"grey"))       

In [ ]:
all_fcs3  %>% head

In [ ]:
all_fcs3  <- all_fcs2  %>% 
dplyr::filter(avg_log2FC>0 & !grepl(gene, pattern = "^MTRN"))  %>% 
mutate(avg_log2FC2 = ifelse(direction == "Down",avg_log2FC*-1,avg_log2FC))  %>% 
mutate(abs_avg_log2FC2 = abs(avg_log2FC2))  %>% 
mutate(label = ifelse(abs_avg_log2FC2>1&p_val_adj<0.05,gene,
                     ifelse(abs_avg_log2FC2>0.6&p_val_adj<10e-100,gene,NA_character_)))  %>% 
mutate(color = ifelse(abs_avg_log2FC2>0.6&p_val_adj<0.05,Dataset,"grey"))       

In [ ]:
all_fcs3  %>% head

In [ ]:
all_fcs3  %>% dplyr::filter(Comparison == j)  %>% 
  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  geom_point() + 
        theme_minimal() +
        geom_text_repel() +
        #scale_color_manual(values=c("blue", "black", "red")) +
        geom_vline(xintercept=c(-0.6, 0.6), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red")

In [ ]:
all_fcs3  %>% dplyr::filter(Comparison == j)  %>% 
  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  geom_point() + 
        theme_minimal() +
        geom_text_repel() +
        #scale_color_manual(values=c("blue", "black", "red")) +
        geom_vline(xintercept=c(-0.6, 0.6), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red")

In [ ]:
all_fcs3$Dataset  %>% table  %>% length

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 7)
all_fcs3  %>% dplyr::filter(Comparison == j)  %>% 
  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label, shape = color)) + 
  geom_point() + 
        theme_minimal() +
        geom_text_repel() +
        scale_color_manual(values=c(scales::hue_pal()(17), "grey88")) +
        geom_vline(xintercept=c(-0.6, 0.6), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25))

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 7)
all_fcs3  %>% dplyr::filter(Comparison == j)  %>% 
  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label, shape = color)) + 
  geom_point() + 
        theme_minimal() +
        geom_text_repel() +
        scale_color_manual(values=c(scales::hue_pal()(17), "grey88")) +
        geom_vline(xintercept=c(-0.6, 0.6), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25))

In [ ]:
all_fcs3$cd4_vs_cd8  <- substr(all_fcs3$Dataset,1,3)

In [ ]:
j

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 6)
df10  <- all_fcs3  %>% dplyr::filter(Comparison == j) 
df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label, shape = color)) + 
  geom_point() + 
        theme_minimal() +
        geom_text_repel(size = 6) +
facet_wrap(~cd4_vs_cd8) +
        scale_color_manual(values=c(scales::hue_pal()(length(levels(factor(df10$color)))-1), "grey88")) +
        geom_vline(xintercept=c(-0.6, 0.6), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()

In [ ]:
Comparison_list

In [ ]:
all_fcs3  <- all_fcs2  %>% 
dplyr::filter(avg_log2FC>0 & !grepl(gene, pattern = "^MTRN"))  %>% 
mutate(avg_log2FC2 = ifelse(direction == "Down",avg_log2FC*-1,avg_log2FC))  %>% 
mutate(abs_avg_log2FC2 = abs(avg_log2FC2))  %>% 
mutate(label = ifelse(abs_avg_log2FC2>0.9&p_val_adj<0.05,gene,
                     ifelse(abs_avg_log2FC2>0.6&p_val_adj<10e-100,gene,NA_character_)))  %>% 
mutate(color = ifelse(abs_avg_log2FC2>0.6&p_val_adj<0.05,Dataset,"grey"))       

In [ ]:
all_fcs3$cd4_vs_cd8  <- substr(all_fcs3$Dataset,1,3)

In [ ]:
options(repr.plot.width = 22, repr.plot.height = 8)

for(j in Comparison_list){
    df10  <- all_fcs3  %>% dplyr::filter(Comparison == j) 
p  <- df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2)) + 
        theme_minimal() +
        geom_text_repel(size = 6) +
facet_wrap(~cd4_vs_cd8) +
        scale_color_manual(values=c(scales::hue_pal()(length(levels(factor(df10$color)))-1), "grey88")) +
        geom_vline(xintercept=c(-0.6, 0.6), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()
    print(p)
}

In [ ]:
for(j in Comparison_list){
    df10  <- all_fcs3  %>% dplyr::filter(Comparison == j) 
 print(df10  %>%  group_by(Dataset)  %>% tally)
 
}

### Correct Colors

In [ ]:
colors_of_populations = c("Mazda RX4" = "red", "Mazda RX4 Wag" = "blue", "Datsun 710" = "green")

In [ ]:
levels(factor(all_fcs3$Dataset))

In [ ]:
all_fcs3  <- all_fcs2  %>% 
dplyr::filter(avg_log2FC>0 & !grepl(gene, pattern = "^MTRN"))  %>% 
mutate(avg_log2FC2 = ifelse(direction == "Down",avg_log2FC*-1,avg_log2FC))  %>% 
mutate(abs_avg_log2FC2 = abs(avg_log2FC2))  %>% 
mutate(label = ifelse(abs_avg_log2FC2>0.585&p_val_adj<0.05,gene,
                     ifelse(abs_avg_log2FC2>0.585&p_val_adj<10e-100,gene,NA_character_)))  %>% 
mutate(color = ifelse(abs_avg_log2FC2>0.585&p_val_adj<0.05,Dataset,"grey"))   %>% 
mutate(cd4_vs_cd8 = substr(Dataset,1,3))

In [ ]:
colors_of_populations = c(
    'cd4_l1_full' = "#ffa66aff",
    'cd4_l2_subcluster'  = "#ffa66aff",
    'cd4_l2_unc'  = "#cd3333ff",
    'cd4_l3_isaghi'  = "#a46464ff",
    'cd4_l3_naive'  = "#a58413ff",
    'cd4_l3_nfkb'  = "#a46c43ff",
    'cd4_l3_proliferating'  = "#9b3097ff", 
    'cd4_l3_temra'  = "#7d252aff",
    'cd4_l3_tfh'  = "#8e6a1dff",
    'cd4_l3_th1th17'  = "#a94e4eff",
    'cd4_l3_th2'  = "#b36672ff",
    'cd4_l3_treg'  = "#ba6d8dff",
    'cd8_l1_full'  = "#67934aff",
    'cd8_l2_nk'  = "#105292ff",
    'cd8_l2_subcluster' = "#67934aff", 
    'cd8_l2_unc'  = "#6d00c0ff",
    'cd8_l3_naive'  = "#004455ff",
    'cd8_l3_prolif'  = "#5f711aff",
    'cd8_l3_tcm'  = "#427c39ff",
    'cd8_l3_tem'  = "#004455ff",
    'cd8_l3_temra'  = "#16481fff",
    "grey" = "grey88"
)

In [ ]:
for(j in Comparison_list){
    df10  <- all_fcs3  %>% dplyr::filter(Comparison == j & !(Dataset %in% c("cd4_l1_full", "cd8_l2_subcluster", 
                                                                            "cd4_l2_subcluster", "cd8_l1_full",
                                                                           "cd8_l2_nk"))) 
p  <- df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2)) + 
        theme_minimal() +
        geom_text_repel(size = 6) +
facet_wrap(~cd4_vs_cd8) +
        scale_color_manual(values=colors_of_populations) +
        geom_vline(xintercept=c(-0.585, 0.585), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()
    print(p)
}

In [ ]:
all_fcs4  <- all_fcs2  %>% 
dplyr::filter(avg_log2FC>0 & !grepl(gene, pattern = "^MTRN"))  %>% 
mutate(avg_log2FC2 = ifelse(direction == "Down",avg_log2FC*-1,avg_log2FC))  %>% 
mutate(abs_avg_log2FC2 = abs(avg_log2FC2))  %>% 
mutate(label = ifelse(abs_avg_log2FC2>0.322&p_val_adj<0.05,gene,
                     ifelse(abs_avg_log2FC2>0.322&p_val_adj<10e-100,gene,NA_character_)))  %>% 
mutate(color = ifelse(abs_avg_log2FC2>0.322&p_val_adj<0.05,Dataset,"grey"))   %>% 
mutate(cd4_vs_cd8 = substr(Dataset,1,3))

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

for(j in Comparison_list){
    df10  <- all_fcs4  %>% dplyr::filter(Comparison == j & (Dataset %in% c("cd4_l1_full", "cd8_l2_subcluster", 
                                                                          "cd8_l2_nk"))) 
p  <- df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2)) + 
        theme_minimal() +
        geom_text_repel(size = 6) +
        scale_color_manual(values=colors_of_populations) +
        geom_vline(xintercept=c(-0.322, 0.322), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()
    print(p)
}

### Without Y genes

In [ ]:
library(biomaRt)
mart <- useMart(biomart="ensembl", dataset="hsapiens_gene_ensembl")

In [ ]:
results <- getBM(attributes = c("chromosome_name", "hgnc_symbol"),
           filters = "chromosome_name", values = "Y", mart = mart)

In [ ]:
results

In [ ]:
all_fcs4  <- all_fcs2  %>% 
dplyr::filter(avg_log2FC>0 
              & !grepl(gene, pattern = "^MTRN")
             & !grepl(gene, pattern = "^HLA-")
             & !grepl(gene, pattern = "orf")
             & !grepl(gene, pattern = "^AP0")
             & !grepl(gene, pattern = "^H1-"))  %>% 
dplyr::filter(!(gene %in% results$hgnc_symbol))  %>% 
mutate(avg_log2FC2 = ifelse(direction == "Down",avg_log2FC*-1,avg_log2FC))  %>% 
mutate(abs_avg_log2FC2 = abs(avg_log2FC2))  %>% 
mutate(label = ifelse(abs_avg_log2FC2>0.322&p_val_adj<0.05,gene,
                     ifelse(abs_avg_log2FC2>0.322&p_val_adj<10e-100,gene,NA_character_)))  %>% 
mutate(color = ifelse(abs_avg_log2FC2>0.322&p_val_adj<0.05,Dataset,"grey"))   %>% 
mutate(cd4_vs_cd8 = substr(Dataset,1,3))

In [ ]:
all_fcs4  %>% dplyr::filter(gene == "ASCL2")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

for(j in Comparison_list){
    df10  <- all_fcs4  %>% dplyr::filter(Comparison == j & (Dataset %in% c("cd4_l1_full", "cd8_l2_subcluster"))) 
p  <- df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2), dpi = 300) + 
        theme_minimal() +
        geom_vline(xintercept=c(-0.322, 0.322), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +geom_text_repel(size = 3) +
        scale_color_manual(values=colors_of_populations) +
        scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()
    print(p)
    ggsave(paste0("../figures/volcano/cd4_and_cd8_",j,".svg"), width = 10, height = 6)
}

In [ ]:
library(ggrepel)

In [ ]:
all_fcs5  <- all_fcs2  %>% 
dplyr::filter(avg_log2FC>0 
              & !grepl(gene, pattern = "^MTRN")
             & !grepl(gene, pattern = "^HLA-")
             & !grepl(gene, pattern = "orf")
             & !grepl(gene, pattern = "^AP0")
             & !grepl(gene, pattern = "^H1-"))  %>% 
dplyr::filter(!(gene %in% results$hgnc_symbol))  %>% 
mutate(avg_log2FC2 = ifelse(direction == "Down",avg_log2FC*-1,avg_log2FC))  %>% 
mutate(abs_avg_log2FC2 = abs(avg_log2FC2))  %>% 
mutate(label = ifelse(abs_avg_log2FC2>0.2&p_val_adj<0.05,gene,
                     ifelse(abs_avg_log2FC2>0.2&p_val_adj<10e-100,gene,NA_character_)))  %>% 
mutate(color = ifelse(abs_avg_log2FC2>0.2&p_val_adj<0.05,Dataset,"grey"))   %>% 
mutate(cd4_vs_cd8 = substr(Dataset,1,3))

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

for(j in Comparison_list){
    df10  <- all_fcs5 %>% dplyr::filter(Comparison == j & (Dataset %in% c("cd4_l3_treg"))) 
p  <- df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2), dpi = 300) + 
        theme_minimal() +
        geom_vline(xintercept=c(-0.2, 0.2), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +geom_text_repel(size = 6, max.overlaps = 15) +
        scale_color_manual(values=c("#ff5599ff","grey88")) +
        scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()
    print(p)
    ggsave(paste0("../figures/volcano/treg_",j,".svg"), width = 10, height = 6)
}

In [ ]:
for(j in Comparison_list){
    df10  <- all_fcs5 %>% dplyr::filter(Comparison == j & (Dataset %in% c("cd8_l2_unc"))) 
p  <- df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2), dpi = 300) + 
        theme_minimal() +
        geom_vline(xintercept=c(-0.2, 0.2), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +geom_text_repel(size = 6) +
        scale_color_manual(values=c("#306aa3ff","grey88")) +
        scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()
    print(p)
    ggsave(paste0("../figures/volcano/unc_",j,".svg"), width = 10, height = 6)
}

In [ ]:
cd4  <- readRDS("../data/processed/L1/cd4_l1_full_filt.rds")

avgexp = AggregateExpression(subset(cd4, Experiment_ID %in% c("Exp16","Exp18","Exp19","Exp20")), 
                             return.seurat = F, group.by = "Patient_Time", 
                          assay = "RNA")

In [ ]:
avgexp  <- NormalizeData(avgexp)
avgexp  <- ScaleData(avgexp)

In [ ]:
dir.create("../figures/volcano")

In [ ]:
options(repr.plot.width = 22, repr.plot.height = 8)

for(j in Comparison_list){
    df10  <- all_fcs4  %>% dplyr::filter(Comparison == j & !(Dataset %in% c("cd4_l1_full", "cd8_l2_subcluster", 
                                                                            "cd4_l2_subcluster", "cd8_l1_full",
                                                                           "cd8_l2_nk"))) 
p  <- df10  %>%  ggplot(aes(x = avg_log2FC2, y = -log10(p_val_adj), color = color, label = label)) + 
  ggrastr::rasterize(geom_point(aes(shape = color), size = 2)) + 
        theme_minimal() +
        geom_text_repel(size = 6) +
facet_wrap(~cd4_vs_cd8) +
        scale_color_manual(values=colors_of_populations) +
        geom_vline(xintercept=c(-0.322, 0.322), col="red") +
        geom_hline(yintercept=-log10(0.05), col="red") +
    
    scale_shape_manual(values = c(21:25,21:25,21:25,21:25)) + 
ggtitle(j) + ggtheme()
    print(p)
    ggsave(paste0("../figures/volcano/all_populations_",j,".svg"), width = 17, height = 6)
}

## DESeq gene markers

In [ ]:
all_markers

In [ ]:
deseq_markers  <- read_csv("../../240218_VN_Diabetes_V05/tables/DESeq_markers/cd4_deseq.csv")

In [ ]:
deseq_markers_cd4  <- deseq_markers

In [ ]:
deseq_markers  <- read_csv("../tables/DESeq_markers/cd8_deseq.csv")

In [ ]:
deseq_markers_cd8  <- deseq_markers

In [ ]:
deseq_markers  <- rbind(deseq_markers_cd4, deseq_markers_cd8)

In [ ]:
deseq_markers$`...1`  <- NULL

In [ ]:
deseq_markers

In [ ]:
deseq_markers  <- deseq_markers  %>% mutate(
direction = case_when(test_type == "Dia T1 vs Ctrl T0" & cluster == "Ctrl T0" ~ "Down",
                      test_type == "Dia T1 vs Ctrl T0" & cluster == "Dia T1" ~ "Up",
                      
                      test_type == "Dia T0 vs Ctrl T0" & cluster == "Ctrl T0" ~ "Down",
                      test_type == "Dia T0 vs Ctrl T0" & cluster == "Dia T0" ~ "Up",
                      
                      test_type == "Dia T0 vs Dia T1" & cluster == "Dia T0" ~ "Up",
                      test_type == "Dia T0 vs Dia T1" & cluster == "Dia T1" ~ "Down",
                      
                      test_type == "PR_0 T0 vs PR_0 T1" & cluster == "PR_0 T0" ~ "Up",
                      test_type == "PR_0 T0 vs PR_0 T1" & cluster == "PR_0 T1" ~ "Down",
                      
                      test_type == "PR_0 T0 vs PR_1 T0" & cluster == "PR_0 T0" ~ "Up",
                      test_type == "PR_0 T0 vs PR_1 T0" & cluster == "PR_1 T0" ~ "Down",
                      
                      test_type == "PR_0 T1 vs PR_1 T1" & cluster == "PR_0 T1" ~ "Up",
                      test_type == "PR_0 T1 vs PR_1 T1" & cluster == "PR_1 T1" ~ "Down",
                      
                      test_type == "PR_1 T0 vs PR_1 T1" & cluster == "PR_1 T0" ~ "Up",
                      test_type == "PR_1 T0 vs PR_1 T1" & cluster == "PR_1 T1" ~ "Down",
                      
                      test_type == "Keto_1 T0 vs Keto_0 T0" & cluster == "Keto_0 T0" ~ "Down",
                      test_type == "Keto_1 T0 vs Keto_0 T0" & cluster == "Keto_1 T0" ~ "Up"
                      
                     ))

In [ ]:
deseq_markers  %>% group_by(cluster, test_type, direction)  %>% tally

In [ ]:
deseq_markers$gene_direction  <- paste(deseq_markers$gene, deseq_markers$direction)

In [ ]:
colnames(deseq_markers)

In [ ]:
colnames(all_markers)

In [ ]:
mrk_sc_deseq  <- rbind(deseq_markers, all_markers)